In [132]:
import numpy as np
import time
from heapq import heappush, heappop


def buildUndirectedWeightedGraph(filename):
    with open(filename,'r') as f:
        lines = f.readlines()
        G = {}
        for line in lines:
            splitVals = line.split()
            edges = {}
            node = int(splitVals[0])
            for splitVal in splitVals[1:]:
                edge,weight = splitVal.split(',')
                edges.update({(node,int(edge)):int(weight)})
            G.update({node:edges})
    return G

def dijkstrasAlgorithm(G,source,destinations,defaultDistance=np.inf):

    # Initialize
    seen = [source]
    pathLengths = {node:defaultDistance for node in G.keys()}
    pathLengths.update({source:0})

    # dijkstra's algorithm
    # find next edges, pick the min path of pathlengths[node] + G[node][edge]
#     edgesAvailable = len([edge for node in seen for edge in G[node] if edge[1] not in seen])
    while len(seen) < len(G):
        currentMin = defaultDistance
        for node in seen:
            for edge in G[node]:
                if edge[1] not in seen:
                    pathLength = pathLengths[node]+G[node][edge]
                    if pathLength < pathLengths[edge[1]]:
                        pathLengths.update({edge[1]:pathLength})
                    if pathLength < currentMin:
                        currentMin = pathLength
                        minEdge = edge
        seen.append(minEdge[1])
#         edgesAvailable = len([edge for node in seen for edge in G[node] if edge[1] not in seen])
    return [pathLengths[destination] for destination in destinations]


def dijkstrasAlgorithmHeap(G,source,destinations,defaultDistance=np.inf):

    # Initialize
    pathLengths = {node:defaultDistance for node in G}
    pathLengths[source] = 0 # Stores distance to
    seen = set() # Tracks seen nodes 
    heapMinDis = [(0,source)] # Used to quickly find min distance of explored nodes
    
    # While reachable nodes unexplored
    while heapMinDis:
        # Take a look at the next node
        currentLength,node = heappop(heapMinDis)
        
        # Skip if seen before
        if node in seen:
            continue
        seen.add(node)
        
        # Loop through edges to unseen nodes
        for edge,edgeLength in G[node].items():
            # Skip if seen before
            if edge[1] in seen:
                continue
                
            # Update best path and add to the heap to keep track of index
            pathLength = currentLength+edgeLength
            if pathLength < pathLengths[edge[1]]:
                pathLengths[edge[1]] = pathLength 
                heappush(heapMinDis,(pathLength,edge[1]))

    return [pathLengths[destination] for destination in destinations]


In [39]:

# This was for a leet code question on max path
grid = [[1,3,1],[1,5,1],[4,2,1]]
grid = [[0,3,8,5,5,1,5,3,3,6,7,5,9,4,6,5,6,8,2,4,2,5,5,3,7,4,8,5,0,2,5,2,5,5,7,2,6,3,5,1,9,4,8,8,9,5,6,7,5,0,3,9,6,9,2,5,5,2,0,8,4,7,0,2,2,0,4,1,3,2,2,4,6,1,5,7,8,1,7,3,1,0,4,9,1,6,4,9,8,6,7,2,3,9,6,8,9,3,9,4,7,3,9,3,6,4,3,4,5,2,9,8,2,3,8,4,9,4,3,9,1,2,4,4,1,0,4,3,5,5,7,2,9,6,8,5,0,1,2,7,3,1,8,7,5,8,1,6,2,6,8,3,8,4,0,2,6,4,5,2,0,2,4,9,1,2,6,3,5,8,3,0,2,1,8,9,9,1,5,1,8,5,5,8,9,5,0,6,1,7,8,1,2,4,2,3,7,9,8,2],[9,1,3,1,4,1,0,3,4,8,7,2,3,4,1,5,1,3,3,2,1,3,6,3,7,8,8,4,9,6,9,9,0,2,0,4,5,1,0,9,9,7,4,4,1,7,1,4,0,4,8,3,7,4,6,5,3,4,1,4,3,0,3,3,4,4,7,1,7,7,1,8,6,7,2,9,4,4,3,4,0,2,0,8,8,6,5,1,3,6,6,8,9,1,1,3,5,0,5,4,0,8,2,8,5,7,8,1,3,3,7,3,7,7,3,6,4,8,9,9,7,5,7,6,9,0,9,6,2,6,1,4,6,3,3,1,2,3,2,5,8,2,1,6,9,4,2,5,3,1,4,0,9,3,8,8,5,7,4,8,6,5,4,2,1,9,6,5,4,0,1,3,2,2,9,2,6,1,7,1,4,4,3,3,9,1,3,5,1,8,5,7,5,9,9,8,9,5,4,3],[8,5,8,0,9,7,4,7,0,2,9,5,8,2,0,7,6,4,2,7,2,7,4,9,9,5,8,0,3,2,5,1,9,4,1,8,1,6,7,4,0,8,9,8,3,9,7,9,5,0,6,7,9,0,7,8,7,7,8,0,9,6,1,8,0,5,8,3,3,5,7,3,6,8,3,9,0,0,8,5,2,4,3,2,6,2,2,3,9,3,4,0,9,7,0,9,2,8,4,5,5,2,0,1,0,3,0,2,6,0,8,8,6,3,2,2,7,7,8,6,0,4,8,1,1,8,2,6,6,6,3,3,0,4,7,1,9,9,5,7,0,3,6,6,6,0,1,3,7,9,1,9,5,9,0,6,9,4,4,7,1,0,1,1,4,0,4,5,9,0,3,1,3,1,0,2,1,3,7,1,4,1,2,9,0,3,7,0,7,2,7,0,2,0,4,8,0,8,5,2],[0,8,5,6,1,5,8,3,8,7,6,2,8,8,1,1,3,1,1,3,3,0,3,7,3,7,7,5,8,2,7,8,1,3,6,2,0,6,7,1,4,5,3,2,4,7,3,9,8,6,2,3,9,8,0,4,7,9,9,7,1,9,8,4,4,4,9,4,1,8,5,5,4,1,9,0,0,5,9,8,1,4,3,2,4,5,6,1,4,6,1,5,5,9,2,9,3,1,5,4,9,1,1,3,4,1,5,4,6,5,4,9,1,7,4,5,4,0,8,8,8,1,5,3,0,7,4,4,8,2,0,0,3,4,3,9,7,9,3,5,6,9,4,9,6,0,4,2,1,4,2,9,6,7,5,6,7,1,2,7,3,5,7,8,1,1,7,8,2,2,3,0,1,9,9,9,0,3,3,3,9,5,4,7,3,9,6,2,3,0,9,8,5,7,7,6,0,6,6,2],[1,1,2,4,1,3,4,3,8,9,8,7,5,2,5,0,4,1,2,7,1,1,5,9,8,4,7,8,3,6,2,4,7,6,8,0,9,4,5,7,4,3,7,1,6,4,1,2,7,3,9,0,4,6,9,5,3,9,3,6,5,8,0,4,4,0,5,6,7,0,5,3,4,2,4,2,8,5,4,5,8,5,6,4,3,7,9,6,6,5,2,3,3,4,8,9,7,3,5,4,5,3,7,1,7,3,3,6,8,9,1,6,4,7,0,8,5,0,4,3,5,9,7,0,5,5,9,2,0,7,8,5,2,7,7,9,0,2,5,8,2,7,6,8,4,9,6,1,1,3,5,8,4,4,8,9,1,9,4,1,8,2,8,0,0,7,0,0,0,5,1,4,2,7,2,9,6,1,2,7,6,7,5,0,3,5,1,4,5,5,5,3,0,6,6,0,3,6,2,5],[3,3,9,8,1,4,7,9,5,1,9,3,1,4,3,4,2,6,9,7,4,6,2,4,2,8,4,6,6,8,1,0,2,3,8,5,7,7,6,2,0,5,5,1,2,0,6,4,8,7,3,2,3,5,6,6,6,2,4,4,1,7,4,5,0,4,2,7,3,8,1,4,4,8,7,6,0,3,2,9,0,5,3,6,0,0,4,8,4,8,3,7,5,9,2,8,6,6,5,1,5,9,7,1,7,5,9,8,0,1,9,3,6,2,9,8,4,3,9,1,3,2,8,0,3,3,8,9,9,6,3,6,5,0,7,4,7,6,4,8,9,3,1,7,8,2,8,4,7,7,5,0,1,4,2,4,9,1,6,8,7,9,7,4,9,4,8,7,3,3,7,4,8,0,2,8,2,0,3,1,9,8,1,0,4,3,4,3,6,0,4,5,1,1,9,3,7,0,2,2],[3,9,7,3,9,1,2,3,3,5,4,2,5,7,2,0,2,6,3,9,9,7,4,2,0,6,5,8,6,7,2,1,8,9,4,9,2,6,2,5,3,8,7,9,5,9,1,0,8,4,9,9,4,5,1,4,3,9,4,9,6,7,0,7,8,7,6,1,5,1,8,9,1,8,0,7,9,1,9,7,7,0,8,3,7,2,0,1,1,4,0,9,3,3,6,4,0,5,7,5,6,5,4,7,5,6,6,5,9,5,4,7,7,3,0,5,5,2,6,6,9,6,7,2,9,4,8,1,9,5,7,7,3,1,6,0,0,5,5,9,2,0,8,0,3,1,5,0,3,3,8,2,1,5,7,1,1,5,4,0,3,1,9,8,5,8,8,5,3,6,4,7,6,5,7,1,6,4,1,9,7,9,2,9,6,9,2,8,6,6,0,1,8,0,9,3,0,0,0,5],[6,6,2,2,1,2,5,7,6,6,9,6,7,1,7,3,2,1,1,0,7,2,2,7,4,3,2,6,5,2,1,1,1,3,5,2,5,0,0,4,8,9,0,5,2,9,1,6,0,2,8,9,6,0,7,2,6,1,8,1,6,9,5,7,5,0,9,2,3,1,6,3,2,8,9,6,7,0,4,9,4,3,9,3,5,6,5,1,9,6,3,5,5,0,2,2,2,4,5,5,7,1,9,2,2,0,0,9,2,5,1,8,8,0,1,3,6,9,5,7,5,0,3,2,0,7,5,2,3,0,0,1,3,1,5,5,1,5,7,5,0,8,3,0,8,5,4,6,6,1,3,1,3,8,3,5,6,0,9,1,0,1,4,4,2,9,1,5,5,8,0,7,6,6,8,6,3,4,2,9,7,6,2,0,6,5,7,2,8,6,4,8,8,8,4,0,0,6,8,7],[4,0,4,3,6,4,9,9,8,4,8,5,2,0,7,8,8,4,3,6,3,9,6,1,7,1,3,9,9,1,6,5,2,3,8,8,7,0,0,8,6,0,5,8,1,5,8,1,9,1,9,4,0,5,5,0,8,1,9,7,2,8,3,6,1,3,7,0,5,7,0,3,9,6,1,2,3,0,3,4,3,4,9,4,0,4,4,0,7,5,8,0,3,3,8,6,8,7,9,6,4,9,9,4,7,3,8,0,5,2,5,8,6,4,4,8,0,8,9,0,6,9,0,1,4,0,8,2,8,7,0,4,8,2,0,6,5,9,8,2,3,3,0,1,9,5,0,2,5,1,2,1,0,4,5,4,6,3,8,4,2,1,1,0,3,1,8,0,2,7,2,5,2,4,7,2,9,9,6,7,0,0,0,2,4,5,6,0,0,6,7,2,7,0,5,2,1,3,2,6],[2,4,3,5,9,2,9,0,1,5,9,1,7,0,3,1,7,1,3,8,0,2,2,7,2,7,2,6,3,4,4,7,1,7,2,2,0,3,4,1,0,4,5,7,6,0,0,5,4,6,3,4,8,8,3,3,5,5,1,0,2,5,8,5,2,2,7,2,6,3,6,8,7,3,6,5,3,6,1,9,2,6,5,3,4,9,8,2,6,9,2,0,4,2,5,6,7,4,1,5,8,9,3,7,2,9,3,7,8,4,7,2,2,4,7,7,3,5,9,2,4,3,2,0,8,0,9,5,4,2,2,2,1,5,0,3,7,5,2,7,1,1,9,3,6,9,0,1,4,9,3,1,5,8,1,3,8,2,0,4,4,2,9,5,9,9,0,8,6,3,5,7,6,7,0,2,6,3,4,0,4,9,3,9,7,5,2,7,9,2,2,4,6,1,1,6,2,2,4,8],[7,2,7,5,9,9,8,5,2,4,7,7,3,1,6,3,8,1,2,7,3,4,1,2,7,5,0,1,9,6,1,8,8,0,3,7,0,1,4,2,7,2,9,3,3,6,6,1,9,0,8,2,7,2,6,4,9,6,8,8,3,1,6,1,2,9,1,2,3,5,4,0,7,6,5,0,4,3,3,3,4,4,7,1,6,4,7,7,2,5,5,5,9,1,9,1,2,2,3,5,7,9,8,7,5,3,9,9,7,5,4,1,9,2,2,7,6,1,4,0,9,9,6,8,2,7,9,4,9,4,0,8,3,0,7,9,5,9,0,2,4,5,5,3,9,9,0,7,1,4,7,0,5,5,8,7,2,9,1,3,3,3,4,6,5,3,7,1,2,0,3,6,7,1,9,6,0,1,3,3,5,2,3,0,8,3,9,2,2,1,6,5,6,2,4,2,5,1,3,0],[3,8,6,0,9,8,6,0,1,1,5,7,4,1,9,4,4,1,6,9,2,4,6,0,6,0,2,4,4,5,4,7,6,2,8,5,2,6,7,4,0,3,3,4,4,4,0,0,5,6,1,9,1,8,0,9,0,4,5,6,0,1,4,8,6,4,3,8,2,1,4,2,6,7,8,2,4,8,4,9,5,6,9,8,6,1,7,6,5,5,5,7,6,1,5,2,5,9,3,7,2,7,0,0,7,0,2,1,9,6,0,4,4,1,2,0,4,1,9,0,6,4,9,5,5,5,7,2,6,2,1,0,0,3,0,7,4,4,8,3,2,0,9,7,2,1,7,6,4,8,8,3,2,8,8,9,5,7,1,3,0,3,5,0,6,5,7,0,1,7,5,3,7,4,0,1,7,0,0,2,8,0,5,3,0,3,2,7,0,6,0,0,1,5,2,7,2,1,0,3],[8,7,7,8,2,9,1,9,9,1,1,8,4,8,1,4,3,5,2,6,1,2,8,2,0,1,2,2,4,4,8,3,1,5,3,5,4,4,5,6,8,8,4,2,9,7,6,2,2,0,8,6,5,7,0,5,0,4,9,4,8,7,9,2,4,4,7,1,9,4,7,9,3,3,3,2,0,1,6,4,2,7,2,7,4,3,2,6,7,3,0,8,1,2,2,5,6,1,8,7,6,5,6,1,8,9,5,0,1,1,7,5,8,9,2,4,2,6,2,2,9,3,0,2,7,2,0,3,3,8,3,1,6,1,2,6,1,7,7,4,1,4,1,1,5,3,6,8,1,8,0,2,3,2,5,2,4,5,6,9,5,1,1,1,2,5,8,5,3,7,1,6,3,2,7,8,7,5,6,8,4,8,1,9,0,8,2,6,5,0,8,0,1,9,4,3,6,2,1,1],[1,2,7,4,5,5,2,2,0,1,3,6,9,6,8,2,4,0,8,1,2,6,1,5,7,7,0,4,9,1,5,2,4,5,6,1,2,9,5,2,2,0,1,1,6,9,5,2,1,4,3,5,2,7,0,2,4,2,8,6,6,5,8,0,0,5,3,2,6,0,7,0,1,8,1,7,9,9,2,2,5,5,7,7,2,9,1,9,1,9,5,7,7,5,9,7,0,4,2,8,5,9,8,6,9,2,5,8,1,7,2,6,3,9,5,7,0,9,6,1,8,3,1,5,9,2,5,1,9,7,2,4,6,0,0,7,4,7,7,7,5,9,5,8,8,1,7,0,2,6,3,0,9,4,8,0,9,3,2,8,2,4,4,0,6,6,7,3,3,4,0,8,3,6,8,3,9,6,3,1,2,6,3,3,3,3,6,4,8,0,2,0,4,6,2,2,2,9,7,5],[5,8,6,0,6,4,3,7,0,8,0,4,7,5,0,2,9,6,6,7,6,8,0,2,6,2,6,0,4,4,7,9,4,3,2,2,8,7,9,0,6,1,7,5,6,7,7,7,5,3,7,3,3,7,5,1,1,3,1,7,7,0,9,3,6,1,5,6,8,4,6,6,7,5,3,4,4,0,1,1,5,0,4,8,7,1,1,1,5,4,8,2,5,7,8,1,0,3,7,1,0,5,7,7,3,3,3,7,3,7,9,9,7,5,9,7,7,1,8,2,7,8,6,2,8,4,5,0,8,4,1,0,2,1,7,5,4,3,4,9,0,5,0,7,1,0,6,8,3,6,2,0,5,8,5,5,5,0,5,5,7,9,5,9,0,4,6,4,7,2,3,7,8,6,7,1,6,3,1,1,2,5,3,9,3,8,4,0,1,1,5,8,0,0,9,0,5,5,6,4],[9,2,2,7,8,1,0,6,6,3,7,8,8,0,7,2,1,3,4,2,5,0,2,5,2,1,8,9,8,6,4,7,8,8,7,8,1,7,4,7,3,3,6,1,4,5,5,5,9,0,9,6,2,1,3,4,2,1,6,2,8,2,9,6,2,8,7,3,6,1,0,1,5,8,2,1,6,8,8,7,8,7,3,0,0,6,6,2,8,4,6,8,6,5,6,0,4,3,3,2,5,6,3,2,4,5,3,0,5,1,9,3,0,2,5,0,9,4,4,9,8,0,7,7,5,5,7,1,9,3,5,6,9,0,8,5,8,1,8,3,4,7,7,4,0,4,4,1,8,8,0,9,8,9,8,5,4,7,9,3,2,4,9,1,5,9,7,3,0,5,8,4,2,7,8,4,2,4,5,2,2,7,3,2,6,3,8,3,1,9,6,3,3,8,7,0,7,4,5,0],[1,4,6,5,1,7,0,5,1,5,0,4,3,3,8,1,7,6,4,0,5,3,5,1,3,4,3,0,0,9,2,1,5,9,7,8,6,9,6,7,4,6,3,9,9,2,3,8,0,7,0,8,0,8,9,3,2,4,6,5,3,0,6,0,1,5,1,7,4,7,7,1,3,2,2,4,6,5,3,7,5,5,7,5,5,8,1,0,2,9,5,8,1,1,8,3,9,9,2,3,6,1,6,1,4,9,6,0,6,1,9,1,6,6,9,4,4,2,4,9,1,9,7,4,2,7,7,1,9,2,7,5,3,3,9,7,4,5,0,1,6,9,4,4,8,3,8,4,5,2,3,8,3,0,3,6,8,2,9,7,4,6,4,8,2,3,7,8,0,7,1,6,9,6,3,7,9,1,1,7,6,7,5,9,7,0,7,7,5,7,6,9,3,3,9,7,8,7,6,9],[4,7,7,5,3,0,2,5,4,6,2,0,3,9,1,2,2,9,0,7,6,8,8,1,3,8,9,2,7,5,3,3,4,2,9,8,3,1,5,9,7,9,9,2,0,2,5,2,1,7,9,9,5,0,1,9,8,2,3,7,7,8,0,1,0,9,1,5,3,8,4,2,7,5,5,8,8,0,2,1,9,2,1,4,2,4,5,2,6,0,9,5,8,9,8,9,1,0,6,4,0,1,6,8,8,3,8,6,5,0,0,4,4,3,9,8,7,6,0,5,7,1,0,5,3,8,6,4,0,3,8,1,6,6,1,4,0,9,3,5,1,3,0,6,8,1,4,7,7,7,2,6,8,2,2,3,2,0,7,5,3,7,6,9,6,9,6,8,0,9,3,1,4,5,9,2,8,6,9,8,5,3,4,5,5,8,9,9,9,8,4,4,6,2,6,2,3,2,2,3],[3,7,7,7,3,6,1,3,4,2,1,9,5,8,7,2,8,6,1,9,4,8,4,2,2,0,4,6,4,6,9,7,4,6,6,9,5,9,4,1,1,8,3,6,6,0,8,6,6,1,6,0,9,0,3,4,2,9,0,8,8,9,7,4,8,5,5,5,4,9,6,5,9,9,3,7,9,1,4,7,4,0,0,6,2,5,0,6,6,0,4,4,1,1,0,9,6,7,6,0,9,3,7,8,4,0,6,6,3,0,3,9,2,5,5,4,0,5,2,7,7,6,3,9,9,6,0,5,3,7,7,2,2,6,1,6,8,9,2,3,1,8,2,3,3,8,9,6,5,1,3,5,9,8,4,8,4,6,5,8,3,4,2,5,2,5,4,0,4,8,3,5,8,7,0,4,5,1,0,3,4,5,8,3,3,4,3,0,0,0,0,6,4,2,3,6,8,9,8,2],[8,3,0,8,1,2,2,8,6,4,1,0,9,1,6,5,5,1,5,8,2,5,6,6,9,9,5,7,1,3,2,9,7,4,9,0,8,2,8,4,6,0,7,8,3,3,3,1,6,0,9,8,7,7,7,6,8,2,6,9,7,0,0,6,4,0,6,4,2,5,9,0,7,6,0,0,1,3,1,7,5,2,6,2,9,5,1,8,7,9,7,6,9,8,3,5,0,1,9,4,8,0,4,5,6,5,8,9,0,9,9,8,4,7,0,3,2,3,3,9,4,3,7,3,3,2,0,3,4,2,7,2,2,3,0,1,8,8,2,1,9,1,9,3,8,1,9,0,7,2,1,1,7,9,7,0,1,7,3,7,1,0,0,6,4,2,7,4,2,9,5,1,3,6,7,1,8,6,4,5,0,5,8,8,6,5,8,8,5,4,7,6,4,7,2,0,1,1,5,3],[3,2,5,6,9,2,9,9,0,5,6,2,1,4,0,9,0,1,9,5,5,7,3,1,6,6,2,8,9,9,1,2,3,8,0,4,2,2,3,4,7,1,7,0,6,9,0,8,2,9,3,7,8,8,1,5,6,5,3,6,6,6,0,9,5,1,6,9,3,9,4,2,3,1,3,9,0,5,7,3,4,2,2,5,0,5,0,7,0,5,5,6,1,5,8,8,8,4,8,1,5,2,4,8,5,9,9,7,4,8,0,0,0,5,5,3,0,5,2,3,0,7,1,4,2,1,2,1,5,2,4,3,4,0,1,1,1,3,9,5,1,1,6,4,8,3,7,1,1,9,4,1,6,7,7,0,9,2,3,4,4,0,9,1,2,1,2,4,6,1,9,7,5,7,1,3,1,8,4,2,9,8,5,7,6,5,0,7,7,3,1,1,5,1,2,8,4,7,2,2],[8,3,9,3,3,3,9,4,3,3,8,3,4,5,2,0,0,2,9,9,8,0,1,3,3,5,1,9,2,5,1,3,9,3,8,2,8,7,8,1,1,8,6,5,5,9,7,8,3,6,7,1,8,8,5,4,4,8,5,8,4,9,1,5,2,0,7,2,7,7,3,0,7,2,5,2,3,2,0,6,0,8,8,9,8,5,5,4,3,0,3,9,9,6,4,3,6,3,5,6,0,9,6,7,3,2,2,6,6,2,2,9,2,2,8,3,9,5,7,3,5,2,4,5,9,9,8,5,2,6,1,5,5,0,4,0,2,6,6,0,1,0,9,5,3,9,8,2,4,6,7,2,8,2,7,7,3,7,5,5,3,8,2,0,8,7,0,2,5,8,5,6,9,4,4,2,6,2,6,2,8,6,4,9,0,3,8,3,1,5,8,6,4,1,9,4,0,9,7,5],[8,4,4,7,0,8,1,6,2,9,9,1,7,5,2,7,9,0,0,2,6,9,0,2,2,9,6,4,9,5,1,9,9,7,8,0,7,1,8,0,2,9,3,0,5,5,7,6,7,8,8,3,9,8,7,3,0,6,7,1,1,8,0,3,6,0,5,5,1,5,5,5,5,8,5,2,3,3,8,3,3,8,8,2,8,6,7,8,2,4,9,5,4,9,0,2,1,5,8,4,1,3,0,8,4,5,0,9,0,8,2,5,8,1,9,6,7,6,5,1,2,4,6,7,6,9,1,7,4,9,2,7,5,4,7,9,9,7,0,2,7,3,7,5,4,7,2,3,3,7,4,6,3,2,5,9,3,6,9,0,8,1,7,5,7,5,6,6,4,6,0,2,9,8,7,5,5,1,0,0,0,6,8,4,1,5,3,4,4,4,4,2,7,4,9,4,1,5,1,5],[3,1,9,5,1,9,0,8,0,1,1,1,9,9,7,0,7,2,5,1,7,9,5,4,5,4,9,6,9,2,2,2,5,3,7,9,2,0,7,5,3,8,8,2,0,5,3,7,9,8,0,6,9,5,3,5,9,4,1,0,6,5,2,3,1,2,4,5,4,2,0,7,2,0,1,4,5,4,3,5,4,3,3,6,8,6,3,7,0,6,9,8,4,4,4,5,6,8,0,2,2,3,1,7,3,2,1,9,9,5,6,5,0,1,3,1,8,6,0,0,3,0,1,7,4,7,4,0,5,4,4,0,7,5,7,3,9,8,4,0,5,0,8,8,3,1,9,3,8,9,4,3,9,7,2,3,4,6,5,1,2,9,1,2,6,8,5,8,9,9,8,4,1,6,2,4,8,3,8,8,3,4,3,2,3,5,8,7,3,3,8,5,5,0,9,1,0,4,1,9],[5,0,4,6,8,8,3,8,2,3,6,7,7,1,9,0,6,7,9,1,3,7,9,8,9,8,1,0,5,3,9,0,5,5,9,3,6,2,2,8,7,0,5,4,2,6,6,0,6,5,4,1,4,3,9,6,3,2,6,8,7,7,1,2,3,2,8,1,6,2,9,3,4,6,9,6,4,5,9,2,2,5,3,8,8,4,4,3,7,2,4,6,0,5,9,5,9,9,6,5,3,7,8,7,5,7,4,9,2,5,4,6,0,7,4,0,2,1,3,9,3,9,5,5,4,6,0,3,7,8,0,0,5,8,8,2,5,4,4,9,1,8,7,3,5,2,3,7,5,8,8,8,8,6,6,4,4,8,0,4,7,0,6,4,1,4,7,6,0,1,8,1,1,5,6,6,9,9,6,4,0,6,5,0,4,1,6,1,9,8,5,8,1,1,3,2,8,2,0,8],[5,8,2,6,4,8,2,5,0,0,0,2,9,7,4,3,8,0,4,9,9,1,8,2,5,3,4,5,5,4,3,0,3,7,6,9,8,0,4,8,3,6,2,2,3,6,5,1,8,2,3,9,5,1,1,0,4,5,5,9,0,1,1,5,0,9,4,8,9,0,8,2,7,2,4,0,8,2,4,7,6,7,6,1,8,0,4,4,5,1,5,7,4,6,2,5,7,8,5,6,9,6,1,8,8,7,8,9,9,2,8,5,1,6,9,1,6,3,5,2,6,2,1,1,0,4,6,7,4,3,4,5,9,7,3,0,4,2,9,4,6,9,1,8,5,2,9,4,7,7,8,4,9,9,5,0,5,1,9,0,6,3,5,6,0,9,8,5,3,7,1,1,8,4,1,3,7,3,7,4,2,5,8,1,5,3,3,0,6,3,2,3,8,8,1,1,9,9,8,4],[8,1,7,6,5,9,9,4,4,9,1,6,6,9,7,3,5,3,6,3,8,8,8,6,8,9,9,9,0,7,5,8,8,3,6,6,4,6,0,8,7,1,4,3,3,3,9,0,8,5,5,6,5,4,5,4,5,4,5,8,4,1,6,2,6,3,0,0,1,1,0,0,4,4,5,9,9,4,1,8,1,7,6,7,3,3,3,0,8,8,8,4,1,7,6,7,2,7,9,3,0,9,5,6,3,0,6,5,7,7,5,8,6,3,7,9,7,2,0,7,3,0,1,4,7,9,4,9,8,3,4,8,3,9,5,8,2,3,5,1,2,0,1,9,4,9,0,1,1,2,0,4,3,1,1,0,0,5,2,9,0,6,9,3,8,6,4,2,9,9,3,2,2,4,3,6,5,3,9,9,6,9,5,9,2,6,1,4,3,5,3,6,2,5,9,2,1,5,4,1],[7,9,5,9,5,8,5,1,3,6,0,9,7,7,0,1,6,4,5,9,9,9,5,3,6,7,7,7,4,1,0,1,2,5,0,8,5,7,9,9,5,1,0,2,0,1,3,6,5,9,8,6,0,5,0,6,4,7,5,9,1,6,0,5,3,3,3,1,2,4,0,8,7,0,0,8,3,6,4,0,7,2,7,9,8,9,5,2,6,2,3,9,8,6,5,4,9,0,5,1,5,5,9,2,7,2,0,3,0,7,3,9,1,0,8,9,9,5,4,8,7,7,9,8,5,6,2,4,7,7,8,2,4,9,6,1,3,9,4,5,6,0,4,7,0,2,9,2,9,5,2,9,4,3,7,0,0,9,6,7,8,4,9,2,6,5,5,9,6,2,7,2,2,1,2,4,6,3,8,5,8,2,6,4,6,3,4,6,4,1,5,2,7,6,4,3,3,2,5,0],[4,4,4,8,5,6,2,3,1,3,1,9,5,7,4,3,1,0,1,5,3,6,0,1,2,4,6,6,8,3,6,4,7,0,2,5,9,7,0,0,0,1,2,7,9,8,1,2,8,2,9,4,1,9,5,3,6,3,1,4,9,7,9,8,0,3,3,1,0,4,1,2,7,3,2,8,3,5,0,2,9,2,6,0,1,3,6,9,8,7,4,7,7,5,6,9,0,1,0,3,7,1,5,7,7,9,5,0,4,8,4,6,2,2,6,3,7,4,3,6,2,9,5,1,6,3,0,6,7,2,9,6,5,7,3,4,8,1,5,3,9,1,9,1,6,7,6,3,2,1,1,6,2,7,7,0,0,9,7,9,1,8,6,8,7,9,3,6,0,0,1,1,1,2,2,7,9,1,3,3,4,4,9,7,3,8,7,6,7,6,5,0,7,1,1,4,1,4,2,3],[4,3,5,7,5,9,5,5,2,0,0,7,4,0,4,8,8,3,4,8,2,9,0,9,3,1,3,4,5,6,9,1,1,4,9,7,6,6,4,8,6,4,5,2,6,1,2,7,5,6,5,7,8,7,6,3,9,1,9,6,9,8,0,1,3,9,0,9,5,4,7,1,0,5,5,7,6,8,4,1,6,1,0,6,8,8,9,9,0,8,8,1,7,8,2,2,7,4,3,4,0,0,7,1,7,2,0,4,0,4,5,7,7,8,3,7,6,5,9,8,3,7,0,0,7,4,4,6,9,7,0,9,0,9,0,7,1,0,1,2,6,9,1,5,7,6,3,5,1,2,4,5,1,4,7,8,8,2,4,9,1,6,9,1,5,1,9,8,4,2,0,2,1,3,8,0,0,1,6,3,3,0,0,4,6,8,4,4,2,0,4,3,6,5,7,3,6,8,1,0],[0,4,3,4,9,3,4,9,4,2,5,9,2,5,5,8,5,9,5,7,9,1,3,7,6,2,2,4,0,3,7,2,9,2,6,9,5,3,0,9,5,5,0,0,1,5,0,8,4,5,6,5,8,1,4,6,5,6,1,5,1,8,7,1,0,6,0,5,9,2,6,4,8,6,4,1,1,7,9,7,2,7,4,3,0,8,9,5,6,0,0,7,0,0,8,2,8,0,7,7,3,5,3,3,3,0,4,6,7,5,5,1,5,9,4,5,7,4,1,5,6,3,3,7,5,1,9,3,4,9,0,9,6,6,2,0,6,8,6,5,5,2,6,0,3,1,6,1,7,7,6,3,2,1,2,8,5,4,1,9,3,4,0,9,0,2,1,8,2,8,3,7,2,1,8,5,4,6,8,3,5,7,7,7,8,1,5,5,5,9,4,0,5,4,2,7,8,3,5,0],[3,0,0,7,3,8,5,8,6,3,1,1,2,0,8,3,4,6,8,9,7,3,2,2,9,6,9,0,9,6,0,5,8,2,4,1,2,9,1,8,3,3,9,7,5,8,0,9,6,9,1,3,4,3,7,3,9,8,3,0,6,6,7,4,8,2,7,1,1,1,9,6,6,1,4,1,1,6,3,7,5,4,2,9,9,9,5,0,9,0,2,5,6,0,1,7,4,8,0,7,1,1,6,7,2,2,1,3,8,4,2,4,0,6,3,9,5,0,1,4,1,5,9,7,7,2,6,1,3,6,1,4,0,7,2,2,9,3,8,7,9,2,3,9,9,9,0,6,9,3,1,0,8,2,0,6,5,8,9,8,5,0,4,7,7,6,1,8,1,9,8,2,4,1,3,3,0,3,9,2,8,2,2,7,7,4,5,4,3,4,4,0,5,8,7,4,7,8,5,0],[0,3,3,4,6,8,9,7,4,0,1,2,5,3,1,4,8,6,8,3,3,4,3,8,4,0,4,3,0,9,4,2,4,9,6,1,7,7,0,1,0,1,6,5,6,7,1,4,6,9,7,9,5,2,9,9,4,5,3,5,5,9,7,1,0,6,2,7,5,4,1,5,5,7,2,2,6,3,8,2,4,8,3,9,0,2,1,5,8,6,2,5,5,9,8,5,7,1,4,3,5,7,8,3,6,3,5,3,6,3,7,1,3,1,2,4,5,3,1,5,1,3,0,8,2,9,5,2,2,0,5,9,9,5,2,6,8,9,9,5,5,8,8,8,9,0,4,7,6,5,2,9,8,3,8,3,4,3,5,8,5,2,9,5,7,2,1,6,3,2,3,8,0,1,9,2,3,3,9,1,1,1,1,9,6,9,4,2,4,1,2,0,5,2,7,3,6,0,1,9],[2,6,0,4,9,9,8,2,4,7,6,5,1,7,7,9,8,3,4,2,5,6,4,2,8,1,7,6,1,0,6,5,6,6,2,5,7,0,0,1,0,8,9,3,5,6,2,3,9,8,7,6,5,2,1,5,5,8,2,7,9,0,4,5,8,6,3,5,7,5,8,9,5,7,2,0,5,6,5,7,5,4,5,2,6,6,7,2,5,1,1,6,3,5,3,1,2,8,8,1,3,7,0,8,6,2,0,2,8,7,1,5,2,6,7,0,5,7,4,2,8,5,8,2,3,1,5,7,0,4,8,5,1,8,6,9,2,8,3,2,6,4,8,0,3,5,0,8,2,7,2,3,2,2,7,7,3,2,4,5,6,2,3,9,2,9,9,4,9,2,9,7,9,7,7,2,2,0,2,7,7,6,0,1,8,7,9,1,1,3,9,8,8,2,9,0,3,8,7,2],[3,6,0,2,3,9,6,7,1,8,6,0,4,6,2,2,5,3,5,7,8,6,7,6,8,6,9,3,7,6,6,0,4,8,4,9,9,0,6,1,0,5,1,6,1,5,0,9,8,7,8,9,4,5,5,2,3,4,8,0,2,6,2,6,4,6,5,5,6,4,6,8,9,8,4,2,5,4,3,6,4,1,5,8,6,2,2,2,9,0,2,1,6,5,0,2,1,7,8,0,1,4,8,2,4,5,7,2,1,0,8,5,2,3,3,0,7,6,2,6,8,7,8,5,2,0,9,5,7,7,7,1,4,8,5,0,3,2,2,4,5,0,2,9,5,5,9,3,3,4,1,2,1,1,9,3,1,0,0,1,8,8,2,4,8,7,4,1,2,7,7,7,9,9,6,7,7,7,2,2,1,3,6,2,7,5,7,8,8,0,9,6,0,3,2,8,1,6,9,5],[5,8,2,5,8,0,4,7,7,6,9,9,1,6,3,8,3,3,9,1,3,0,9,3,4,1,1,7,8,2,2,5,0,6,2,0,6,6,7,5,4,7,6,8,5,0,8,8,3,7,0,8,8,1,1,4,3,4,1,3,6,5,8,8,1,1,9,9,7,8,4,4,5,1,2,0,3,0,1,8,8,3,6,8,4,9,4,7,3,5,0,1,0,9,1,1,2,0,0,9,9,6,3,6,7,7,9,0,8,2,8,8,5,6,8,9,5,2,7,0,9,9,3,9,0,5,2,2,5,4,2,6,0,7,5,0,7,6,2,7,8,3,5,3,9,5,2,7,7,1,7,6,1,1,7,1,6,1,4,3,5,8,2,7,7,7,7,4,3,2,1,1,5,8,6,6,3,8,3,2,0,1,0,3,4,9,4,2,0,0,7,7,0,9,5,8,8,4,4,1],[6,8,4,3,8,0,0,4,1,3,8,3,6,9,6,2,0,2,4,1,5,2,0,5,3,5,5,2,2,2,5,0,2,0,4,0,2,6,6,5,9,5,0,8,6,6,0,6,1,7,9,6,1,0,3,4,7,1,6,1,5,4,2,7,4,8,9,8,4,6,6,5,1,6,5,7,5,8,3,6,5,5,4,8,5,9,2,4,2,1,6,7,5,0,4,1,8,4,1,4,2,7,1,3,6,9,0,1,7,5,9,4,2,5,2,7,6,6,4,9,7,0,6,4,0,3,7,0,9,9,6,1,8,9,6,6,8,8,7,5,5,8,1,8,5,3,7,2,0,1,3,9,1,1,6,3,6,3,5,7,2,1,8,3,1,6,9,1,4,9,9,0,7,0,0,5,6,7,9,6,9,2,7,2,5,3,8,2,7,3,9,1,5,0,4,8,6,6,1,3],[5,0,5,4,1,5,9,7,2,0,5,3,2,4,6,8,0,4,2,7,9,1,0,6,3,7,6,2,3,8,5,0,8,2,4,1,7,6,8,1,6,5,5,1,0,3,1,0,9,3,9,8,6,1,5,0,0,1,4,5,9,9,5,0,3,2,1,0,0,2,1,6,7,8,9,9,1,0,1,0,5,2,1,4,4,6,4,4,9,0,0,1,1,7,1,4,9,2,4,9,6,7,8,4,6,9,5,7,0,7,0,7,1,1,1,5,9,7,2,0,7,4,1,8,3,4,2,5,9,8,6,5,6,6,9,4,6,7,1,6,6,3,5,7,6,7,5,5,4,7,6,4,3,9,2,6,4,7,3,3,5,0,8,3,6,0,7,2,7,1,0,3,6,6,2,3,5,7,8,9,6,6,3,9,8,8,8,2,5,1,7,2,3,5,8,0,7,5,4,6],[8,5,1,5,3,4,8,8,1,8,9,8,7,5,9,5,3,9,9,0,3,6,4,6,3,2,6,1,8,3,9,8,0,1,3,3,5,3,3,8,2,2,8,9,9,8,6,2,9,5,4,2,1,9,9,4,1,7,7,1,0,7,0,0,0,3,5,5,7,8,5,1,3,4,2,2,4,8,7,3,3,1,8,6,2,7,2,4,6,0,5,9,9,7,9,9,3,7,6,0,7,1,3,0,7,5,5,3,3,4,9,8,5,7,4,8,4,6,4,2,8,1,1,7,9,3,6,2,0,4,4,9,6,7,0,5,4,7,9,9,1,8,7,8,5,1,8,1,9,4,5,8,6,7,5,5,0,4,9,2,8,5,1,6,4,3,2,8,0,3,7,3,1,4,4,8,7,2,1,6,7,6,6,5,5,2,2,5,8,1,9,8,6,1,5,0,4,9,0,7],[2,7,2,5,3,6,3,0,9,6,8,8,4,5,3,0,9,5,5,7,8,7,5,4,8,2,6,4,1,6,3,3,5,6,8,8,2,3,8,3,1,6,1,6,3,6,6,2,1,3,1,9,0,9,5,0,1,3,5,5,9,8,0,4,6,1,2,9,4,2,2,6,1,6,4,4,2,0,9,6,5,0,5,6,9,3,8,3,6,3,8,8,4,0,2,0,1,7,9,6,1,4,4,2,2,8,7,4,0,8,0,5,8,8,3,0,3,2,3,9,7,3,7,3,5,2,4,7,9,5,3,2,1,9,5,3,9,4,0,1,2,2,8,2,0,2,2,3,6,7,5,5,2,2,9,8,6,5,5,7,0,0,0,2,1,5,7,0,1,7,1,5,0,1,7,2,5,0,8,3,9,3,9,2,7,0,2,4,5,7,3,5,9,3,9,0,0,9,0,1],[6,3,6,8,4,4,1,2,6,1,5,5,6,4,9,5,4,1,1,9,8,5,7,7,8,8,7,9,7,9,2,4,4,9,4,1,5,7,3,1,8,8,8,6,5,8,2,9,9,3,1,8,8,0,7,9,8,5,0,6,6,2,2,1,1,6,2,6,4,7,9,4,5,8,1,2,6,5,4,5,0,5,5,9,7,3,0,7,0,0,5,6,2,7,9,6,6,3,4,2,0,4,8,8,4,1,0,0,6,4,7,7,1,3,8,8,6,8,6,6,0,1,4,2,1,4,8,9,7,3,3,0,9,1,8,3,3,8,5,9,5,2,8,6,5,6,7,3,6,5,1,6,8,6,1,1,2,9,0,1,4,3,1,5,7,9,0,0,0,5,1,5,8,0,3,5,8,0,9,5,7,0,3,6,8,4,7,0,6,0,2,2,5,3,8,2,5,8,4,5],[6,8,2,4,8,5,9,8,8,0,3,5,3,7,3,1,1,3,2,9,3,6,2,0,1,0,5,6,0,9,3,6,7,7,2,7,3,4,8,3,4,1,8,7,0,4,1,4,7,5,3,2,1,7,2,2,7,7,1,0,9,6,6,8,4,9,8,9,5,6,2,9,9,2,9,2,6,0,8,5,5,1,7,8,9,2,2,8,1,5,8,2,2,7,1,8,8,9,7,3,7,1,4,6,3,5,8,2,7,6,7,4,0,7,2,1,9,5,1,2,2,2,5,4,9,6,4,9,7,3,4,4,4,8,0,0,6,1,4,3,7,1,8,9,0,2,0,1,9,2,4,2,4,9,8,5,5,3,4,4,6,8,8,3,8,0,3,4,1,7,0,1,0,8,0,1,2,3,2,2,7,6,4,1,7,4,8,4,7,2,8,6,2,8,9,0,1,2,7,2],[1,7,5,1,7,8,4,9,1,7,3,8,5,9,1,3,4,9,7,3,3,8,9,5,6,8,7,7,2,6,2,3,3,9,7,2,7,1,4,0,0,7,0,8,9,3,1,3,2,0,8,7,8,8,4,7,8,2,6,3,8,8,8,4,0,5,8,7,9,2,0,9,2,0,7,1,4,0,6,8,3,4,6,1,4,2,0,5,6,7,8,7,7,6,1,9,4,9,7,3,4,9,4,6,9,2,7,5,4,5,4,7,9,2,1,6,4,1,1,1,0,1,0,0,9,3,9,3,4,8,8,8,7,3,4,9,7,3,4,1,8,8,1,0,2,2,8,7,5,9,0,8,2,0,0,1,5,1,7,9,2,5,8,9,0,4,0,9,0,5,3,0,5,4,2,8,6,0,7,3,9,7,1,3,9,3,5,4,5,4,5,7,1,3,8,2,0,9,1,0],[4,6,2,9,0,5,9,8,5,6,2,7,5,5,0,4,9,5,0,6,9,6,3,1,1,1,5,3,2,8,5,6,5,8,8,7,3,7,6,8,6,0,5,1,5,8,8,6,3,8,2,5,6,5,8,0,9,5,5,1,3,1,0,0,9,8,8,4,7,4,4,3,6,0,7,1,0,5,8,5,5,2,0,4,8,0,4,9,5,1,0,1,2,0,1,1,0,1,7,0,7,2,5,3,4,2,5,4,9,5,9,5,9,2,9,9,2,5,8,0,6,1,1,9,1,4,2,4,8,2,4,5,4,1,1,8,4,8,2,3,3,3,0,4,5,1,4,8,6,2,0,5,3,3,6,7,9,0,3,7,2,9,5,6,0,6,6,6,4,0,2,9,4,2,3,9,4,7,9,2,2,9,7,7,2,5,6,4,6,9,1,8,0,8,7,1,4,3,9,8],[4,1,9,8,6,5,9,2,4,9,4,6,8,4,6,3,9,4,7,5,4,0,6,6,9,3,9,5,8,9,6,4,2,5,2,8,0,4,2,5,3,9,3,3,3,9,8,4,4,7,2,0,8,8,8,9,3,8,4,3,7,0,8,1,6,2,2,8,6,4,5,9,3,9,5,8,0,3,3,6,1,5,6,1,5,5,0,0,5,6,3,4,7,1,5,5,4,7,5,0,2,1,2,7,0,9,6,0,2,9,7,5,6,5,8,1,2,8,1,7,7,4,1,4,8,9,1,2,6,6,4,0,7,8,8,9,7,4,0,2,3,9,9,9,6,8,2,1,8,3,8,5,9,2,9,7,3,2,1,9,9,8,0,6,8,8,8,8,4,8,0,7,9,1,8,5,9,0,8,8,5,7,5,6,1,7,4,4,9,7,5,8,7,7,7,6,5,5,4,9],[5,6,8,4,9,8,1,9,0,0,9,7,9,4,4,2,1,0,8,1,7,5,1,5,3,0,3,0,5,9,2,2,7,2,8,6,1,2,5,3,4,6,1,3,3,5,7,6,5,7,9,4,2,1,9,7,1,2,8,9,3,2,1,0,4,0,7,7,2,4,1,6,1,2,1,4,9,0,2,4,7,2,8,1,3,0,1,6,4,9,5,8,1,9,8,7,1,5,5,3,0,6,1,3,8,4,9,9,4,1,3,3,3,3,6,6,3,7,5,8,8,2,6,1,1,4,9,4,0,4,9,2,2,0,5,2,6,4,1,2,5,4,5,9,7,4,7,3,1,4,1,2,7,9,3,0,3,4,5,5,8,4,9,2,7,4,4,5,0,5,8,6,1,3,7,1,9,6,4,3,1,7,5,0,6,0,0,1,5,7,9,5,4,8,8,1,5,2,6,5],[8,4,3,1,0,2,2,1,9,8,4,0,5,9,0,3,0,2,5,5,0,6,0,4,6,0,7,1,3,5,9,3,2,2,4,4,7,9,5,6,7,0,8,5,1,0,8,3,2,5,8,4,3,1,0,0,1,9,1,6,5,2,1,9,7,6,5,4,7,0,0,6,2,0,1,4,0,2,7,2,9,8,9,3,1,9,3,4,9,6,1,6,9,2,7,6,0,2,0,9,2,2,6,7,2,7,1,2,1,0,6,1,0,7,4,3,9,7,8,0,5,1,6,4,5,5,2,8,7,4,7,1,8,3,8,0,3,1,4,6,4,3,7,4,2,3,8,1,2,8,1,8,9,7,4,6,2,7,4,1,3,4,3,2,9,1,4,4,3,9,1,7,4,0,3,6,4,3,8,6,1,9,6,2,9,1,9,3,0,5,5,3,9,0,5,1,1,0,5,6],[1,6,5,7,7,1,3,3,4,3,1,6,5,8,0,6,9,9,9,1,7,6,4,8,6,2,9,0,2,5,8,5,3,4,2,2,5,5,5,1,1,7,9,6,7,0,2,6,1,3,9,8,2,3,7,0,5,8,2,9,3,1,4,9,7,6,1,4,4,9,5,5,6,7,3,3,7,7,1,8,2,0,9,4,5,8,7,1,6,9,0,2,2,7,1,1,3,4,5,7,3,3,4,1,0,7,6,7,6,7,7,9,9,8,5,7,6,2,8,5,4,0,9,8,7,2,0,3,6,5,2,0,8,7,3,0,6,0,9,3,9,7,2,1,5,7,8,4,2,8,1,6,0,0,4,0,2,6,3,8,2,5,0,2,4,6,3,1,6,2,6,7,1,8,8,9,7,6,5,9,6,6,7,7,6,4,7,0,0,2,0,4,9,3,9,4,9,4,7,5],[6,3,4,0,3,3,9,0,1,4,2,8,0,1,7,8,7,4,8,0,6,8,4,7,3,5,1,2,9,0,9,6,3,4,8,8,9,7,9,0,3,3,0,5,6,7,5,4,3,3,4,1,3,0,9,7,6,2,1,5,3,1,3,6,7,3,5,6,0,6,6,5,1,9,0,7,6,5,3,2,0,7,3,6,0,2,5,8,5,6,5,8,9,1,6,6,4,3,2,5,1,1,2,2,0,5,0,8,2,3,0,5,3,4,1,5,8,8,3,5,4,8,5,6,1,2,2,6,5,7,3,7,8,5,1,0,0,3,0,5,7,1,0,2,7,3,9,7,1,2,3,7,2,8,3,4,2,8,0,0,5,3,9,5,0,0,5,3,4,7,8,3,8,8,7,7,1,6,7,2,0,0,1,4,0,5,8,3,5,0,5,0,5,4,5,6,6,2,1,0],[9,9,5,0,9,2,9,0,0,6,4,0,8,7,5,1,4,5,6,9,8,1,9,3,5,6,1,1,0,2,4,2,3,1,4,2,4,3,4,6,2,0,7,2,8,4,3,2,9,9,2,7,0,3,3,7,0,4,1,0,9,7,4,2,8,8,7,4,4,1,1,6,2,0,0,2,4,6,6,5,5,8,5,8,2,8,7,4,4,8,6,5,7,1,8,6,1,7,0,5,8,1,3,2,1,4,4,7,0,3,3,7,1,8,7,5,8,5,1,4,5,8,0,3,1,0,1,2,7,1,0,7,3,3,0,6,9,6,4,9,9,7,9,3,7,6,0,7,3,2,3,9,2,5,4,5,5,5,7,4,6,7,2,1,3,4,8,2,0,4,4,2,3,3,5,2,9,7,1,5,1,4,4,3,0,0,8,5,5,8,2,3,7,4,7,2,8,7,5,0],[1,9,2,4,4,9,8,5,9,9,0,0,3,6,4,3,6,4,1,3,4,3,7,4,9,4,6,9,1,1,9,4,2,4,0,6,5,8,2,4,7,4,5,2,1,1,6,7,7,9,1,4,4,0,8,5,6,4,4,9,8,5,3,2,9,3,1,7,3,5,3,0,9,0,2,0,3,0,0,1,1,1,5,7,3,5,2,9,9,6,0,9,2,3,4,3,8,5,2,3,0,6,5,9,6,9,2,0,0,2,3,3,5,0,1,8,5,3,9,6,2,9,8,6,4,2,9,4,7,4,7,9,0,2,0,8,3,2,0,5,4,5,9,1,5,0,9,0,5,0,9,7,1,7,5,7,1,7,1,8,1,0,9,3,4,9,1,8,2,4,3,8,9,4,0,7,6,1,9,2,4,8,1,5,7,7,3,8,4,6,8,5,9,7,0,3,7,1,3,1],[7,7,1,9,1,3,6,8,5,5,2,1,6,5,8,3,2,1,4,8,0,2,5,9,2,5,4,1,9,8,2,8,5,3,7,8,9,5,8,4,1,2,7,9,8,5,4,2,9,8,3,9,3,8,0,5,4,4,6,5,2,0,3,9,5,1,0,4,6,8,0,9,3,7,0,1,3,7,5,4,5,8,3,0,9,5,7,3,9,3,0,2,5,3,3,1,6,3,7,5,4,8,6,7,7,9,0,0,6,7,4,3,6,9,4,5,4,1,0,6,7,2,0,4,5,3,5,4,9,3,1,3,3,7,0,0,6,2,3,4,9,9,8,7,1,2,2,5,5,4,3,4,6,3,9,4,9,6,8,8,1,1,1,4,0,3,7,7,5,0,1,6,1,1,4,2,3,6,0,1,3,3,7,9,7,6,3,6,5,3,6,6,4,7,3,5,0,2,4,7],[2,7,3,3,9,7,6,4,6,6,7,9,1,5,8,8,3,4,6,0,9,2,7,6,1,2,1,3,4,7,0,8,6,6,3,7,3,9,2,9,7,1,0,9,8,9,9,4,5,8,4,4,2,3,0,4,5,3,9,1,2,0,9,9,6,3,6,1,4,0,3,4,4,3,3,2,4,4,6,9,4,1,6,7,4,6,3,2,2,2,3,4,2,5,5,0,8,4,4,4,6,7,8,0,0,3,3,5,8,1,6,2,4,2,1,9,9,4,3,3,7,8,9,1,3,5,4,1,9,8,8,5,5,6,8,7,0,1,2,0,4,9,4,9,3,6,0,4,2,3,7,1,3,7,3,7,4,9,0,3,9,8,0,6,5,8,3,7,1,8,7,6,9,1,7,2,9,7,7,2,2,6,3,5,3,8,2,7,9,5,2,8,3,5,4,0,3,0,7,5],[8,6,3,9,8,0,1,9,9,0,1,3,7,7,0,2,5,5,2,5,0,4,5,3,9,2,4,5,2,3,2,2,0,5,1,0,7,4,9,8,5,3,3,4,0,3,6,7,8,8,4,8,5,0,4,6,4,0,1,6,3,3,0,5,0,1,5,9,7,7,9,4,2,2,8,4,6,7,1,4,7,8,5,4,8,9,1,2,9,2,0,4,8,0,0,0,3,7,0,0,6,9,5,8,2,5,2,0,4,4,4,2,4,9,8,2,0,9,6,9,4,8,4,4,0,4,4,3,3,4,3,0,6,0,8,0,8,1,0,2,7,4,6,1,6,5,5,6,6,3,8,0,3,2,4,3,8,9,6,1,5,1,1,1,2,2,1,2,3,3,4,2,8,1,5,6,8,2,4,4,5,2,7,0,6,1,3,6,2,9,8,0,0,1,1,4,3,5,6,8],[8,3,2,8,6,7,6,4,1,3,0,8,5,7,8,4,1,1,0,5,0,0,5,3,2,9,9,7,6,8,6,4,3,0,5,9,0,3,5,1,6,5,0,4,3,8,8,6,2,8,1,4,1,7,9,3,8,9,0,4,9,8,0,2,9,5,3,1,1,8,2,7,3,4,1,8,3,9,4,7,0,6,1,3,5,1,6,3,2,8,7,1,9,7,5,0,5,8,1,6,6,3,5,1,0,7,0,3,8,4,2,0,2,3,3,7,4,1,0,8,0,9,9,1,9,6,1,4,4,2,2,2,7,7,4,7,4,4,2,5,0,4,5,3,0,1,0,6,2,3,5,4,4,4,5,3,3,6,7,7,0,1,0,0,9,4,9,5,0,4,2,0,8,0,5,0,1,8,7,3,1,2,8,5,8,5,1,1,2,0,1,4,2,1,4,3,7,6,0,9],[0,3,1,0,3,7,1,6,5,8,9,8,2,9,5,2,5,6,4,9,9,5,3,1,8,0,6,7,6,6,6,6,1,7,6,6,6,9,4,3,9,6,1,1,5,9,4,2,7,0,3,6,7,9,7,5,9,5,2,7,4,0,5,7,9,3,4,6,3,8,9,4,4,3,6,2,2,2,6,9,4,0,6,1,9,5,6,0,3,0,9,7,0,6,4,1,1,0,7,4,1,7,1,7,0,9,9,4,1,8,3,5,0,1,6,9,9,4,1,2,6,2,1,8,8,5,9,9,8,7,6,9,6,9,6,8,8,8,2,9,6,7,6,6,9,4,7,0,0,0,4,6,4,5,4,4,0,3,5,0,2,3,9,8,2,6,6,0,6,0,1,2,8,9,0,9,3,9,9,5,9,3,3,5,0,7,9,2,3,4,3,5,8,2,6,0,0,2,1,6],[3,4,0,3,4,2,2,9,1,3,5,2,6,8,7,6,8,6,8,1,3,1,8,1,6,4,1,6,9,4,5,4,9,5,7,3,0,9,4,1,4,9,6,0,0,5,8,8,4,8,9,7,2,9,8,0,4,1,6,5,6,3,9,7,1,8,2,1,9,6,4,3,8,0,5,8,8,5,6,2,3,7,9,7,6,7,7,2,0,6,9,8,1,8,5,2,6,9,3,5,6,0,0,4,2,5,2,0,0,8,2,6,7,1,3,3,0,3,8,1,1,7,9,2,8,7,5,6,6,0,4,2,0,4,8,3,0,0,5,0,0,8,6,7,1,2,3,2,5,3,3,8,2,4,0,0,1,5,7,8,8,1,2,0,5,1,5,7,3,1,8,6,9,6,5,0,0,0,4,7,3,9,5,6,4,6,8,7,3,5,7,3,8,0,4,4,3,9,1,6],[0,1,4,1,0,0,4,2,0,0,0,4,0,5,2,6,3,0,3,9,8,3,2,6,3,6,0,8,6,4,6,8,7,1,2,9,1,8,2,1,8,4,7,8,1,9,6,5,2,2,4,0,5,6,6,0,5,9,8,3,5,6,1,4,7,5,4,8,3,8,2,4,2,9,4,3,1,1,8,5,3,4,5,0,3,3,0,8,2,0,1,7,6,4,4,6,2,8,6,5,6,8,1,0,0,6,3,3,7,4,8,2,0,3,4,3,6,6,3,1,6,6,0,4,3,4,0,5,4,7,2,2,7,4,2,7,0,6,2,9,2,0,3,4,5,7,8,4,3,3,5,1,0,7,5,5,2,8,0,6,7,2,9,6,8,3,6,0,1,8,9,3,9,2,8,6,1,8,0,4,3,5,5,3,3,3,8,7,3,0,5,2,5,6,8,3,0,4,4,1],[5,5,7,6,0,7,2,1,7,3,8,0,0,5,6,5,0,6,2,3,7,0,5,2,6,6,5,8,2,1,2,7,7,9,3,7,8,6,0,5,1,8,5,1,4,1,7,4,8,1,0,5,1,7,9,0,3,6,8,8,8,0,5,7,1,1,6,9,9,6,4,0,7,0,3,3,3,0,9,1,2,9,8,5,7,9,5,2,6,6,0,6,6,8,3,8,1,9,9,2,7,4,2,4,6,7,7,1,8,7,5,2,8,3,9,7,5,5,0,1,1,2,7,9,2,0,9,3,1,9,5,8,5,9,5,3,7,2,4,7,1,9,1,0,5,0,9,0,5,9,1,8,4,8,0,6,0,9,2,3,0,9,3,5,1,8,8,8,3,3,7,6,2,8,8,9,0,8,9,8,9,0,6,3,0,6,2,2,8,6,7,8,5,1,4,6,9,4,6,4],[7,5,1,2,5,9,1,6,7,3,4,7,5,2,2,6,1,4,0,9,0,8,9,8,9,3,4,0,8,3,5,7,8,6,1,4,7,5,2,7,8,6,6,3,0,8,1,1,5,2,2,7,2,2,5,3,7,2,3,5,5,0,5,5,6,8,1,6,3,3,3,1,1,9,7,2,9,8,5,4,2,8,2,6,2,7,9,9,9,3,7,6,3,2,4,2,2,5,0,6,1,3,9,2,4,6,6,3,7,2,0,1,2,2,8,4,1,7,5,1,0,2,9,6,6,5,0,9,1,0,7,2,5,6,6,1,3,5,4,2,7,4,3,1,8,1,7,0,9,2,3,1,7,4,9,5,0,9,6,1,1,3,5,6,0,3,9,5,8,6,9,7,0,2,8,1,4,7,1,5,2,4,8,1,8,8,6,8,9,3,1,1,6,8,9,8,2,9,5,2],[5,4,0,7,7,0,0,3,8,1,0,2,7,8,3,8,6,9,8,8,2,2,9,1,0,0,1,4,1,7,7,8,1,9,8,0,9,8,3,9,2,3,1,1,4,4,9,2,4,0,0,8,2,1,1,2,2,3,9,5,2,6,4,3,5,4,6,6,2,1,6,6,7,7,8,1,2,7,3,8,7,4,8,1,5,2,4,9,5,5,7,9,3,1,4,8,7,0,6,9,2,2,8,9,2,6,2,4,5,5,4,3,1,4,4,9,6,0,8,3,5,5,2,8,8,7,8,5,9,7,7,3,9,7,2,1,3,6,7,8,2,3,3,3,8,0,4,4,2,3,8,0,0,2,0,1,1,9,8,1,8,7,4,7,4,9,1,7,5,8,8,9,4,1,5,2,3,9,8,6,4,8,6,5,1,6,8,4,7,6,5,5,6,2,5,0,1,6,0,8],[6,0,8,0,3,5,4,7,4,3,3,1,3,1,8,4,7,6,9,7,4,6,2,0,8,7,3,1,5,5,0,2,5,0,4,8,7,9,7,1,4,2,2,7,3,2,4,1,8,3,0,3,1,2,5,2,2,8,3,7,5,5,1,0,7,6,1,4,7,8,8,1,3,2,8,6,5,2,9,3,7,9,8,1,2,4,3,6,4,8,5,2,4,7,2,3,5,5,8,4,6,8,5,9,0,5,5,5,8,5,1,7,6,1,8,0,7,3,6,2,2,2,4,8,1,8,1,8,6,1,2,2,9,9,3,0,4,8,7,4,5,8,2,4,0,2,4,9,6,1,3,0,5,7,8,6,8,9,6,6,1,8,8,2,7,3,2,3,3,0,8,9,0,2,5,2,4,9,2,2,0,5,2,5,5,0,3,5,2,1,1,3,1,1,5,8,4,0,2,7],[2,2,8,4,4,3,9,8,3,3,1,5,0,3,3,7,6,8,2,8,0,5,3,1,8,0,2,4,2,6,4,4,8,4,1,4,0,0,2,5,3,5,0,5,1,5,3,7,4,7,7,6,3,0,9,3,2,1,8,5,9,4,1,9,8,2,3,8,2,8,3,7,3,6,3,6,1,6,5,7,3,2,3,8,4,5,4,9,6,4,4,8,8,5,7,6,0,3,7,4,1,0,2,6,8,7,3,0,5,0,9,0,5,3,1,9,8,5,0,6,9,4,6,9,2,4,7,4,7,4,8,0,7,2,6,5,9,9,7,4,2,7,5,7,2,6,8,2,3,9,0,4,5,7,3,9,3,0,3,2,5,2,2,4,4,0,1,6,2,9,0,4,8,7,1,0,5,9,4,0,0,4,4,6,3,7,5,6,0,1,0,7,3,4,1,9,5,2,5,7],[3,8,3,1,7,4,3,5,5,7,5,6,4,0,2,7,9,9,6,9,0,8,6,5,3,9,7,0,2,4,9,7,2,2,1,2,8,4,7,5,4,4,1,8,4,5,7,6,7,3,5,9,2,4,5,7,3,4,9,7,8,8,5,3,2,6,5,0,2,2,5,6,6,9,6,3,4,4,9,3,9,6,3,3,0,8,0,4,4,9,1,2,9,8,5,1,6,0,3,9,4,1,7,3,0,4,8,6,8,9,0,7,5,3,3,8,3,3,2,7,5,5,9,6,6,7,8,2,9,1,1,6,2,1,9,4,5,9,1,5,8,1,4,5,6,7,3,9,3,7,8,8,5,9,4,3,6,4,5,6,6,9,4,0,0,3,5,7,4,6,2,4,9,6,1,5,4,7,6,7,4,4,7,1,3,1,4,2,8,2,0,6,3,4,6,5,9,3,2,5],[1,4,9,0,0,0,7,4,7,5,3,4,9,0,7,5,4,4,9,2,8,9,8,1,5,4,6,4,0,8,9,1,2,0,4,4,0,3,9,0,9,2,6,0,5,3,5,9,7,6,1,7,7,9,8,2,5,4,8,5,4,9,9,8,9,3,3,2,8,2,2,7,6,0,8,1,3,5,0,3,4,3,0,1,2,1,4,8,5,4,5,2,4,6,0,3,1,3,7,0,7,1,9,6,1,9,9,5,7,0,0,1,5,2,2,8,3,8,8,1,3,5,3,7,2,3,2,3,9,0,3,6,1,3,2,5,2,2,2,9,4,2,2,9,6,7,9,0,5,7,1,0,5,6,9,7,1,2,0,0,4,6,7,7,1,1,2,5,5,6,7,9,8,9,1,7,6,0,7,2,0,0,4,7,8,4,4,9,8,6,0,2,2,9,1,5,0,6,1,6],[2,8,7,3,7,0,0,6,3,7,0,3,9,4,0,7,8,6,6,6,2,8,0,7,9,2,2,0,8,5,8,0,3,7,3,3,8,5,9,1,4,1,6,3,5,8,2,6,6,1,4,8,9,5,7,9,9,2,1,9,7,1,1,3,0,7,6,0,2,7,1,7,0,9,2,7,9,5,3,5,6,0,6,7,5,3,8,6,7,1,5,5,4,6,0,5,3,6,5,6,5,9,5,5,8,7,2,0,2,8,5,0,8,1,8,5,7,8,3,4,0,8,1,4,6,1,1,2,9,7,8,6,6,5,3,6,2,8,6,5,8,4,7,8,5,5,3,4,6,6,1,8,6,4,2,4,8,4,6,9,3,4,6,1,1,1,7,4,1,6,1,9,0,8,7,7,6,2,2,2,0,5,0,6,9,4,1,7,0,7,7,3,4,5,6,5,6,4,1,8],[2,2,9,2,1,7,9,9,9,3,1,0,8,3,8,0,9,9,7,0,9,6,5,5,1,2,2,0,8,4,0,0,6,9,4,9,8,5,8,8,9,1,0,9,6,8,1,6,0,9,8,1,7,3,6,1,7,8,3,5,2,3,7,1,4,1,0,3,7,1,3,8,2,3,9,9,3,1,7,3,2,5,4,9,0,0,2,8,1,5,5,5,0,5,6,7,6,9,2,5,0,5,5,2,0,5,3,3,8,2,9,0,7,3,1,0,6,4,0,7,1,7,4,2,2,1,9,1,0,1,8,0,8,4,4,8,1,0,3,9,4,2,9,4,8,0,4,4,6,4,3,8,1,7,2,6,8,1,7,0,4,5,2,4,9,9,4,2,1,7,1,5,2,2,9,0,3,5,4,1,9,9,1,3,6,3,1,7,6,8,9,2,5,2,6,7,3,0,9,4],[0,1,9,4,3,1,4,8,6,0,2,8,9,3,3,7,7,4,4,3,4,4,6,9,8,4,6,1,7,8,7,9,9,6,3,4,9,7,5,6,9,7,4,8,0,9,5,9,3,2,5,7,8,1,8,8,7,7,9,4,5,6,3,6,4,6,0,4,5,5,2,4,4,8,4,5,7,0,6,2,4,1,1,2,4,9,0,2,6,1,8,1,9,4,7,5,2,0,9,8,7,3,4,2,1,9,9,0,1,5,2,5,7,3,9,1,5,1,5,1,4,4,5,3,8,4,8,0,4,0,0,2,5,5,6,9,4,5,9,7,2,2,2,9,7,3,3,2,6,8,6,0,4,1,5,2,5,5,5,2,7,5,6,3,2,2,2,8,9,1,5,1,5,9,3,3,2,6,7,8,4,3,0,1,4,5,3,2,1,0,6,8,8,2,3,0,6,5,1,5],[9,6,8,4,6,1,9,0,7,7,1,4,2,1,5,9,7,0,1,0,3,9,8,1,3,4,3,9,9,4,6,0,1,6,7,9,8,6,9,7,5,0,3,8,4,0,7,1,3,0,1,6,1,1,9,4,5,2,5,7,9,3,9,0,9,6,1,9,5,2,7,0,3,2,0,9,3,9,0,8,1,3,4,2,6,3,8,4,7,3,3,6,6,2,8,8,9,9,7,4,2,4,6,7,9,7,6,4,8,8,2,0,3,6,4,9,1,3,5,0,8,8,9,7,1,7,5,2,9,2,8,3,9,4,2,8,3,0,2,4,8,4,6,3,2,0,2,5,5,0,7,6,8,6,3,1,4,8,5,5,2,3,8,1,0,2,9,3,2,3,9,0,7,5,3,1,6,7,8,3,7,6,9,8,2,2,1,8,2,7,5,5,2,3,6,2,5,8,8,9],[1,7,9,1,3,4,4,1,4,3,4,3,9,4,3,3,8,5,2,1,2,7,6,4,3,2,9,0,2,7,0,4,4,1,7,9,8,1,0,4,4,5,7,3,1,3,9,9,8,1,0,2,0,8,8,5,3,7,6,5,6,8,9,3,1,6,2,9,0,5,3,4,0,3,0,3,6,9,4,6,2,5,8,4,5,8,0,8,6,8,4,4,8,5,7,9,4,2,9,4,7,4,8,9,9,0,2,7,9,6,3,3,3,3,8,9,2,0,9,0,0,3,4,8,1,4,7,5,6,8,9,3,5,9,2,6,0,6,4,1,4,7,5,0,3,5,9,5,5,0,7,7,6,1,5,7,7,4,2,3,5,3,8,2,2,2,8,4,8,2,6,3,2,3,5,5,8,4,2,5,6,9,2,2,2,9,9,0,5,3,5,0,6,4,4,1,6,3,5,7],[7,3,2,1,8,7,8,8,3,0,3,9,1,7,2,6,8,3,8,4,7,5,6,5,9,1,6,8,6,4,5,3,9,9,5,8,6,5,6,1,6,2,0,9,1,4,7,0,8,5,6,7,1,4,2,2,7,9,0,3,5,7,9,4,8,4,4,4,9,3,5,7,5,8,9,8,4,6,0,4,4,6,1,7,3,6,1,0,7,4,4,2,3,3,8,2,9,3,6,0,6,4,0,3,4,9,1,8,7,2,5,3,0,8,2,3,4,4,4,1,8,0,5,1,3,4,5,4,9,4,4,7,0,4,0,4,5,3,4,5,5,1,8,8,0,3,1,4,7,7,8,7,7,3,0,2,9,6,6,8,2,1,5,2,7,7,6,3,1,2,0,8,4,0,6,4,3,0,0,2,9,8,1,9,2,4,3,3,0,0,2,4,3,9,6,2,9,4,7,2],[6,7,0,2,8,9,8,3,1,9,6,0,7,7,1,1,1,5,5,3,7,9,7,2,8,5,4,9,9,4,1,6,1,2,0,1,3,9,5,4,8,3,6,5,2,0,9,6,5,4,1,4,3,1,6,3,6,2,3,6,8,4,4,0,8,6,3,1,5,0,7,3,5,4,1,8,4,0,6,1,6,7,7,9,8,5,2,7,7,7,5,6,2,1,8,0,7,3,4,3,4,1,8,1,7,1,9,1,3,5,4,9,5,1,0,5,6,3,2,6,0,9,4,4,0,4,7,0,7,1,3,3,4,3,5,2,7,6,5,0,4,0,2,9,3,2,6,0,7,1,8,0,0,2,6,3,8,3,3,5,6,8,1,1,1,8,3,8,6,8,1,0,0,3,1,4,7,0,6,7,1,6,7,1,0,3,4,8,9,9,5,5,7,8,6,9,6,1,9,3],[2,2,5,2,7,9,8,5,9,6,4,0,2,1,3,4,6,0,2,5,9,8,3,7,8,9,8,5,3,9,0,5,2,5,9,9,6,0,6,5,6,0,5,1,3,9,5,2,9,0,9,0,0,2,7,8,4,7,3,9,7,5,4,9,3,5,0,9,5,9,5,4,1,2,7,7,1,2,9,2,4,8,5,4,3,2,3,9,0,8,8,9,4,2,8,9,9,0,8,5,9,5,1,1,8,8,0,1,2,9,6,7,9,1,1,4,5,6,3,7,5,1,6,1,5,6,0,5,7,0,2,8,6,3,9,4,3,1,7,7,0,3,4,2,6,8,6,2,6,0,9,3,3,8,4,9,4,6,6,1,7,8,2,5,3,1,1,6,3,8,5,5,2,0,9,8,0,6,2,8,8,4,2,1,2,8,2,6,5,8,0,4,6,2,9,1,5,0,7,0],[8,5,8,2,5,7,3,7,5,5,7,3,9,9,7,1,8,1,0,5,0,2,9,8,6,8,0,1,0,9,4,8,4,4,3,1,1,6,0,9,3,8,2,3,9,1,6,7,3,8,4,5,0,3,5,6,3,5,0,3,7,6,4,3,0,7,7,3,5,7,2,8,7,7,1,7,8,0,6,3,8,1,0,9,4,6,7,0,3,7,3,2,5,7,6,5,6,3,1,1,2,3,0,0,0,3,7,1,3,5,6,2,6,7,3,3,5,0,3,8,0,8,3,5,8,9,3,4,4,4,8,6,7,8,6,0,3,5,1,7,1,9,1,9,6,4,4,3,4,9,4,4,8,7,0,6,6,3,2,0,9,0,6,6,0,5,6,4,0,9,3,3,9,4,3,7,0,9,3,4,9,7,1,7,4,3,5,2,8,7,4,7,0,2,5,0,7,4,6,0],[5,1,3,6,5,8,4,7,8,7,4,7,6,5,6,0,0,1,2,0,0,8,7,0,0,4,3,8,8,9,8,4,1,3,2,8,4,6,8,2,5,2,1,3,9,7,3,9,0,7,1,0,5,0,3,6,4,6,6,5,7,6,1,0,9,3,1,5,2,9,7,9,3,0,3,2,9,6,3,9,6,4,2,3,4,5,9,0,1,7,5,0,3,6,1,5,2,2,0,4,3,0,3,6,0,6,8,0,5,3,1,1,9,3,4,3,8,6,5,1,3,1,2,9,9,5,4,1,9,6,5,2,6,1,0,9,7,0,1,4,3,4,7,2,8,4,7,8,0,2,2,5,5,6,6,5,1,2,6,2,9,4,6,7,5,6,8,4,8,1,9,3,6,6,7,6,0,4,4,2,8,8,0,4,4,6,1,7,1,9,1,2,3,7,9,8,5,0,3,3],[1,4,6,7,2,3,3,5,9,0,7,0,0,9,6,7,8,7,6,9,8,8,1,2,7,2,2,3,2,7,8,6,1,5,5,4,0,1,9,2,3,8,4,3,8,0,0,8,9,9,9,7,7,0,1,4,4,4,7,9,1,6,5,5,3,2,9,5,3,0,7,6,0,3,0,0,3,2,8,4,1,7,4,0,9,5,5,6,1,4,5,3,2,2,0,7,4,1,3,0,1,2,6,1,8,8,4,1,3,2,8,6,2,2,7,1,9,4,7,1,0,4,6,5,8,6,2,5,7,7,5,0,2,3,3,0,2,7,3,5,2,1,3,4,5,0,5,7,6,5,8,9,1,6,4,0,4,8,7,1,6,2,3,0,7,6,0,9,6,5,6,8,9,0,2,4,2,9,3,9,4,1,0,8,9,4,0,3,2,7,6,0,1,9,0,8,8,2,8,6],[8,6,4,7,6,6,3,1,7,7,2,4,0,2,2,0,6,2,5,0,9,2,1,0,3,1,8,3,4,8,9,2,7,3,1,5,1,4,6,9,3,0,3,4,2,5,6,0,9,1,1,8,5,2,0,9,5,0,2,1,1,2,3,8,7,6,3,9,3,2,0,6,2,5,0,5,2,8,7,1,0,8,1,7,2,3,8,0,3,1,1,6,5,7,4,4,3,0,5,6,2,5,3,4,2,5,1,4,4,9,7,6,9,8,5,4,1,4,4,7,7,5,3,4,4,0,8,8,0,4,6,2,1,1,8,4,7,0,8,3,1,8,1,2,6,6,6,0,0,2,7,9,0,2,3,4,2,2,4,4,6,1,8,9,2,7,3,1,9,4,6,2,2,7,4,0,6,3,0,8,5,9,8,7,2,3,2,6,5,8,3,3,9,1,3,4,8,8,7,9],[2,4,1,6,3,6,7,1,9,9,0,6,9,0,4,3,3,8,1,1,6,4,4,8,8,9,2,8,8,1,8,2,5,9,1,1,7,0,2,8,9,4,5,0,6,1,5,2,9,9,3,5,5,9,5,3,9,7,2,9,1,0,3,8,1,4,9,9,4,4,7,4,0,4,6,9,5,2,1,4,1,6,2,8,5,7,2,6,7,4,7,8,6,1,8,7,5,8,8,2,4,8,6,6,2,4,5,0,8,8,4,1,4,8,0,2,8,2,0,7,8,8,7,4,1,5,3,6,5,2,8,9,0,6,8,4,3,3,4,1,2,1,3,8,9,3,2,9,5,3,6,5,3,3,9,4,9,2,2,4,6,3,6,6,1,4,3,4,9,7,6,3,8,1,4,0,4,6,9,1,9,6,6,4,1,7,8,0,9,1,7,6,4,5,4,5,1,7,0,2],[5,8,6,5,9,0,5,5,8,5,8,0,3,6,4,4,3,3,7,4,4,4,2,0,1,7,7,2,6,9,4,3,7,2,9,8,4,6,5,3,3,3,5,8,9,9,3,4,2,0,9,8,6,1,0,9,0,8,3,9,7,9,2,5,2,3,5,8,2,1,3,5,6,8,4,6,8,9,0,2,1,1,3,9,5,3,8,5,1,1,6,9,0,9,6,4,4,1,3,8,4,8,4,1,7,0,9,7,1,1,1,2,3,4,1,0,8,1,5,1,2,4,2,4,5,8,1,9,2,4,8,8,4,4,9,3,6,0,0,7,2,2,9,7,8,2,7,8,3,4,0,7,8,2,3,3,3,4,5,5,0,3,3,5,9,5,8,5,5,1,4,7,5,5,6,3,7,3,2,2,8,2,1,6,6,4,2,9,1,7,6,1,2,2,8,1,7,9,8,4],[0,2,4,5,9,0,0,6,4,2,0,4,6,1,2,3,7,4,2,0,1,9,4,5,1,2,8,0,3,6,4,3,0,0,0,9,1,1,7,7,3,7,3,0,0,5,5,0,0,7,0,3,8,4,9,1,9,7,1,2,6,8,6,8,0,6,0,1,9,7,0,3,7,3,5,7,1,0,7,1,9,0,6,0,4,7,1,3,7,5,6,5,5,4,3,5,0,3,9,2,3,1,5,2,5,2,9,8,4,9,1,3,9,7,3,5,7,7,9,4,2,5,9,7,9,4,4,1,0,3,3,5,5,0,7,2,4,8,0,8,7,1,4,8,0,9,4,7,6,5,3,0,2,4,7,3,1,4,6,1,7,0,8,4,2,5,8,9,3,0,7,3,1,3,3,4,3,9,3,9,4,7,0,6,1,9,9,4,5,6,5,5,8,3,9,2,0,0,1,4],[0,1,9,4,4,2,8,9,2,1,9,8,8,1,7,2,2,8,6,8,6,4,3,4,9,4,9,0,4,2,6,7,5,7,1,2,1,9,1,5,2,0,4,3,3,1,7,6,9,5,6,6,9,1,2,1,5,3,3,2,8,1,9,3,8,0,5,9,1,7,5,3,9,1,6,3,2,5,1,3,1,9,9,2,0,4,3,5,7,8,7,5,9,8,1,7,8,6,9,1,5,6,7,5,7,3,0,9,1,3,2,2,2,4,4,2,0,0,9,9,8,9,5,0,7,8,9,8,6,8,9,2,4,6,9,1,2,9,2,3,2,7,7,6,3,1,8,3,3,7,2,4,8,9,4,6,7,3,6,4,4,5,6,0,4,5,4,8,6,6,3,0,5,0,6,8,3,6,1,7,3,6,1,4,5,5,0,5,0,6,9,4,3,7,7,9,4,1,7,2],[7,0,2,5,0,8,3,4,4,7,3,9,5,6,3,0,3,5,5,3,3,6,0,7,5,7,6,1,8,6,3,7,6,5,2,9,5,8,5,1,7,0,3,2,6,8,2,9,4,0,4,9,8,4,6,6,3,5,7,1,3,1,1,1,6,5,0,4,3,7,5,0,7,8,2,3,7,7,2,3,9,7,2,9,3,1,5,7,8,3,0,1,4,1,2,2,7,5,6,2,2,4,3,0,2,7,3,9,6,8,4,7,5,7,7,8,8,4,7,8,9,8,9,5,9,3,8,8,8,6,1,1,0,6,1,5,3,6,6,2,4,1,9,1,8,8,2,8,5,9,6,4,7,5,0,9,8,0,7,9,8,0,0,1,6,3,6,2,9,2,4,4,3,5,5,3,4,9,1,9,9,7,5,8,4,5,7,5,7,7,4,6,7,4,7,6,7,5,8,8],[7,4,2,1,9,0,4,5,9,8,6,0,5,2,9,2,9,8,7,7,5,3,5,5,9,2,3,8,7,1,6,6,5,1,9,6,3,6,2,4,4,8,5,1,0,4,3,0,4,2,9,0,5,4,7,6,8,0,4,7,3,1,3,0,2,3,6,5,1,8,1,5,9,6,6,9,2,2,1,7,6,2,9,4,6,8,0,6,0,5,3,3,6,9,3,0,4,1,7,5,0,8,2,9,5,0,0,9,2,4,6,1,6,7,5,5,5,5,3,5,2,9,8,8,8,3,0,4,5,7,1,7,8,3,8,5,5,8,4,8,4,1,1,3,0,6,0,6,3,3,1,6,4,2,6,4,5,7,8,2,6,1,9,4,6,7,9,2,8,4,2,4,7,3,7,7,9,9,5,2,5,7,0,9,1,7,4,6,4,4,1,2,6,0,7,2,0,6,6,0],[2,8,4,1,1,4,1,2,3,6,7,0,5,7,0,6,4,6,5,0,0,6,3,8,8,2,3,0,0,9,0,5,8,5,6,9,1,7,2,4,6,9,5,1,8,7,8,5,3,3,5,5,1,0,4,9,2,7,0,5,6,2,0,6,9,6,6,0,6,0,7,2,1,2,3,9,1,3,4,4,8,2,9,9,4,5,1,9,2,3,4,1,5,4,7,5,2,5,5,8,7,2,2,8,6,8,0,9,1,6,5,0,0,5,1,5,2,2,4,5,7,8,6,5,4,5,0,6,1,7,7,8,2,9,9,0,7,1,0,1,7,7,3,8,2,4,5,7,9,9,2,8,9,0,3,3,5,5,9,8,5,6,9,9,8,0,1,5,1,1,8,0,1,1,0,3,8,5,0,9,6,4,7,5,6,3,0,4,8,0,2,5,8,3,4,6,5,6,4,6],[9,4,9,0,6,9,6,6,7,8,5,5,5,4,1,1,7,3,5,8,5,0,3,4,5,0,2,1,8,8,7,7,5,8,0,3,0,6,9,7,6,6,2,1,2,5,3,2,1,8,0,6,0,5,2,6,7,7,7,5,5,6,3,0,5,5,3,5,1,4,4,7,2,8,9,5,6,4,7,7,4,9,3,5,6,6,1,4,3,8,1,0,6,6,3,1,1,8,8,4,5,2,2,7,3,3,2,9,9,1,8,3,0,3,8,9,1,9,5,6,9,6,7,6,3,2,9,6,0,8,1,7,2,5,5,7,8,7,8,9,1,6,2,3,0,1,2,1,2,7,0,4,4,7,2,9,1,1,5,3,1,8,1,6,3,6,3,3,5,4,2,6,0,5,0,2,8,4,6,0,2,6,6,8,5,8,9,6,2,4,9,5,3,0,1,6,8,7,2,4],[1,4,2,3,9,2,8,7,7,4,0,1,2,6,9,7,7,8,3,9,4,4,4,9,5,8,8,3,7,0,9,0,4,2,3,6,6,1,5,5,7,7,6,9,6,7,6,3,7,1,4,4,8,0,5,3,0,3,8,7,5,0,7,2,2,3,8,0,4,5,6,2,3,4,3,9,4,0,4,1,3,8,7,1,0,3,6,1,6,7,8,4,7,6,6,9,9,6,9,5,1,7,9,4,2,3,5,6,5,1,9,0,9,9,2,0,2,0,1,0,7,1,4,4,9,2,5,0,8,7,6,0,6,5,6,0,0,4,8,5,5,8,6,5,7,0,7,1,0,0,3,0,1,8,4,3,0,2,3,1,1,9,1,7,7,9,8,7,3,6,3,9,6,1,6,5,1,5,8,3,5,2,3,8,2,0,1,4,2,7,5,5,6,6,4,3,8,4,3,1],[3,8,2,9,9,8,7,2,3,5,5,0,9,9,1,1,9,4,6,3,1,1,8,0,0,4,5,0,9,8,1,2,8,4,3,7,4,0,9,0,8,7,2,7,6,5,1,5,0,9,0,3,0,0,5,2,4,1,2,3,1,6,5,0,2,9,9,6,1,1,6,1,8,1,1,4,6,2,1,8,1,1,2,3,3,9,8,7,0,2,1,4,8,8,4,0,9,5,9,3,6,5,4,4,8,5,8,5,9,1,3,0,4,7,6,7,7,4,5,9,6,8,3,5,6,9,7,8,5,6,1,1,4,5,8,2,3,6,7,2,0,3,5,4,2,1,2,9,5,7,1,1,7,6,8,5,8,6,3,3,2,4,6,6,2,4,1,7,3,0,9,3,3,4,7,6,7,1,7,2,0,0,4,7,7,4,3,5,0,6,0,5,3,8,3,7,5,4,4,8],[7,5,3,0,0,2,8,7,4,8,2,4,8,8,4,5,2,7,2,5,5,2,2,8,3,5,7,8,2,1,8,9,7,3,1,9,5,2,8,1,0,0,8,8,0,2,6,3,1,8,0,6,1,2,7,6,9,4,4,1,8,4,2,5,7,6,6,2,8,4,6,8,7,4,8,9,8,4,4,9,3,4,7,6,8,6,2,8,1,8,1,1,2,4,6,9,2,4,3,0,0,9,0,9,3,8,9,1,3,3,2,6,0,0,2,8,6,6,6,9,4,8,0,6,4,8,5,8,4,8,8,5,0,0,4,3,8,5,7,1,1,9,9,1,1,3,9,0,9,8,1,5,8,4,3,4,2,0,2,9,9,2,4,1,2,0,4,0,6,3,4,7,5,5,0,8,9,9,8,0,9,2,8,7,6,1,1,0,2,3,9,3,5,3,4,9,6,0,2,4],[4,8,3,9,3,3,7,4,4,8,7,4,0,5,1,6,6,3,6,0,8,8,3,6,3,7,5,9,8,7,3,4,5,8,5,1,1,4,5,6,2,2,2,4,7,3,2,6,8,1,6,7,9,0,5,2,9,0,4,9,0,9,5,7,8,0,8,9,5,4,7,9,6,9,6,6,5,8,4,3,9,0,2,8,2,7,3,4,0,7,5,0,6,1,9,4,1,8,6,8,2,5,0,0,5,6,8,2,4,2,7,4,5,0,4,9,9,7,3,1,4,9,3,3,2,3,7,5,1,5,4,5,1,6,7,8,2,8,0,8,0,7,4,7,9,9,7,1,8,2,2,5,1,6,8,3,9,5,1,0,1,5,7,2,3,4,0,7,2,2,5,5,9,0,4,1,1,1,2,9,4,4,4,7,0,2,3,1,0,4,1,3,1,0,5,4,5,7,3,9],[1,8,6,2,0,1,3,1,2,7,3,8,2,9,8,4,4,1,6,4,7,9,7,0,0,4,6,7,3,9,8,4,7,5,6,8,8,2,1,2,9,4,1,1,6,1,8,0,2,4,4,1,5,3,1,7,9,9,6,2,8,5,6,7,2,4,7,0,6,9,2,6,5,5,9,1,6,7,3,0,3,9,1,9,2,4,6,1,3,5,3,3,0,1,1,2,6,8,4,4,9,6,0,7,2,0,0,8,7,4,1,3,3,2,2,8,9,0,1,4,5,7,8,7,8,1,9,6,1,3,1,3,2,3,0,4,3,2,4,3,6,5,6,2,0,0,0,1,2,3,5,8,0,5,5,9,6,7,7,0,2,0,3,4,4,5,0,9,7,7,4,6,4,2,8,4,4,0,5,7,3,3,5,4,8,2,5,7,9,4,7,4,5,2,8,1,7,1,2,4],[8,9,0,2,1,0,9,6,2,6,3,6,9,0,2,0,4,9,7,4,3,6,8,0,8,8,3,7,1,6,1,9,5,4,4,8,6,5,6,9,1,1,5,3,3,9,5,8,0,2,4,5,0,4,6,0,2,1,7,4,7,0,5,4,6,1,5,3,8,3,2,2,5,9,5,8,0,0,8,0,4,2,5,6,6,3,6,1,5,5,7,2,5,2,9,2,6,4,5,4,9,9,6,4,8,3,5,8,5,3,8,9,8,5,5,4,9,3,5,4,8,2,8,4,7,7,8,5,3,5,1,3,4,8,9,4,3,6,4,9,0,4,0,0,9,8,4,8,1,2,4,0,6,3,6,5,0,6,0,6,1,2,1,7,2,0,3,5,7,7,6,9,1,7,9,2,5,5,3,8,9,7,0,6,0,6,1,3,4,4,9,7,6,2,6,8,2,9,5,1],[8,4,0,9,3,1,4,0,9,7,8,8,4,9,6,5,7,0,0,2,4,9,1,0,1,8,0,5,7,7,7,6,1,9,7,6,3,1,6,4,0,7,4,5,6,1,2,5,3,2,9,7,3,1,9,6,1,9,1,8,8,0,6,2,2,6,8,7,9,5,1,0,2,7,7,0,8,9,7,1,1,7,0,6,0,9,2,1,8,5,1,9,8,0,1,0,6,1,9,7,6,0,7,0,9,6,2,8,5,0,1,8,9,4,4,1,3,6,2,4,2,3,3,0,5,6,2,1,7,1,1,4,3,0,6,2,7,1,2,2,3,6,1,2,0,7,3,5,4,7,9,8,2,2,0,8,8,4,1,6,5,2,2,0,3,0,4,0,1,9,4,6,5,7,8,7,5,3,2,1,2,2,9,5,6,1,3,7,5,4,5,2,9,9,4,2,9,8,4,3],[7,0,9,4,8,0,3,3,5,6,4,8,0,5,3,8,8,8,7,3,4,2,5,3,1,9,7,3,9,3,8,9,4,7,5,2,9,9,7,7,7,3,5,9,0,0,7,8,8,5,1,4,9,8,8,1,9,7,4,8,1,4,9,7,1,5,1,3,6,0,0,3,3,7,2,3,7,1,1,7,8,4,1,8,4,1,1,3,1,7,3,4,1,5,1,4,0,2,7,6,4,9,1,7,8,3,2,5,6,5,4,5,9,8,5,5,9,8,8,0,5,3,4,6,8,7,2,0,1,2,6,5,1,7,2,0,2,6,7,1,1,4,8,2,2,3,7,3,1,7,4,8,3,0,6,1,0,8,2,1,0,0,7,4,0,1,6,2,0,5,3,1,9,3,6,3,6,5,7,9,3,3,9,8,5,5,9,5,4,3,7,6,6,6,2,6,9,0,0,9],[6,6,3,7,9,9,1,8,4,0,7,9,3,7,7,8,4,9,4,0,4,3,9,0,9,1,6,0,2,9,0,0,5,5,7,4,4,0,2,0,0,2,2,3,1,9,4,7,0,0,8,5,3,7,7,4,0,4,4,2,5,6,2,0,1,2,6,7,2,1,8,3,3,2,6,6,3,2,3,4,2,3,1,5,0,8,1,3,4,8,5,9,6,0,9,8,2,8,7,4,9,7,9,4,9,8,0,3,0,5,9,3,9,0,0,1,0,2,4,5,2,2,4,8,4,6,8,6,4,6,2,5,3,2,9,3,0,1,8,2,6,7,5,7,9,6,1,9,0,7,4,2,1,1,2,5,9,1,3,5,7,8,0,2,2,1,7,4,2,5,6,0,4,2,0,3,0,3,3,2,0,9,6,4,2,8,1,3,1,5,8,8,5,0,1,7,1,0,3,5],[6,9,6,0,3,6,6,5,9,9,9,1,0,5,7,3,6,9,6,7,6,7,8,1,7,1,0,1,1,3,6,9,2,4,0,8,0,6,3,1,7,3,5,9,0,2,4,6,3,3,6,9,0,6,0,9,7,0,2,0,5,9,0,0,3,0,8,6,8,1,9,7,6,6,8,7,1,3,5,4,8,1,4,8,7,6,9,6,9,4,7,4,3,7,4,8,9,2,6,9,6,8,8,4,4,6,1,7,1,9,2,9,2,8,9,2,4,1,8,3,5,7,0,8,6,6,8,7,1,7,6,9,5,6,3,1,3,7,9,6,6,1,6,8,1,5,2,5,6,1,1,3,0,1,1,7,9,2,4,0,9,3,9,6,9,3,9,4,0,8,1,8,1,9,6,2,6,1,0,5,2,1,8,4,4,2,1,3,4,8,4,5,1,5,3,2,8,2,7,0],[1,0,8,4,1,7,7,7,8,9,2,2,0,3,6,6,5,0,9,1,0,5,8,3,1,1,5,1,5,4,2,8,4,0,3,5,9,2,5,7,1,9,1,3,2,8,9,9,0,0,0,0,6,0,3,9,1,0,0,9,5,4,7,1,5,0,9,4,4,4,4,7,5,5,0,8,5,1,9,5,2,0,7,0,2,2,1,4,3,1,5,8,6,2,9,3,5,0,9,9,4,5,7,2,1,7,0,8,9,9,6,3,1,3,3,4,6,6,0,1,7,7,1,5,9,2,0,4,3,2,4,9,7,3,1,0,0,3,9,1,5,5,6,6,0,1,2,8,7,4,9,7,1,0,4,3,3,5,7,8,7,3,7,6,6,9,7,9,2,6,2,7,3,9,6,3,0,0,4,8,7,3,7,0,6,1,3,1,6,3,9,5,6,8,2,5,9,9,4,2],[7,6,1,2,5,9,7,8,2,1,8,1,5,5,1,3,6,7,4,3,0,5,0,8,3,2,3,3,3,9,7,2,6,0,4,3,2,2,1,4,5,1,7,0,6,8,5,3,7,9,8,9,6,8,0,0,1,3,5,6,5,4,1,1,4,7,6,8,9,0,2,7,1,9,7,8,0,3,1,9,4,9,9,3,9,9,5,0,4,0,9,9,6,0,2,0,7,9,9,9,9,3,6,0,5,5,0,7,0,1,6,7,2,7,0,2,8,5,4,3,7,5,4,3,5,9,5,5,8,6,6,9,0,4,1,7,9,2,4,2,5,2,9,0,0,9,2,8,6,8,3,3,4,8,8,1,9,3,6,7,0,4,8,2,8,1,9,8,5,5,0,3,7,1,3,7,2,7,8,8,7,1,3,1,1,3,3,0,6,1,9,8,8,9,0,6,1,1,6,8],[6,8,1,6,9,6,5,1,5,3,1,3,5,6,4,8,9,9,9,8,3,8,6,1,8,9,7,1,0,4,9,9,2,3,5,4,9,2,7,5,6,1,0,3,7,4,1,9,6,2,7,1,3,5,2,3,6,1,4,7,7,5,8,2,0,5,6,2,7,5,9,3,6,9,8,6,3,0,5,1,4,4,2,7,1,6,0,8,8,6,7,7,2,5,9,2,0,7,6,9,3,5,5,9,4,3,5,8,5,2,1,0,8,4,7,0,0,0,0,0,6,7,8,8,4,9,3,4,7,9,5,0,5,0,1,9,6,9,9,1,1,3,1,2,7,1,4,9,1,4,2,7,1,2,8,5,1,3,1,8,4,6,8,9,9,2,1,5,1,2,8,4,5,0,6,4,3,0,6,6,4,8,5,7,0,3,2,1,8,5,2,3,4,0,2,3,4,5,0,5],[0,8,0,5,0,6,2,3,9,8,1,3,8,7,1,0,0,5,3,9,1,5,4,5,8,6,0,2,4,0,8,4,8,8,1,9,6,3,4,5,1,6,1,1,5,4,1,5,9,5,6,2,0,0,7,8,9,9,1,5,9,1,9,0,1,2,1,7,6,7,5,9,3,8,1,0,2,2,6,1,7,4,4,0,7,3,8,8,3,1,5,4,4,6,4,5,8,7,5,4,5,2,4,0,0,7,1,2,9,9,5,9,5,1,9,4,5,9,2,0,3,7,6,7,5,3,5,4,0,0,8,5,2,4,8,2,1,9,4,3,8,1,4,5,3,3,0,0,4,2,0,9,2,8,9,7,1,4,3,2,4,4,9,8,8,7,0,2,8,6,5,6,7,9,2,2,4,4,2,0,8,4,2,0,3,1,0,4,5,5,6,1,9,8,9,8,5,1,2,6],[9,7,2,8,8,6,1,4,0,5,6,9,0,8,1,3,9,3,7,6,1,6,9,0,4,8,0,1,1,2,9,2,1,2,3,9,0,4,5,3,1,2,4,1,0,7,4,2,3,4,8,4,0,8,4,6,8,5,9,2,9,9,6,1,3,9,2,5,5,8,0,7,0,4,8,2,4,5,4,7,9,5,3,1,5,9,9,5,6,8,9,6,9,6,7,4,5,9,0,1,9,2,8,9,7,8,4,3,3,0,2,2,5,5,3,2,6,4,0,3,5,9,9,4,5,8,1,1,9,1,2,9,3,2,0,2,0,4,7,4,5,9,8,2,6,4,7,3,8,7,8,3,6,9,8,2,7,9,3,8,2,7,7,7,1,8,2,1,4,1,7,1,1,6,4,9,0,3,4,8,0,2,4,6,3,4,8,0,5,3,1,7,2,8,6,3,8,0,7,5],[2,4,6,5,0,2,6,0,5,1,1,5,5,5,4,9,9,2,1,4,8,2,3,2,3,1,8,3,2,5,8,6,9,7,3,2,9,9,4,7,2,7,4,8,2,8,9,3,3,2,9,3,5,4,5,0,6,5,3,0,0,4,6,2,1,1,6,2,0,0,1,3,8,6,3,2,6,4,8,9,6,7,4,3,2,2,3,0,7,9,0,0,3,8,4,6,9,0,0,9,2,2,4,2,0,7,7,6,1,5,8,0,2,4,3,6,6,7,6,4,8,6,6,1,4,0,9,5,2,9,5,6,1,9,9,3,9,6,2,2,3,0,2,7,6,6,4,3,5,2,9,3,1,5,6,7,7,5,3,1,4,8,7,8,9,8,3,0,6,5,5,9,7,7,7,4,5,3,9,2,5,8,5,6,5,3,4,2,8,9,5,5,9,2,5,0,1,0,1,7],[6,6,9,5,5,6,9,1,9,8,5,6,8,1,5,3,6,9,7,5,0,2,2,9,7,9,9,8,9,2,7,5,0,8,1,6,4,2,9,5,3,4,2,1,7,7,5,4,8,4,1,8,7,5,9,4,4,0,4,3,3,3,1,3,2,4,1,6,6,2,4,1,7,6,3,6,5,0,2,3,4,3,3,3,8,4,7,2,4,3,8,7,7,9,3,9,3,4,7,1,7,1,3,6,9,8,4,4,0,7,9,4,0,4,8,1,8,7,3,3,1,3,2,8,2,5,9,7,2,8,9,1,0,4,9,1,2,3,6,2,0,7,8,3,4,8,4,2,6,9,7,9,3,0,9,5,7,8,3,9,6,2,2,8,6,1,0,0,5,8,4,7,7,2,0,1,3,4,4,1,4,1,0,7,3,9,4,1,9,7,2,7,1,5,8,9,6,8,9,3],[8,3,1,7,8,3,9,1,8,5,2,4,8,4,1,2,5,5,5,6,5,7,3,6,2,3,6,1,3,7,4,1,3,7,9,3,1,0,6,1,5,0,5,3,4,8,7,1,5,2,7,0,2,2,9,4,6,7,5,1,6,2,3,9,9,4,4,0,4,0,3,1,0,8,6,6,6,6,9,4,0,6,4,2,1,5,7,9,2,4,0,9,6,5,0,6,1,5,8,7,7,2,8,0,2,7,6,1,3,8,5,3,4,1,8,7,7,5,6,9,1,9,8,8,4,1,6,6,6,6,5,3,0,4,3,3,3,2,4,6,0,1,1,6,2,9,6,9,6,4,9,8,3,9,6,0,0,4,8,8,0,3,2,1,9,7,6,2,1,0,0,3,1,2,2,5,3,8,5,0,4,4,0,0,5,6,2,6,2,0,4,2,5,8,5,7,8,1,1,1],[3,2,5,6,6,9,2,1,9,7,1,3,3,1,3,8,9,5,4,3,7,1,6,5,1,3,2,9,5,3,1,8,7,8,5,3,7,7,5,6,4,8,9,7,2,5,7,3,0,4,7,0,5,5,5,6,8,9,8,3,4,9,4,2,7,9,7,6,8,4,2,4,3,3,3,7,8,0,2,1,4,9,1,1,6,8,0,5,7,8,0,1,9,6,5,6,7,3,2,5,7,6,1,2,9,4,1,8,5,4,9,1,5,2,3,2,0,3,9,9,1,1,0,0,8,8,8,5,1,2,1,0,8,2,3,9,9,6,7,4,0,8,5,8,2,8,0,2,3,1,3,4,2,6,6,0,4,6,6,5,8,9,7,8,3,2,8,2,9,7,8,1,8,6,9,2,6,9,5,0,0,0,4,3,6,3,5,0,9,1,7,0,2,5,0,6,7,8,0,6],[8,9,8,8,5,9,0,1,9,7,3,1,8,0,6,6,3,2,9,4,5,6,4,8,3,5,6,1,5,6,9,5,7,9,5,4,9,6,8,0,5,3,3,5,3,0,2,6,4,1,3,9,9,9,9,3,4,7,6,2,6,5,9,5,7,5,0,8,3,0,8,8,3,1,4,7,3,8,5,7,1,0,9,0,0,0,3,4,8,9,8,4,7,8,9,4,3,1,2,8,3,2,6,9,5,2,8,9,0,5,8,1,6,9,4,6,0,7,2,0,9,1,4,6,1,5,2,6,9,6,4,2,8,2,1,5,5,9,4,7,5,5,1,3,4,5,9,4,4,1,4,3,4,0,1,7,8,3,3,7,1,9,1,1,2,5,7,7,6,3,6,3,8,7,6,3,4,5,7,9,9,4,4,5,6,6,5,4,1,8,3,5,8,5,6,2,2,3,9,8],[9,7,2,7,7,8,0,1,6,0,2,7,4,7,4,2,5,9,7,8,0,2,3,0,9,0,2,1,5,3,0,4,0,4,2,7,2,4,1,0,4,5,9,0,2,4,3,9,5,2,8,5,6,1,5,6,3,9,7,9,2,7,5,5,1,9,2,6,4,5,8,8,1,0,1,5,4,6,5,1,8,5,9,6,6,4,2,2,4,0,1,8,9,8,3,3,8,8,1,2,3,9,2,6,9,3,2,5,9,9,9,9,4,8,6,2,4,8,4,0,0,7,9,0,6,2,5,6,2,6,8,8,7,2,4,9,6,8,4,7,9,3,7,5,3,5,0,8,5,4,8,6,4,7,8,2,2,5,8,4,1,8,2,8,0,9,7,8,9,4,6,9,9,5,4,3,0,4,3,7,1,1,3,7,9,3,9,3,8,9,9,1,7,2,2,9,3,1,8,2],[5,4,1,7,9,8,2,1,4,5,8,5,6,4,4,7,9,3,2,0,4,2,1,3,6,5,3,9,7,1,1,4,7,5,1,6,5,5,9,9,0,9,7,9,5,1,8,7,5,1,7,1,5,0,5,1,6,8,2,5,9,5,9,6,0,3,4,5,8,5,7,1,6,4,2,2,5,2,1,2,5,0,4,0,0,9,1,8,7,3,3,8,9,5,6,1,8,2,9,6,7,6,9,3,2,1,7,7,4,0,0,9,0,4,2,3,5,5,3,4,1,7,2,2,2,8,3,2,0,4,8,9,2,0,4,4,1,4,4,7,4,4,9,7,0,1,0,5,6,3,9,7,2,3,9,6,3,5,8,5,1,9,4,4,9,0,0,0,4,4,0,1,0,9,8,0,2,0,7,0,3,6,0,8,1,1,4,4,6,5,9,0,4,5,6,3,6,6,5,2],[3,5,5,3,6,5,4,8,5,3,1,1,2,1,9,3,2,5,8,1,0,9,1,6,7,7,9,5,5,7,7,8,4,5,2,1,0,8,1,6,1,2,7,3,3,8,9,6,3,9,7,6,8,0,2,5,7,4,0,4,1,0,3,5,5,7,8,5,7,2,3,0,6,0,6,2,0,5,8,4,4,7,2,2,9,4,0,8,0,2,2,1,2,7,9,9,6,9,7,3,3,0,6,0,3,2,2,5,7,2,1,3,1,3,7,0,0,7,8,0,2,2,4,6,0,5,6,8,4,5,4,8,7,0,0,2,2,2,8,1,4,9,6,5,3,3,7,5,3,5,5,5,9,9,1,9,6,9,0,1,6,4,1,4,4,1,6,8,5,6,1,9,6,7,6,1,0,3,6,3,0,1,8,9,3,2,1,9,3,1,2,0,5,5,4,1,8,2,9,3],[1,2,4,7,9,0,0,9,5,6,5,5,9,5,7,4,7,8,6,3,1,0,3,6,6,9,7,6,3,8,0,4,0,6,1,9,9,3,0,4,9,7,2,1,3,9,5,2,7,3,5,0,4,0,6,0,9,5,6,3,5,8,7,5,5,1,6,4,4,8,0,6,6,2,7,1,3,4,5,2,8,3,2,2,3,8,4,3,5,2,8,0,1,5,7,8,8,5,4,3,6,4,1,4,9,8,7,2,4,2,5,4,5,7,6,9,8,2,4,3,5,2,6,8,9,5,6,8,1,0,3,9,6,4,3,5,4,2,8,0,4,5,5,0,2,3,1,0,6,5,6,1,9,2,1,8,9,7,8,0,9,3,1,5,9,6,3,5,8,3,6,3,8,1,5,2,4,8,5,2,3,1,5,4,5,6,2,4,5,3,7,4,6,8,2,8,7,7,3,5],[0,9,0,0,2,5,2,7,3,7,9,8,0,7,2,5,3,7,0,9,2,9,3,8,9,7,6,6,4,2,2,6,1,2,6,4,0,9,3,5,8,4,4,9,1,8,4,5,7,6,6,9,7,1,0,7,9,6,3,5,0,7,2,2,2,8,8,4,9,1,9,8,5,5,9,7,6,5,4,3,2,0,3,1,1,3,8,2,1,4,8,2,3,0,6,5,0,4,9,0,5,1,0,0,8,9,9,4,6,5,8,8,5,3,2,9,6,0,1,9,4,1,1,8,3,7,5,4,1,7,6,6,0,8,9,8,9,0,5,5,6,3,6,3,6,8,2,4,0,6,3,7,9,7,5,3,4,2,9,8,1,7,6,1,5,5,2,6,8,7,3,4,0,9,7,8,9,2,4,0,0,7,7,9,4,4,4,1,8,3,9,0,2,5,1,7,3,3,5,1],[2,9,7,4,0,6,2,0,8,8,2,0,6,9,0,2,5,6,5,3,0,6,3,2,2,7,2,7,2,9,0,5,8,7,9,1,3,4,1,2,2,3,4,0,4,4,5,9,3,0,2,3,7,6,7,1,5,1,0,7,1,0,4,1,7,4,2,0,0,5,4,2,0,9,5,4,5,2,3,8,4,8,3,1,6,3,4,1,6,4,0,7,6,5,1,3,1,5,6,1,1,2,5,1,1,0,8,9,4,3,9,9,1,5,2,7,8,7,0,4,3,1,4,0,6,7,5,9,2,1,0,5,4,5,9,7,8,7,8,2,0,0,3,4,5,6,1,3,3,2,9,8,5,5,0,3,2,6,2,5,9,2,0,5,9,1,5,9,8,5,4,1,5,7,5,0,3,6,5,8,0,5,9,5,2,9,0,5,7,2,2,9,4,4,6,4,6,3,5,4],[9,9,7,4,7,2,7,2,1,2,1,3,9,0,1,2,1,1,7,9,4,9,0,8,3,6,4,1,0,0,6,1,9,3,5,8,8,4,1,9,7,4,2,8,4,3,0,7,5,9,8,9,0,8,9,6,5,6,7,7,6,5,8,7,1,5,8,9,0,1,0,9,5,2,7,1,8,0,8,5,1,7,6,4,7,7,2,2,3,9,1,9,7,9,9,0,7,9,9,7,0,1,8,5,3,7,8,3,7,8,8,1,7,6,5,5,4,7,7,7,8,1,9,7,2,0,7,9,9,8,8,9,9,8,6,3,6,6,8,3,4,9,4,2,5,1,7,9,8,6,7,9,7,6,6,2,8,4,3,7,2,2,6,2,0,4,7,6,0,7,2,6,6,8,0,2,0,7,1,0,6,0,1,5,8,8,7,6,4,1,3,6,5,1,0,7,7,9,6,9],[7,0,6,3,8,6,7,0,6,1,3,2,1,4,9,2,2,7,8,6,0,4,5,7,7,5,4,5,7,2,6,4,2,4,9,1,3,7,3,9,8,6,3,1,1,2,3,3,1,4,2,3,8,7,0,7,4,5,4,1,9,1,7,2,5,7,5,0,6,8,1,6,5,4,7,6,9,1,1,2,5,3,6,5,2,6,2,9,3,7,2,3,8,0,7,5,9,2,6,7,0,7,3,5,4,0,3,5,3,7,7,0,0,5,5,5,4,8,6,9,5,8,2,5,0,1,2,9,3,0,8,6,0,1,3,6,4,7,1,7,4,0,0,6,6,5,3,2,5,9,1,2,8,6,7,8,7,2,0,3,2,8,9,4,2,2,0,8,1,3,7,7,4,7,4,2,3,9,4,8,9,7,3,9,3,0,9,1,4,1,6,9,2,5,3,4,9,4,4,1],[7,1,8,3,9,4,7,4,6,1,4,7,1,7,6,4,0,7,7,4,9,3,3,1,8,9,7,8,5,1,1,2,2,9,8,3,6,5,7,2,9,2,9,0,9,7,4,9,4,4,6,3,7,1,6,8,0,3,6,5,6,7,0,9,8,8,2,4,5,2,8,4,4,7,6,5,4,3,7,1,7,3,4,4,4,3,2,5,6,0,2,3,9,2,4,0,0,6,6,8,0,5,2,4,4,9,0,1,4,9,2,1,2,6,7,6,9,0,3,8,2,6,3,4,0,7,4,3,5,0,1,6,7,5,2,2,4,4,3,8,3,5,1,5,3,9,4,5,1,9,5,5,7,8,9,8,5,3,1,2,6,4,8,5,9,1,9,6,5,2,4,9,9,8,6,5,7,0,2,0,0,7,5,9,5,5,7,2,0,0,4,8,4,5,4,4,6,3,0,3],[8,6,2,9,6,1,4,3,1,8,3,3,5,1,3,2,6,2,4,8,5,1,7,1,6,3,5,4,6,7,7,6,6,2,6,2,3,2,8,4,1,3,0,8,4,5,1,2,9,7,1,4,0,0,6,6,3,3,0,1,3,0,8,1,2,6,5,5,0,3,1,3,9,3,2,5,8,5,8,0,4,1,6,5,1,4,3,6,8,4,9,3,4,7,6,8,5,1,3,6,7,6,9,6,0,3,1,0,8,1,0,3,4,7,8,7,1,1,3,1,7,3,6,3,2,2,1,0,6,6,6,3,3,7,1,3,1,4,5,1,6,6,4,2,5,4,0,8,8,3,0,7,8,8,1,3,3,4,3,9,3,9,2,6,8,3,1,9,7,6,1,5,4,7,8,1,2,0,0,2,3,2,9,4,0,2,7,3,9,0,4,2,1,6,0,9,1,1,1,1],[9,2,6,6,1,6,7,5,6,9,7,2,1,9,8,4,3,5,9,2,7,4,6,8,2,6,9,4,9,0,5,1,4,3,7,6,2,6,1,0,6,1,4,9,2,2,5,5,9,5,0,8,1,6,8,3,5,8,9,4,0,6,5,5,0,4,3,4,1,6,4,9,9,9,8,1,3,4,9,3,1,9,1,4,7,2,9,4,0,9,1,0,5,8,7,7,3,2,1,6,9,8,5,0,7,5,2,2,1,1,5,4,2,9,8,1,1,8,6,3,9,7,5,6,5,5,6,0,7,9,6,8,7,3,9,6,1,1,9,2,4,6,9,8,5,7,9,8,7,5,1,8,2,9,5,0,4,3,0,1,2,7,2,0,2,1,8,3,4,7,8,8,4,7,6,1,4,7,2,4,3,5,2,7,4,9,7,0,2,8,2,5,7,4,7,9,7,5,5,1],[5,3,1,1,0,9,4,6,6,8,2,1,4,5,9,0,6,8,1,9,8,5,6,7,1,5,7,8,2,4,1,9,7,4,0,9,3,7,5,1,5,8,3,1,5,4,2,1,4,5,2,3,0,0,0,3,7,9,3,0,3,4,1,2,8,2,1,3,1,9,6,6,9,1,8,6,7,2,9,2,7,4,5,9,4,5,2,4,7,5,6,0,9,7,5,9,9,6,4,2,7,0,9,8,4,9,4,1,1,6,3,0,0,8,9,6,6,1,0,3,6,6,5,7,4,0,8,5,9,2,0,8,4,1,7,8,0,3,0,3,9,5,3,9,4,2,8,2,5,0,7,1,7,2,0,3,5,8,0,4,2,0,4,6,1,1,7,3,7,9,6,6,4,1,8,0,5,6,2,0,8,9,1,5,4,3,0,9,1,3,5,3,5,9,2,9,3,9,2,0],[8,1,8,2,4,6,5,0,4,7,0,3,9,4,0,5,7,3,4,1,6,1,4,3,2,6,2,7,7,7,7,5,8,8,0,2,6,7,4,3,4,5,6,5,1,8,0,8,1,6,9,9,7,6,5,2,4,7,9,2,6,9,9,4,7,1,9,3,8,5,6,5,0,4,0,1,5,3,2,8,9,3,8,9,9,5,1,6,2,0,8,9,9,9,5,8,1,6,4,1,2,2,6,4,9,9,8,4,2,2,2,1,5,2,2,7,7,3,3,2,4,3,3,5,4,0,6,5,9,2,9,1,4,7,7,3,6,5,7,0,7,2,4,5,4,6,2,4,0,7,6,6,2,9,3,6,1,9,4,0,1,3,3,8,0,1,1,9,8,1,9,8,3,3,5,9,2,7,3,2,6,1,0,8,2,3,4,4,5,8,4,8,3,0,6,4,3,0,5,1],[3,4,1,8,0,6,9,2,5,3,6,1,6,6,1,9,1,6,5,8,6,1,7,0,1,5,6,4,7,1,8,2,5,9,0,7,8,0,1,3,5,7,7,3,5,8,2,7,6,7,7,5,1,4,5,2,2,1,9,1,4,7,4,1,8,4,9,6,6,0,2,3,0,1,7,5,9,1,4,8,1,2,3,2,8,0,6,2,3,5,4,7,2,8,0,3,4,9,1,3,2,3,6,4,4,5,9,6,7,6,6,8,0,9,0,0,1,8,3,4,4,7,3,8,5,3,1,1,5,3,4,7,8,1,1,3,6,2,1,5,0,7,3,0,8,5,1,9,4,4,5,0,3,8,8,8,1,2,9,6,7,4,5,5,7,8,0,5,1,1,1,3,0,6,4,8,2,7,7,8,1,2,8,6,2,8,4,6,0,5,4,7,1,0,5,0,0,5,6,3],[9,7,7,9,3,1,0,7,0,7,7,1,2,5,7,6,4,3,2,4,0,7,4,4,9,9,4,9,4,0,3,5,9,0,5,5,3,5,4,3,4,2,6,8,7,5,5,3,0,9,0,0,6,4,4,5,5,1,7,1,3,0,7,5,2,4,0,5,1,6,8,5,8,6,6,8,1,3,1,1,2,3,3,1,9,0,8,4,1,5,6,4,7,5,9,9,9,1,4,0,0,4,7,8,0,3,8,1,6,2,4,1,5,8,2,5,8,2,1,1,0,7,7,7,2,7,7,1,0,3,3,0,8,1,1,0,6,9,4,3,3,8,6,9,8,0,6,6,2,7,9,2,7,7,2,9,6,9,3,6,4,6,9,2,9,2,5,6,3,9,1,7,7,7,8,8,9,4,4,1,1,6,6,8,3,8,0,9,9,5,7,3,1,6,8,3,0,3,9,4],[4,0,1,3,9,9,1,8,5,8,1,8,4,7,7,7,7,9,8,8,4,5,4,7,4,4,0,4,9,1,8,3,1,1,6,0,0,0,0,5,8,2,4,2,1,3,1,1,4,9,9,0,4,3,7,0,7,0,5,6,1,5,1,5,7,0,7,9,0,8,7,8,2,3,2,3,6,5,4,2,6,4,2,2,9,1,3,9,1,8,7,5,5,9,0,2,1,9,4,1,9,1,1,1,4,3,5,2,0,9,4,6,5,8,8,7,9,1,8,1,1,5,8,9,6,0,1,7,9,7,0,9,8,1,2,4,6,7,6,6,9,2,4,6,0,3,5,0,6,3,3,0,1,1,9,9,1,2,9,2,0,9,3,0,1,6,5,9,5,1,6,4,6,0,3,6,5,8,6,4,4,1,4,5,2,5,6,5,7,5,0,9,5,3,0,8,1,5,7,7],[8,5,3,4,8,6,3,5,7,1,9,1,3,5,8,7,0,4,3,0,2,5,9,7,8,9,7,2,4,6,9,5,2,2,1,2,1,4,7,0,6,9,3,1,4,3,8,7,9,3,7,1,8,6,0,9,6,7,1,2,4,0,7,8,4,1,2,7,7,9,7,5,8,2,6,5,5,7,2,5,0,9,6,1,5,7,0,3,6,3,6,2,5,5,2,1,6,4,9,4,4,8,9,4,3,8,9,8,5,3,5,7,2,2,8,0,1,0,3,9,3,1,2,8,7,6,2,5,1,1,9,7,1,1,1,4,1,3,5,6,6,0,3,1,4,4,3,7,4,6,7,0,8,1,0,7,7,2,2,0,5,2,7,7,5,9,1,6,4,8,4,2,9,9,5,3,3,8,1,8,7,0,0,7,1,0,4,0,5,6,1,0,0,8,7,7,9,1,5,5],[9,9,8,8,9,5,4,2,4,7,2,3,7,4,0,8,5,6,0,0,4,1,0,5,2,0,4,1,3,0,7,2,9,7,3,0,2,9,3,8,6,7,1,3,2,3,3,9,9,3,9,4,7,1,1,9,3,5,0,6,5,9,9,7,8,4,7,3,3,2,1,9,0,5,4,4,8,7,3,0,2,4,4,9,5,7,8,9,2,1,7,0,2,8,9,1,2,8,4,7,1,7,8,3,2,2,7,3,1,2,3,4,6,9,3,3,6,4,4,0,7,2,2,9,0,3,2,5,2,8,2,5,6,1,8,0,3,7,3,5,9,6,1,7,7,6,0,5,0,5,6,9,7,0,9,9,4,3,4,8,2,7,3,0,0,3,0,3,2,4,0,3,2,3,0,0,0,0,5,2,5,3,2,4,6,3,4,2,6,8,0,0,5,5,0,7,0,1,1,2],[7,3,5,1,7,7,3,7,7,1,9,3,4,3,9,2,6,3,4,3,2,6,5,9,3,6,7,3,9,0,7,8,5,4,9,4,1,3,1,9,4,3,4,0,8,3,3,5,9,7,0,1,6,5,2,9,1,9,3,2,1,0,0,7,7,0,1,8,5,5,9,9,0,3,9,8,9,2,5,8,2,5,1,8,1,3,9,4,5,2,9,6,5,9,5,2,1,9,2,6,6,2,5,8,7,7,8,8,1,4,6,5,9,7,5,2,3,5,9,0,9,8,8,4,7,4,8,1,5,3,7,3,7,5,1,4,4,9,3,5,5,1,3,5,1,8,9,4,3,8,4,5,6,2,9,6,8,0,9,5,5,6,8,2,3,9,8,7,1,1,5,6,5,0,3,6,8,3,2,4,3,8,9,2,0,0,8,1,2,7,6,7,5,5,1,1,6,0,0,7],[3,7,6,8,7,9,4,6,2,6,2,8,4,1,0,7,3,8,0,6,7,8,5,4,5,7,5,2,9,6,1,2,3,7,1,1,9,7,9,3,6,1,1,2,4,1,1,7,1,1,5,8,0,3,5,5,0,0,9,1,8,1,3,2,0,6,5,9,4,6,3,0,9,4,2,3,8,4,2,9,7,0,8,9,3,3,5,5,5,6,6,4,9,9,8,0,6,5,1,2,1,6,2,0,3,6,5,1,2,9,0,2,9,0,1,2,5,6,7,1,3,5,7,4,7,7,4,5,2,8,7,5,4,1,7,7,9,2,8,2,1,9,4,3,1,5,5,7,4,5,0,9,0,7,3,9,4,0,4,8,8,1,3,2,4,0,0,4,4,0,8,7,1,4,0,3,1,8,0,5,3,0,6,5,9,2,7,5,2,1,3,2,5,8,4,9,0,6,5,6],[9,3,3,0,9,6,3,1,4,3,6,9,5,5,6,4,7,3,9,1,7,2,3,2,2,9,3,2,8,9,8,7,4,4,7,4,0,1,5,4,6,3,5,2,8,1,6,7,7,8,8,4,2,1,6,5,3,1,7,1,2,8,8,7,2,7,3,2,0,0,8,9,3,3,1,4,6,9,3,3,7,2,7,0,5,5,7,0,7,4,1,9,4,1,8,6,9,3,0,1,3,8,0,9,3,3,5,0,3,8,3,2,2,3,4,0,8,1,0,7,8,4,9,2,5,9,9,6,3,1,0,8,0,0,7,3,4,4,5,9,5,9,1,9,4,8,9,2,9,2,2,7,8,3,0,3,2,1,2,7,2,2,6,4,2,5,0,8,0,5,9,7,6,3,6,2,1,8,7,0,2,9,0,0,2,2,3,6,3,7,4,7,9,2,4,4,7,6,4,9],[3,4,8,0,7,5,4,8,3,1,0,5,0,0,5,4,4,0,3,7,0,7,7,9,1,1,5,0,9,2,2,2,6,0,4,5,5,9,5,0,0,5,7,3,8,4,7,2,7,0,2,7,9,1,8,0,4,6,1,3,8,3,5,6,5,0,1,3,1,6,5,1,3,3,4,3,9,2,6,6,4,0,5,4,3,6,6,7,2,9,2,2,4,7,8,0,9,9,3,0,7,8,2,0,3,8,4,5,2,2,3,7,4,9,3,7,7,9,6,1,9,8,3,5,5,3,5,7,4,0,7,1,1,1,3,6,2,7,1,4,1,5,1,5,6,6,4,3,6,0,6,7,8,1,2,6,4,0,3,8,0,2,1,3,6,6,0,8,4,1,2,7,8,6,5,4,2,9,9,0,0,5,9,0,6,4,6,2,4,9,2,6,4,3,2,0,0,2,8,6],[5,2,5,4,0,0,0,5,0,0,7,2,7,7,2,4,3,9,6,9,0,9,7,4,4,9,4,6,3,4,2,9,9,0,5,9,0,5,4,2,7,2,6,5,1,1,9,4,0,7,3,2,8,0,7,3,2,3,1,5,0,6,6,9,6,1,0,6,9,7,1,6,1,7,1,2,0,2,8,2,2,3,5,0,5,4,5,7,9,7,5,9,3,1,0,1,3,1,9,2,0,0,8,1,0,2,3,0,4,3,3,6,6,0,9,3,4,6,1,3,3,6,5,8,9,5,9,2,8,9,6,8,1,7,9,3,9,4,4,5,9,9,4,5,9,3,9,5,9,2,8,5,0,5,3,9,1,5,4,9,6,0,0,9,7,9,3,6,6,9,4,5,8,8,3,9,1,4,4,2,8,4,7,8,0,3,9,1,0,3,2,6,6,2,5,3,4,0,2,2],[9,6,7,9,4,2,8,7,6,2,9,4,7,7,4,7,2,4,0,2,9,4,8,5,9,5,1,5,6,3,7,7,1,6,7,7,9,5,4,5,0,5,2,7,2,8,6,6,2,6,8,4,2,8,9,1,4,2,6,2,5,5,1,8,2,8,5,1,4,9,8,4,5,2,3,9,1,1,8,5,7,6,9,1,5,1,3,1,3,1,3,1,7,4,9,9,5,7,2,1,8,2,7,5,5,0,7,8,1,5,3,0,1,5,3,6,8,6,9,3,0,4,4,7,1,6,8,6,3,2,7,1,4,6,7,1,8,6,9,9,1,5,1,2,2,6,1,0,3,0,3,5,5,0,2,6,6,2,4,1,4,1,4,0,9,3,2,7,9,3,8,0,8,9,5,0,7,6,2,0,8,6,7,5,6,1,1,4,3,5,5,9,8,1,0,7,5,4,4,4],[7,2,7,8,3,2,0,1,8,3,3,8,1,3,4,9,4,7,3,0,5,0,9,5,1,1,5,6,5,9,3,3,4,0,3,9,2,3,2,2,8,8,0,9,1,6,0,7,4,3,7,9,3,7,6,7,8,1,3,4,3,6,9,7,6,2,8,0,7,3,2,8,3,5,9,4,1,0,1,7,5,1,8,1,8,5,8,6,6,3,2,9,0,3,8,8,7,7,1,7,2,3,5,5,0,4,1,2,6,4,1,2,5,0,3,3,5,3,2,1,6,6,1,8,0,9,9,9,8,0,6,2,5,1,7,6,8,0,0,6,5,1,8,0,1,3,6,8,6,0,0,5,8,3,3,0,4,2,0,5,4,6,7,0,0,7,8,0,7,8,6,2,1,5,5,5,0,3,3,9,5,5,4,3,0,7,4,7,2,6,2,6,4,1,8,4,8,6,6,6],[6,3,0,8,0,5,5,0,0,0,9,5,8,3,9,8,3,5,7,5,3,9,3,7,3,2,4,1,0,0,9,7,5,0,7,5,7,4,6,8,4,7,3,2,3,4,3,6,9,0,3,4,2,8,2,5,2,8,6,3,0,6,2,6,8,1,1,5,5,9,3,1,7,7,4,0,1,9,8,3,9,1,7,1,1,1,8,4,9,5,9,2,3,3,8,3,4,9,8,1,9,2,2,8,1,8,8,2,7,6,5,7,9,5,0,0,6,9,6,8,6,5,0,1,0,0,4,4,9,2,7,0,6,2,8,7,0,8,0,0,6,7,9,7,4,9,8,3,0,6,3,8,2,3,1,2,5,5,9,6,0,8,7,6,0,7,4,3,8,6,3,4,5,2,2,0,3,2,5,6,8,8,4,2,3,6,7,8,3,8,4,3,6,1,2,9,1,8,2,9],[4,7,5,1,9,9,3,2,1,8,0,2,6,5,6,9,3,5,9,6,5,4,2,4,7,4,3,8,2,7,9,8,4,7,1,3,6,5,7,0,5,8,4,4,5,2,5,0,8,5,8,3,9,0,9,6,6,2,7,0,1,8,0,5,5,2,0,4,9,0,6,4,0,0,8,5,4,4,7,2,1,5,8,0,6,7,8,4,2,7,5,3,6,5,1,3,9,1,7,8,1,5,5,1,7,3,8,2,9,5,4,0,1,4,2,9,4,3,3,6,0,0,1,8,8,2,2,7,4,1,8,7,7,3,9,4,6,7,6,8,5,3,0,8,9,5,7,3,8,2,1,8,5,5,7,3,7,1,2,3,2,0,1,1,3,2,6,2,1,4,2,8,9,2,6,9,7,5,2,7,8,6,6,5,1,5,8,0,8,2,4,0,3,7,2,8,1,0,0,2],[6,2,1,6,7,9,7,4,5,9,4,5,7,2,0,8,7,0,9,5,2,5,7,5,2,1,6,5,3,6,9,0,1,2,8,8,2,5,4,7,6,8,2,4,0,4,2,9,4,3,6,6,8,4,4,2,7,0,7,1,8,7,1,9,9,1,9,3,6,4,0,4,4,4,8,7,8,3,6,2,8,3,1,7,9,5,1,6,7,1,7,5,8,0,7,9,3,8,3,1,4,5,6,9,0,4,6,0,9,4,5,8,9,6,7,8,3,8,7,0,1,6,7,9,9,6,1,2,5,6,4,9,1,0,0,3,6,8,4,8,5,9,6,4,5,5,5,0,5,4,2,7,0,1,8,9,8,9,4,5,5,8,6,7,0,9,0,8,7,6,6,2,5,2,9,2,9,4,2,5,8,6,2,0,8,2,2,8,2,6,5,9,6,1,8,6,2,1,4,2],[7,1,6,5,5,5,7,7,1,2,2,1,8,6,2,6,8,6,4,2,2,9,2,8,3,2,6,5,3,2,9,1,5,6,6,3,1,5,0,5,7,4,6,8,2,0,6,0,6,3,5,0,2,9,8,7,1,6,5,5,9,4,8,6,0,6,9,4,1,1,1,1,5,7,1,7,0,7,0,8,0,7,9,5,6,9,2,9,8,9,4,7,6,4,5,6,0,7,2,4,8,3,5,4,3,6,3,5,5,5,3,6,2,4,1,8,4,5,8,2,7,4,1,3,9,6,1,9,3,6,3,4,9,0,8,4,8,3,1,4,9,5,2,1,9,5,2,5,2,2,7,9,6,0,4,7,9,6,9,4,2,4,8,3,5,8,0,5,2,1,1,3,8,3,4,0,0,8,5,5,0,5,6,9,5,1,6,4,7,5,9,1,0,7,6,7,8,8,2,0],[0,6,5,0,1,1,0,2,0,6,9,2,1,5,1,6,6,0,3,5,7,4,8,9,3,5,8,1,3,3,3,5,9,8,6,0,0,6,4,2,4,3,4,5,9,8,4,5,0,9,3,7,3,1,7,6,6,7,0,2,0,3,9,9,4,5,0,6,4,6,8,8,0,2,6,1,2,2,6,2,1,1,0,6,3,9,2,1,6,4,3,9,8,3,8,4,0,0,0,6,7,8,7,9,2,3,0,5,7,8,9,8,0,9,4,3,0,8,4,9,3,0,8,3,5,8,7,7,9,9,4,8,7,3,7,1,8,7,8,5,7,8,5,7,9,1,2,2,9,9,1,4,1,1,7,6,9,6,5,0,5,1,8,4,4,5,8,2,4,6,9,2,4,4,1,4,5,6,6,5,5,9,1,6,2,1,4,1,7,9,4,5,1,2,1,5,0,9,0,6],[6,9,0,2,4,4,6,1,0,4,8,7,5,0,3,7,1,9,1,0,8,5,7,1,9,9,9,9,0,9,8,8,0,8,1,6,2,9,8,4,6,6,1,1,8,4,9,9,5,0,2,4,7,9,7,6,0,6,6,3,7,6,1,8,4,4,4,9,6,2,3,2,1,5,5,9,1,6,9,7,6,3,3,5,2,0,2,3,7,0,8,4,6,9,4,2,6,9,1,2,3,5,6,4,2,1,4,3,8,5,2,6,8,5,2,2,8,4,7,5,6,5,1,2,7,6,4,3,7,6,7,0,3,3,7,7,4,3,2,4,0,5,1,0,2,5,2,0,9,0,7,5,7,1,9,4,9,3,9,6,1,6,8,6,9,7,5,6,0,8,0,0,5,3,0,9,8,5,0,7,7,9,4,4,2,5,1,1,9,0,9,2,9,0,1,8,7,6,4,8],[4,7,0,1,0,3,1,9,0,3,8,7,2,5,3,5,0,6,8,1,9,8,4,8,8,7,8,5,3,5,5,0,2,8,1,2,1,2,3,3,5,4,0,0,9,5,5,1,2,3,3,1,3,9,1,1,6,9,9,9,4,6,1,6,4,3,1,7,7,6,0,3,0,2,5,1,8,0,3,2,5,6,5,9,7,6,2,3,5,3,4,2,0,6,0,4,1,3,2,8,0,2,3,2,5,8,4,5,0,9,7,8,7,2,9,4,0,1,9,7,5,3,9,7,1,0,1,4,3,5,5,5,8,8,0,5,9,4,2,1,5,9,9,2,3,0,6,3,2,5,0,9,0,2,6,2,2,9,8,7,5,3,5,5,4,5,2,5,9,4,6,6,5,8,8,0,8,4,5,2,9,7,3,1,9,1,5,3,1,4,1,6,7,6,3,3,1,5,8,2],[1,5,8,6,3,6,8,3,2,3,6,3,0,9,4,2,1,0,5,4,4,6,2,3,4,5,7,7,2,5,1,3,2,9,1,5,7,9,9,1,4,7,4,6,8,1,8,1,1,4,5,7,2,7,0,9,4,9,6,6,5,8,9,7,9,2,5,9,1,6,2,7,5,7,4,3,8,2,5,1,6,0,8,1,0,0,0,4,0,8,1,7,6,2,4,8,5,1,9,8,9,1,6,4,0,0,8,0,4,5,1,3,5,1,4,7,2,6,2,4,4,5,1,3,7,7,3,4,9,2,3,0,5,9,5,6,1,5,8,5,0,0,8,7,3,2,5,7,8,9,1,3,4,2,8,3,2,1,0,1,3,3,3,8,2,8,6,3,3,5,0,3,7,9,1,0,1,8,0,2,7,1,7,3,6,5,8,8,6,8,1,9,1,4,9,5,3,8,0,8],[5,1,2,2,0,5,4,3,3,4,5,2,8,2,5,4,9,3,2,5,4,3,6,5,7,6,3,2,6,3,1,1,6,3,5,8,0,9,2,3,6,9,5,4,4,2,0,3,7,2,9,1,7,5,9,6,3,2,9,1,7,0,2,4,5,7,2,7,9,6,2,5,6,9,1,0,1,1,5,8,5,4,0,2,2,1,8,7,3,9,9,0,1,1,6,8,1,9,5,0,7,7,7,3,6,0,5,7,1,3,8,8,7,0,0,1,1,0,9,6,0,8,6,3,1,3,2,4,4,7,4,1,5,3,7,3,3,4,3,6,7,1,4,7,3,6,0,4,7,9,0,9,9,8,2,3,3,6,7,7,4,4,1,1,7,0,6,1,6,9,7,6,0,4,5,5,0,5,9,7,7,1,8,6,0,3,1,5,9,9,5,5,3,6,8,0,8,5,3,6],[4,3,2,7,7,9,2,7,5,4,7,2,5,7,0,7,0,2,3,2,1,8,7,6,6,8,6,6,3,2,4,7,5,9,6,4,8,9,3,3,3,2,5,0,0,6,0,2,8,5,6,1,5,6,7,3,4,5,1,7,7,5,6,4,4,3,0,3,2,6,8,7,8,4,9,8,2,9,3,2,6,9,3,1,5,2,4,9,7,5,8,7,3,5,3,7,8,6,2,2,2,1,1,0,7,0,1,9,2,4,3,8,5,6,2,3,8,8,4,7,6,3,6,9,0,0,8,8,6,1,2,8,4,3,0,1,5,1,2,7,7,5,8,5,3,0,8,3,8,4,0,4,9,7,5,9,9,4,9,7,7,1,7,1,6,7,4,2,1,6,1,0,3,9,5,6,1,5,9,0,2,1,6,1,8,2,1,9,8,0,6,5,4,3,8,0,3,2,4,4],[0,6,6,3,7,2,9,9,9,0,1,1,1,7,3,2,1,6,1,1,6,8,8,0,3,6,3,6,0,9,2,0,5,9,5,5,3,4,4,2,6,5,6,8,4,1,0,8,7,1,1,3,1,0,6,5,8,1,3,9,0,6,1,8,5,7,3,8,3,9,2,0,6,0,8,0,1,0,0,8,3,2,4,5,2,2,0,0,3,5,1,5,1,3,3,8,2,8,6,5,7,1,5,3,1,5,6,5,7,6,3,1,0,9,6,4,1,8,7,4,3,8,2,7,1,5,5,5,6,4,1,3,5,8,9,8,6,7,3,3,5,9,4,6,8,2,0,2,2,7,8,8,8,0,5,1,8,0,7,4,4,8,9,1,8,8,0,4,5,5,0,1,6,6,9,7,9,1,9,3,1,7,1,9,8,6,0,6,7,9,2,3,9,1,5,8,2,7,4,9],[4,4,0,1,1,1,8,2,3,9,5,6,6,7,5,6,5,7,4,2,7,6,6,8,8,1,6,0,0,1,9,4,7,2,5,8,5,5,2,8,6,0,6,5,7,1,1,2,1,6,7,0,4,5,8,2,8,7,4,8,0,4,2,7,8,0,8,3,7,0,4,6,2,0,1,9,4,2,4,5,0,3,7,7,8,5,9,8,2,4,6,4,0,0,2,8,2,0,3,0,2,7,6,5,8,9,6,2,3,2,9,4,7,6,1,7,3,2,5,8,8,1,2,8,2,6,8,6,6,2,6,1,1,4,8,1,3,6,5,9,1,4,5,8,2,8,6,6,0,1,4,9,5,8,9,9,5,0,5,3,4,4,6,5,0,6,7,6,3,2,5,6,7,0,4,1,8,2,7,0,6,3,1,3,4,3,2,9,3,9,2,7,3,1,2,6,7,1,2,0],[4,9,8,3,1,5,4,9,7,4,1,5,9,3,8,3,6,2,2,9,2,7,8,7,0,2,5,7,4,9,0,8,8,8,3,9,3,9,0,3,3,2,8,3,5,9,8,3,1,3,2,5,0,2,5,0,4,0,9,8,0,9,8,0,8,3,0,3,3,2,6,6,4,5,1,1,6,0,4,7,3,8,5,5,0,0,7,7,0,6,5,2,6,6,3,6,9,5,9,2,7,6,1,4,3,2,7,9,4,2,8,7,2,3,2,5,3,1,2,6,8,7,8,6,5,1,4,7,6,3,9,6,1,0,2,6,5,9,7,9,3,6,9,8,9,3,3,5,5,5,1,5,4,9,1,2,3,5,9,1,0,8,7,4,1,1,0,8,3,8,7,8,6,8,6,5,2,9,2,7,6,3,2,3,3,5,5,6,2,4,9,4,4,9,8,5,0,1,5,3],[1,5,2,7,3,0,4,5,2,7,4,8,2,8,1,5,3,6,3,5,2,5,2,7,4,0,4,4,3,0,0,4,5,4,3,0,4,8,6,6,7,0,5,9,1,8,7,4,7,2,2,9,7,4,8,1,6,3,8,0,5,8,4,2,2,0,2,6,8,0,3,7,1,0,6,2,0,5,8,9,8,2,9,7,6,9,9,5,4,7,5,9,7,1,1,9,1,6,7,1,6,2,8,9,2,5,1,5,0,2,4,0,4,5,8,3,5,7,8,9,6,5,1,3,8,4,4,0,0,1,1,7,4,2,6,8,9,0,3,1,2,0,2,8,5,0,3,2,9,1,4,7,8,5,2,7,9,6,9,2,9,2,9,3,4,5,4,3,7,7,5,1,9,7,0,5,9,3,9,0,7,3,7,7,0,9,4,2,7,3,4,6,6,5,2,0,2,6,6,0],[3,1,1,5,0,1,2,9,7,1,9,6,7,8,3,9,9,0,1,6,3,7,2,1,4,6,4,7,2,2,7,8,3,8,3,5,2,5,6,1,6,7,7,5,5,2,5,4,2,8,2,8,6,6,1,0,3,7,9,5,9,6,3,2,7,6,9,9,3,5,2,0,2,9,5,9,3,2,3,6,1,7,6,9,4,9,9,7,7,9,4,6,7,8,1,4,6,0,5,0,6,7,2,0,8,9,0,2,2,5,0,5,3,8,4,7,7,5,6,4,6,0,1,4,0,2,0,9,4,6,9,2,5,3,3,4,4,3,6,8,8,8,3,1,6,7,0,3,5,6,8,1,7,1,7,7,5,8,8,9,6,9,2,1,2,7,5,9,0,3,7,8,1,1,2,7,0,2,3,5,9,3,7,8,4,4,7,1,4,6,0,0,5,4,2,0,1,7,9,3],[1,6,4,4,9,6,4,0,0,7,5,9,0,4,9,4,9,7,7,3,3,9,4,0,4,8,0,7,5,9,1,8,8,7,5,7,3,9,7,5,6,3,5,6,9,6,2,8,3,9,4,8,8,0,9,4,8,9,4,5,1,7,6,9,4,3,6,9,2,6,4,8,1,1,6,0,8,0,1,3,1,7,2,1,7,1,8,7,2,2,4,5,1,2,6,7,5,5,6,7,1,2,7,2,6,5,4,4,7,5,9,8,2,1,0,1,4,0,0,9,2,7,4,5,1,3,2,7,8,0,6,1,4,6,3,0,1,9,6,9,5,6,9,9,9,1,3,4,1,5,3,5,2,9,0,4,4,4,3,2,4,1,3,1,7,8,1,1,8,0,2,5,6,1,4,5,5,9,1,6,5,6,2,9,8,4,5,2,9,8,5,5,2,0,6,9,9,0,2,9],[0,4,4,6,8,0,3,5,0,5,1,7,3,5,8,1,0,4,4,9,4,1,6,6,1,3,8,2,3,0,3,3,7,7,1,7,0,6,2,2,3,3,1,7,1,9,8,1,3,4,2,8,5,8,6,7,1,4,1,6,5,5,1,4,4,2,1,6,1,5,8,4,0,9,1,1,9,2,2,4,6,4,2,2,5,1,1,6,5,4,5,2,1,6,6,8,1,9,4,2,4,3,8,5,4,2,8,5,4,1,0,2,7,4,6,2,5,9,1,1,4,8,3,7,4,2,5,7,3,0,9,8,5,8,5,1,0,3,7,6,6,7,8,6,1,5,8,9,4,1,0,0,9,5,8,6,9,3,3,3,5,5,3,0,3,0,4,5,5,1,3,2,0,1,0,3,8,8,2,5,2,2,7,3,0,5,9,1,1,3,4,6,8,9,9,3,9,3,0,5],[6,3,9,6,4,9,9,5,9,2,0,3,6,7,7,8,5,6,0,6,1,6,4,1,6,5,4,7,0,4,4,6,9,3,4,4,4,4,1,6,8,3,9,6,2,6,5,7,5,7,5,6,3,2,0,1,7,6,9,0,3,3,6,2,9,1,8,5,7,9,1,7,2,3,3,7,1,0,6,6,7,2,5,3,6,5,4,5,1,5,5,4,1,2,9,2,5,7,9,4,7,3,1,1,6,4,8,7,7,5,6,6,9,1,9,7,8,6,2,1,3,8,8,4,2,7,8,9,4,8,3,3,3,6,7,9,2,5,8,9,2,4,6,3,7,7,2,5,3,5,9,9,5,9,5,9,6,4,0,2,4,5,6,7,1,3,8,5,0,6,5,5,3,3,0,0,0,3,8,6,0,7,5,5,6,0,6,4,6,6,6,2,1,4,9,4,9,7,9,0],[6,6,7,9,1,7,9,2,0,9,0,0,8,7,7,6,9,5,0,8,1,9,0,4,5,0,0,5,9,0,7,7,8,4,6,0,3,8,4,6,7,4,8,8,3,8,4,2,3,7,2,7,6,3,1,1,3,2,8,4,4,7,2,2,3,0,4,7,8,8,5,8,4,3,6,7,1,0,2,7,7,4,4,5,7,5,9,2,9,7,9,5,5,1,0,0,3,4,7,2,3,2,0,7,8,8,7,1,8,1,8,8,5,2,3,5,0,2,7,1,2,8,7,9,1,9,9,5,3,9,7,8,3,9,6,1,7,5,5,7,6,3,5,1,8,9,8,8,3,8,9,5,6,8,4,8,7,6,5,3,5,4,1,8,3,7,2,2,4,7,1,0,0,7,4,8,8,2,8,3,0,0,9,9,8,5,9,8,1,4,1,6,8,2,7,3,2,9,5,6],[6,8,9,8,7,3,9,5,7,7,9,0,9,0,9,8,7,8,8,9,4,1,7,2,3,6,5,7,5,2,6,3,0,5,4,0,0,3,5,7,2,6,9,2,8,8,2,6,8,2,7,2,3,4,6,8,1,3,6,8,5,2,4,8,9,8,8,9,3,5,8,7,4,8,9,2,8,3,0,9,5,7,3,0,4,0,9,7,3,5,5,1,9,1,9,8,9,9,7,4,6,7,2,0,5,3,5,4,9,5,3,4,3,6,7,7,8,6,4,4,1,1,5,2,3,6,0,4,5,9,1,1,6,3,4,2,8,9,8,7,6,3,4,9,1,1,8,0,7,4,4,0,8,1,2,1,7,2,7,4,1,8,7,9,3,1,1,2,2,1,9,9,4,3,0,6,4,9,8,3,5,4,3,3,5,7,6,4,9,6,8,2,4,7,2,8,9,5,2,3],[7,1,2,3,7,3,1,3,4,9,7,1,3,2,5,0,0,3,4,1,9,4,6,6,2,8,6,3,5,8,6,4,1,1,8,0,6,9,4,2,1,3,3,6,5,8,7,7,4,3,9,3,0,5,1,2,5,7,7,0,7,3,7,9,6,7,9,2,8,5,4,9,8,8,8,4,8,7,1,4,2,2,8,2,7,1,6,2,9,3,5,6,9,4,7,5,1,7,0,1,2,4,3,1,4,1,7,5,8,0,9,2,3,9,5,0,1,1,5,0,5,0,8,4,6,6,1,7,3,1,8,7,8,1,0,2,4,9,9,4,0,9,7,3,8,2,5,1,5,2,1,0,2,2,6,8,8,0,7,3,1,8,2,1,1,3,4,6,2,5,2,4,4,9,7,5,1,5,8,9,7,2,9,2,4,8,2,2,0,2,7,1,0,9,3,1,4,9,9,7],[6,2,1,1,1,1,8,5,8,6,4,5,8,5,9,4,5,2,8,5,4,5,9,6,5,4,9,9,5,1,8,1,3,2,4,6,3,4,3,1,3,9,8,3,7,0,8,4,2,6,0,8,2,1,6,9,5,5,0,2,8,1,3,1,3,8,0,8,4,5,1,7,5,9,1,4,9,9,8,3,5,0,3,9,3,9,0,0,7,1,2,5,2,8,9,7,6,1,5,2,6,6,0,3,7,1,9,9,0,8,4,7,0,0,9,4,1,9,4,8,0,9,6,4,7,7,3,5,8,8,7,6,6,7,2,4,8,1,5,0,1,1,0,2,1,9,8,3,8,4,3,1,3,1,7,2,8,1,7,8,1,7,5,8,4,9,4,5,0,1,7,2,2,7,6,4,8,4,9,9,0,2,2,6,4,1,8,4,2,8,3,6,5,0,4,9,9,0,6,9],[1,4,3,3,3,9,9,2,5,8,3,6,3,5,2,9,6,2,3,1,0,8,7,5,8,3,5,7,3,3,9,4,7,4,9,1,4,1,5,1,1,8,9,4,5,1,5,3,6,9,4,6,7,3,2,6,6,9,5,1,2,6,7,0,3,9,3,7,0,0,0,1,8,0,8,5,3,3,8,9,4,3,6,2,8,0,0,5,9,7,8,3,6,6,5,9,5,8,8,7,0,0,0,8,0,8,3,4,4,2,3,0,7,1,4,5,1,6,2,2,4,1,6,2,9,3,3,4,4,1,1,4,1,1,5,4,2,8,8,8,2,3,8,1,7,3,7,0,9,1,3,5,4,1,9,3,6,2,7,0,5,0,5,9,4,0,3,8,0,3,6,5,8,6,6,7,1,3,8,3,7,3,8,1,6,0,5,2,2,4,3,0,7,0,1,1,2,4,1,2],[9,7,9,9,5,6,7,7,1,7,0,8,0,0,0,6,0,7,0,5,1,3,5,0,5,6,3,7,2,4,2,3,1,1,4,7,7,3,6,9,0,6,9,2,8,1,0,9,8,1,6,2,6,1,2,2,9,6,1,3,2,3,8,4,7,4,3,6,8,9,7,0,7,7,5,5,8,5,6,9,8,2,3,5,5,7,9,6,5,0,9,8,6,9,4,3,4,7,9,4,8,7,4,7,6,1,2,6,7,9,7,5,3,0,2,9,0,1,5,5,4,7,3,0,6,9,5,2,6,6,6,6,5,3,3,1,4,8,0,3,9,7,1,2,0,3,1,0,7,9,7,3,6,3,5,2,2,2,7,1,8,3,7,6,8,3,9,5,3,9,8,4,9,9,6,9,5,0,9,4,1,6,9,7,9,4,1,4,6,8,5,6,4,4,2,4,9,2,9,2],[1,8,6,2,9,5,1,6,7,2,0,8,1,9,7,0,5,8,6,1,9,1,0,3,6,2,9,5,6,9,0,0,9,6,2,8,3,4,5,0,6,7,0,7,9,7,0,6,6,6,0,7,0,0,2,6,2,1,3,1,0,3,1,9,2,3,0,5,7,7,8,6,6,8,5,5,8,5,4,6,4,4,3,4,4,5,2,8,6,7,9,9,1,0,0,5,4,0,0,3,9,0,9,6,1,5,3,9,2,7,5,8,1,0,4,7,5,6,6,3,4,7,4,7,0,5,2,6,7,4,9,9,5,1,5,8,8,8,7,0,8,4,1,1,4,5,9,1,2,7,6,8,4,1,7,4,8,9,0,7,5,2,6,2,3,3,0,3,4,7,5,2,3,6,5,7,2,4,0,6,1,7,4,8,0,3,2,0,2,5,9,9,9,8,2,4,1,2,7,7],[2,2,1,5,9,7,5,3,1,5,9,3,4,5,1,6,8,5,6,2,2,6,1,1,4,5,5,7,8,4,5,0,7,6,7,8,3,2,1,7,0,2,2,6,9,5,3,7,0,1,1,5,9,4,6,5,0,4,3,0,8,0,2,7,8,9,5,2,4,8,1,6,2,3,2,1,8,5,0,0,7,1,5,8,6,4,4,8,8,7,8,8,9,0,6,7,9,1,1,3,2,2,1,4,5,4,6,5,1,6,8,0,0,3,9,6,9,3,4,9,2,2,8,1,4,4,8,3,7,0,9,9,4,0,6,2,6,4,7,8,2,5,8,2,1,7,0,0,0,6,0,2,0,8,5,4,4,4,0,1,6,1,3,0,1,1,4,8,7,2,8,9,9,6,4,2,4,6,3,6,3,5,1,5,5,6,2,1,2,2,4,8,5,7,1,6,0,5,6,7],[9,4,7,1,3,3,3,9,9,8,5,4,3,8,0,0,7,4,1,9,8,6,0,3,5,1,1,6,8,0,3,8,4,2,9,9,7,4,8,7,3,6,1,8,4,3,9,1,9,2,1,7,8,1,0,6,4,4,2,4,4,7,2,0,2,3,0,9,8,8,8,1,4,2,9,9,7,0,2,7,3,5,4,1,8,7,7,2,1,1,7,5,1,1,7,3,7,7,4,7,6,3,8,2,5,9,3,2,2,6,1,5,1,8,8,2,5,8,6,8,1,5,5,2,7,2,7,6,0,2,3,8,7,3,0,4,4,6,6,6,4,0,1,7,8,2,9,5,2,6,3,3,1,8,6,0,2,5,6,4,7,9,2,6,2,5,0,9,1,9,5,7,9,9,4,7,1,4,2,3,2,7,8,3,7,4,4,9,2,2,4,9,2,8,8,6,3,8,5,6],[9,3,5,8,2,0,7,5,6,1,8,8,8,6,1,5,3,7,7,5,0,3,6,4,1,6,0,7,5,8,5,4,1,1,5,5,1,2,0,7,4,0,5,2,8,8,0,3,6,7,0,6,0,9,0,3,5,2,0,0,2,8,7,5,9,2,2,2,6,2,1,0,4,8,5,5,6,7,8,2,6,1,0,8,0,2,1,5,7,4,8,1,2,5,9,3,9,1,5,5,6,6,8,0,6,5,5,2,2,6,7,0,7,9,8,7,4,1,4,1,5,2,4,9,7,3,2,8,5,9,6,1,7,6,3,3,1,1,8,5,9,5,5,6,6,5,5,0,6,9,3,4,2,8,5,1,1,0,2,8,9,8,1,9,4,7,2,7,8,0,2,7,7,9,5,4,6,2,4,4,1,0,8,5,8,6,9,1,6,1,2,5,9,3,6,5,2,9,2,0],[1,6,9,9,7,6,5,3,8,1,9,2,1,8,7,9,6,6,3,2,7,5,9,8,0,6,3,3,7,7,5,8,5,5,9,2,3,4,7,2,8,7,4,9,7,3,1,3,2,4,7,1,1,6,2,1,4,7,6,1,5,2,0,0,7,1,5,2,8,4,4,6,1,0,5,0,6,6,5,8,2,2,9,3,1,1,7,5,9,3,7,6,5,9,8,4,2,5,7,0,0,3,6,3,4,4,6,0,2,1,0,5,4,1,0,5,5,7,2,6,1,9,2,8,0,2,3,3,8,2,5,0,5,4,3,1,8,1,1,0,3,3,7,7,5,0,4,0,7,6,6,0,8,0,1,0,4,6,5,2,8,1,4,3,5,0,5,5,1,8,7,4,2,7,3,9,7,9,9,6,8,7,9,8,9,0,8,3,6,6,8,4,9,4,9,6,4,6,1,6],[7,0,2,9,7,8,0,4,7,1,3,7,8,2,5,9,4,6,4,0,2,4,6,1,9,7,7,3,6,0,1,3,0,6,4,8,4,4,4,3,5,7,1,5,1,6,6,5,2,0,7,6,5,5,7,4,3,6,9,1,6,1,4,9,7,8,7,1,2,3,4,9,3,7,6,4,4,2,2,8,2,1,5,9,7,4,3,2,3,3,3,1,6,7,0,5,7,7,6,1,3,2,0,6,0,6,2,6,8,4,4,2,6,1,2,3,8,7,7,1,2,0,2,8,9,3,3,6,2,1,7,5,6,7,1,8,5,4,4,5,0,0,9,8,4,1,3,2,1,0,5,3,0,9,2,1,4,7,7,7,9,4,4,7,3,8,5,8,4,1,5,4,1,5,3,7,8,8,9,9,9,6,5,1,6,7,3,0,6,0,9,7,5,6,4,0,4,9,9,8],[0,4,4,4,1,9,1,0,8,3,1,7,9,6,0,7,5,3,0,2,6,1,9,1,7,4,1,1,5,2,1,8,9,6,2,0,7,3,2,5,8,4,4,0,2,5,7,8,0,9,0,6,1,1,9,8,7,3,2,3,5,3,1,6,1,3,9,9,8,3,6,7,7,1,9,2,8,6,0,8,8,2,7,9,3,6,9,1,9,1,6,7,7,7,5,8,2,4,7,2,8,6,1,7,7,0,9,7,9,1,5,7,3,4,8,7,1,7,0,2,1,8,9,8,7,7,6,1,1,6,3,1,2,7,1,1,9,2,8,8,4,5,7,7,0,5,6,3,5,8,5,6,6,7,4,3,4,2,4,7,8,0,9,2,9,2,5,8,4,3,9,8,1,6,8,3,4,6,6,9,7,1,5,3,0,1,9,4,3,5,2,4,7,3,6,6,5,2,5,9],[7,4,0,0,0,0,3,6,8,9,5,5,3,0,1,3,1,2,8,7,7,2,1,5,5,9,1,2,1,8,1,1,2,3,1,5,5,5,1,4,6,9,9,9,9,2,3,3,4,3,0,2,5,3,9,2,2,2,4,6,1,7,7,5,1,0,2,6,7,6,2,4,5,2,5,4,6,0,9,1,3,1,5,8,4,4,0,7,8,6,3,1,4,2,9,7,4,1,3,2,7,8,8,2,2,5,9,8,6,0,9,9,2,4,0,8,0,2,7,1,9,2,2,3,4,3,0,9,5,5,3,4,5,3,7,7,8,8,6,4,8,7,4,2,4,6,3,6,8,0,7,9,3,2,2,7,5,4,8,2,2,3,7,7,6,6,7,7,4,3,1,4,2,7,7,6,3,0,3,2,2,2,1,5,4,6,5,2,2,5,4,6,9,1,6,7,7,3,4,3],[8,8,8,0,5,7,7,9,9,2,3,1,4,6,7,9,4,2,3,7,9,7,3,8,1,1,8,0,4,2,4,2,0,4,3,8,1,2,7,0,4,2,1,8,8,0,9,3,2,2,2,2,2,7,2,5,9,2,5,3,5,1,8,7,5,3,7,8,7,6,0,1,8,2,1,9,2,3,4,7,5,6,1,9,5,5,4,4,8,2,0,5,5,0,4,3,5,2,1,2,8,2,3,9,6,6,8,0,9,2,9,7,0,0,8,5,8,5,2,6,7,4,1,2,6,7,7,1,9,9,3,0,3,8,9,1,4,7,1,6,1,1,3,1,3,1,8,1,6,2,9,5,8,2,0,4,0,7,5,1,8,8,1,3,8,0,4,3,7,6,1,0,9,4,1,2,7,2,6,6,6,5,3,5,8,3,9,8,3,7,9,3,7,3,7,6,3,1,1,3],[9,2,3,8,8,7,3,7,1,9,3,7,6,9,2,4,4,4,4,7,3,6,3,0,9,0,6,4,3,7,7,3,1,3,1,9,2,6,7,3,5,2,2,4,1,7,8,6,1,5,5,4,3,8,6,2,0,3,8,4,2,6,7,4,1,0,3,3,7,0,8,4,3,0,8,6,7,9,2,0,4,0,6,9,0,3,1,1,6,9,5,0,7,4,4,0,4,0,3,3,2,3,0,7,4,0,4,1,9,8,2,5,0,0,4,1,3,7,2,1,9,9,2,8,3,8,9,9,8,4,5,3,8,5,0,2,5,6,5,7,5,9,4,5,0,9,8,3,8,2,5,9,1,9,8,6,7,9,8,8,3,3,3,3,0,3,7,5,0,3,4,7,4,9,2,4,0,1,0,8,3,7,8,7,8,8,5,7,7,3,7,2,8,0,6,8,4,5,6,4],[0,0,3,5,9,5,1,9,8,3,0,4,0,0,3,8,8,8,8,7,4,5,9,2,6,7,3,0,3,9,6,3,9,9,8,1,6,2,0,7,5,2,1,8,2,4,8,2,4,6,9,0,2,9,5,8,6,8,0,1,7,6,5,8,7,5,9,3,7,0,0,3,4,3,1,7,9,9,9,6,8,1,6,0,0,1,8,8,9,0,0,6,6,7,5,5,2,4,0,0,6,3,5,1,6,8,0,8,9,1,4,7,2,0,9,4,2,9,5,3,9,5,2,7,2,9,2,6,5,5,6,2,8,3,5,6,3,7,4,5,8,8,4,3,1,4,7,3,3,2,6,5,9,8,2,3,7,7,0,5,2,8,7,0,4,4,6,7,1,3,2,1,1,9,4,2,3,4,7,8,8,6,3,8,6,8,3,6,5,3,1,7,2,0,9,6,4,7,5,7],[2,0,8,6,9,5,8,2,9,8,2,7,4,6,7,2,6,1,8,1,6,9,0,0,1,1,6,5,0,4,4,3,4,5,9,3,0,9,7,9,7,9,8,3,7,8,8,5,9,6,6,7,8,8,8,9,9,4,7,2,0,1,5,4,6,4,9,8,5,8,9,3,0,8,8,9,8,6,5,9,5,3,6,3,2,4,4,3,1,1,5,1,5,2,8,3,8,9,2,4,8,1,9,0,1,7,9,9,6,6,8,1,0,5,6,4,9,0,9,0,4,7,4,1,9,2,4,8,1,8,4,1,0,3,1,3,0,3,3,8,9,3,9,1,0,7,5,2,0,5,2,4,2,8,7,3,0,1,3,4,0,7,7,2,0,9,7,3,4,0,1,5,6,3,7,6,0,4,8,0,9,3,6,3,1,3,9,4,5,2,0,7,2,7,1,2,8,8,7,4],[1,9,0,7,2,7,5,4,1,6,5,3,1,1,8,4,5,9,0,2,4,0,1,6,0,4,0,8,2,8,5,5,7,5,4,1,4,0,5,5,8,2,0,9,4,1,3,1,0,4,5,4,6,6,2,6,2,3,7,4,3,2,2,2,7,6,3,1,8,0,8,6,3,1,7,9,2,1,0,4,7,7,1,5,5,3,2,7,8,1,3,1,5,5,3,4,2,8,5,0,9,5,9,4,8,6,3,2,7,5,9,6,2,0,2,7,5,6,6,6,9,9,7,4,5,3,8,7,3,5,9,2,2,8,8,1,5,3,3,4,8,2,3,2,4,7,1,0,3,7,8,2,7,5,8,2,0,6,1,4,1,2,8,3,1,7,6,8,2,0,4,1,4,9,5,1,6,7,3,1,4,1,3,3,8,1,7,9,9,0,3,0,3,3,6,6,2,2,6,5],[4,0,8,1,0,3,2,6,0,5,8,7,8,3,2,6,7,0,5,8,0,0,1,5,4,7,1,8,1,7,3,8,0,1,1,2,5,3,0,7,0,0,6,8,6,9,4,3,1,2,3,1,2,4,7,8,3,0,7,5,0,2,5,0,4,6,4,1,1,4,8,3,5,5,3,1,6,9,6,7,1,1,0,6,6,9,6,9,0,3,6,0,6,1,2,2,9,6,3,2,2,3,7,7,0,0,0,6,0,8,3,3,0,6,1,6,5,8,7,5,1,4,7,9,7,1,1,9,7,6,3,2,0,1,9,0,1,2,7,3,2,2,9,2,0,0,0,6,8,8,3,2,4,1,3,3,4,7,2,4,3,6,6,3,7,7,6,0,9,5,6,2,9,5,6,0,5,7,6,6,7,9,0,3,2,3,6,7,2,1,1,6,9,7,1,6,4,9,8,6],[4,4,0,4,1,6,6,9,5,4,5,2,5,7,5,8,0,4,7,5,5,0,1,4,9,4,2,5,4,0,1,0,7,1,4,8,0,2,9,5,6,4,0,2,1,7,2,4,1,9,9,6,9,2,2,0,8,4,5,4,7,7,5,6,0,1,6,2,4,6,8,0,0,0,2,4,7,4,0,9,5,1,7,6,5,0,6,3,4,2,8,3,1,3,9,3,6,6,6,0,2,6,3,4,6,7,0,5,4,0,4,9,3,4,8,8,4,4,4,0,8,2,4,9,7,5,5,3,1,1,6,5,9,1,0,5,8,2,0,2,3,7,4,8,1,2,7,7,8,1,9,9,5,3,0,4,1,5,9,2,8,5,0,7,6,2,2,7,4,5,9,7,4,5,8,7,9,5,6,0,8,5,9,3,1,9,7,2,5,8,6,3,4,6,1,2,8,5,9,3],[2,1,2,8,8,0,5,8,7,1,8,5,9,9,0,0,8,9,2,5,8,0,9,4,9,2,6,7,9,8,2,4,1,7,2,9,9,0,9,7,1,9,4,2,8,7,2,9,6,6,4,6,7,5,0,6,9,9,5,9,9,0,3,0,7,7,1,8,7,1,7,9,0,2,1,1,9,6,0,7,4,4,4,1,0,6,9,9,5,7,0,6,7,5,6,6,3,8,4,0,1,2,1,3,6,5,4,7,1,4,6,5,9,2,9,1,9,0,2,6,9,3,3,8,8,1,4,1,1,1,2,2,5,3,6,3,8,2,0,1,7,6,9,6,9,8,9,0,0,1,6,0,4,1,8,3,3,5,6,6,6,0,1,3,4,9,6,4,1,6,6,8,4,5,6,5,5,5,5,5,7,4,7,3,7,6,8,2,1,5,9,9,5,0,2,1,9,8,6,0],[6,4,1,0,1,7,8,6,3,3,3,2,9,1,5,7,9,6,9,2,1,0,1,8,0,5,0,1,3,8,4,1,2,5,1,5,4,9,3,9,5,6,1,6,9,9,3,8,5,5,2,8,5,3,6,8,0,8,1,5,6,5,6,0,2,0,5,7,1,8,6,6,7,0,5,6,9,8,7,4,5,9,2,3,5,0,1,7,9,2,3,5,0,9,8,4,9,5,1,1,4,0,9,1,0,6,9,1,7,8,7,2,0,9,5,7,1,8,4,0,1,7,8,3,7,8,7,8,3,1,1,7,1,3,0,3,9,2,4,6,0,1,1,2,2,6,9,3,7,4,6,0,1,6,3,0,4,0,1,7,3,2,7,4,7,7,9,9,1,3,7,4,6,8,6,8,7,6,4,4,0,0,4,3,6,7,4,0,9,5,9,5,9,6,1,7,6,1,8,7],[6,5,1,3,4,0,3,1,6,7,7,8,9,1,1,7,0,7,9,1,4,9,8,4,5,0,3,1,1,1,9,9,6,2,2,2,2,6,5,0,5,2,8,7,5,2,4,7,9,4,1,4,3,9,0,0,1,5,4,4,8,5,4,6,7,6,9,2,4,4,2,2,9,3,9,6,5,3,6,6,9,9,2,4,0,4,7,2,9,1,6,9,6,2,6,3,9,5,7,5,1,2,9,2,5,8,1,2,4,7,0,5,8,3,0,8,9,7,2,1,8,1,0,4,3,6,9,4,1,7,2,5,1,1,7,6,0,0,0,6,9,0,1,7,5,1,8,5,0,2,6,0,3,6,6,9,5,8,3,8,7,5,3,8,7,3,4,9,3,4,5,5,6,6,2,2,0,2,7,2,5,5,5,0,3,1,1,8,9,5,7,8,2,2,6,1,5,2,2,1],[6,7,6,5,6,0,7,8,3,6,2,0,3,7,2,6,1,4,7,2,1,6,1,3,8,7,7,6,0,1,9,8,9,5,3,7,7,2,7,2,8,9,4,1,9,7,0,0,1,7,2,2,3,3,7,1,3,6,9,3,8,8,3,9,3,7,8,3,9,5,5,0,6,0,1,7,9,1,9,0,0,2,4,3,7,3,7,2,0,6,7,0,5,1,1,0,8,9,3,9,6,1,9,4,1,3,2,0,6,3,2,7,5,8,2,5,1,9,9,3,6,7,5,1,8,6,1,8,5,7,7,1,8,9,8,9,4,2,1,2,5,5,9,3,3,4,8,4,3,7,0,9,6,5,2,4,2,6,2,7,3,2,1,1,3,9,2,9,3,5,1,8,0,3,1,3,7,1,9,2,1,9,4,7,5,6,2,9,2,6,8,7,8,9,8,3,0,2,2,3],[7,4,4,7,9,5,0,6,7,2,8,8,1,2,7,8,1,9,9,5,6,8,3,6,9,3,0,2,6,2,7,5,8,1,3,7,7,5,3,6,9,4,6,3,8,3,1,9,5,3,5,3,1,0,9,2,5,1,4,3,4,2,9,4,5,4,2,2,1,7,0,1,3,6,4,2,2,7,3,7,0,0,0,1,0,1,4,6,3,0,9,9,2,0,3,8,4,7,2,6,7,3,9,0,1,3,2,3,0,8,2,1,8,2,4,9,4,0,7,9,3,8,8,7,1,3,5,5,1,0,3,8,5,2,8,8,7,3,2,8,1,4,1,1,9,7,2,5,0,1,4,3,0,2,0,1,5,8,8,8,0,2,6,5,6,7,5,4,2,7,4,5,4,7,6,5,4,1,0,4,2,4,9,2,8,0,5,5,0,4,4,0,8,2,5,6,1,0,2,5],[0,6,0,6,3,9,1,0,0,3,6,2,9,6,7,7,8,4,2,8,0,8,8,0,3,5,7,4,7,9,0,9,8,2,7,3,1,0,3,3,3,2,8,2,0,5,1,8,9,6,8,2,4,8,2,9,5,9,4,2,1,6,4,1,8,1,4,2,2,0,7,7,2,5,2,4,2,3,2,4,9,2,6,6,0,8,5,7,0,1,1,1,7,5,2,8,9,6,2,1,8,9,0,2,5,2,6,7,6,0,1,7,2,7,3,4,6,9,1,6,2,3,9,0,0,1,0,9,9,2,2,0,1,3,2,6,7,9,4,5,9,7,3,4,5,6,0,3,7,2,1,0,7,0,2,7,3,2,9,4,4,1,4,5,6,9,2,4,8,8,9,9,5,2,3,2,1,4,5,0,8,8,2,7,0,6,6,3,8,5,0,4,9,6,0,5,5,4,1,5],[4,1,5,1,5,0,4,8,6,1,1,6,0,5,3,2,2,0,6,0,5,8,7,6,4,9,2,2,3,3,7,9,6,4,0,4,5,4,4,3,8,5,0,0,1,3,2,5,5,0,7,3,8,4,9,5,3,3,7,8,7,4,7,5,1,8,1,8,4,6,1,4,1,3,4,4,9,7,1,4,7,9,7,8,3,9,3,9,2,2,7,1,8,5,7,9,5,8,7,1,6,1,6,0,4,0,4,3,9,6,8,9,7,7,9,0,8,2,1,1,4,9,2,2,6,1,4,1,2,1,4,8,2,0,8,9,3,5,2,2,3,2,3,0,2,2,2,0,6,4,1,0,5,6,5,1,7,9,4,9,2,8,0,7,9,8,6,4,3,0,6,6,5,2,8,7,4,1,9,1,7,3,3,2,9,0,5,8,1,9,0,4,9,0,3,8,0,1,2,6],[1,1,4,8,3,3,7,9,6,7,0,5,2,6,7,1,6,4,1,0,3,3,6,2,5,9,3,6,0,7,2,3,0,6,4,3,1,1,5,9,0,7,4,2,3,3,5,2,7,7,2,0,2,8,5,8,9,0,4,9,7,8,4,8,6,8,3,8,0,0,9,2,8,4,5,1,7,0,5,5,9,9,7,2,7,4,0,6,4,6,7,2,4,4,2,2,4,7,0,6,8,2,9,6,6,4,9,3,4,7,0,6,6,8,8,6,4,0,4,9,6,2,3,2,8,5,4,2,4,7,9,2,9,8,0,5,2,0,0,8,7,1,4,5,1,4,1,5,4,6,6,2,0,9,4,8,4,1,2,1,8,1,3,7,1,4,4,5,6,6,4,5,9,0,0,2,7,2,8,3,8,6,6,8,6,2,8,2,3,0,3,3,2,7,2,3,3,6,9,1],[5,5,6,4,7,8,7,6,0,7,0,0,3,8,8,9,2,8,4,8,9,7,1,1,6,6,6,9,2,7,2,9,4,8,4,4,7,1,0,9,8,0,0,3,0,0,3,3,9,7,1,0,6,4,1,3,0,9,4,5,9,7,4,3,7,8,9,6,1,0,8,1,2,0,5,3,2,0,8,1,9,1,3,5,5,4,0,6,6,5,3,5,4,7,0,1,8,2,0,1,2,8,3,4,8,0,9,0,0,7,4,9,8,7,6,6,4,7,4,2,2,7,9,6,6,1,9,6,3,1,8,5,9,3,2,9,3,1,0,3,1,6,4,1,5,0,7,9,7,1,3,1,0,2,9,9,4,9,5,7,0,5,5,2,8,7,1,1,0,3,6,3,1,0,5,7,3,4,8,0,8,2,2,8,4,3,7,0,2,3,8,5,0,5,9,9,4,2,2,4],[6,1,0,9,1,5,6,4,1,5,7,9,9,1,8,5,4,7,6,9,2,6,6,3,1,7,2,5,9,6,1,7,7,1,7,1,8,3,7,2,0,4,1,1,7,1,7,4,9,3,3,1,1,1,4,4,8,8,1,7,7,2,5,4,6,4,5,6,9,5,8,0,1,0,3,9,1,0,3,2,5,8,4,8,1,8,2,9,9,3,8,6,8,5,0,4,9,8,2,9,5,1,1,6,3,4,5,6,7,0,9,2,0,3,3,1,3,5,2,4,9,1,0,7,6,3,3,6,3,5,5,8,8,8,4,1,2,2,0,9,4,9,4,5,4,7,8,7,2,1,2,3,2,4,2,8,9,5,6,2,3,3,2,1,1,7,5,6,1,5,5,5,6,1,2,0,0,1,9,5,4,3,8,6,8,3,6,7,8,5,2,3,8,4,5,2,3,2,8,4],[7,5,2,5,7,6,5,7,9,4,4,3,8,3,1,8,8,0,7,6,5,9,2,5,4,7,7,7,1,5,4,0,1,6,5,0,2,0,9,2,6,4,7,6,7,9,4,5,9,2,3,6,1,5,1,7,4,9,7,7,6,1,7,7,9,4,7,3,4,7,7,3,1,5,9,0,6,4,7,7,8,2,3,1,0,4,9,6,5,6,4,2,9,3,9,0,8,9,3,4,8,1,7,1,6,9,3,4,3,2,1,3,4,4,4,6,0,3,5,6,1,1,8,2,4,9,2,4,8,6,1,6,9,8,9,7,7,2,1,2,4,2,5,1,6,0,7,8,5,4,4,9,5,2,1,2,2,4,6,2,2,7,1,1,8,0,8,7,3,9,0,9,3,7,0,1,9,0,9,5,4,4,6,2,6,7,4,0,3,0,3,5,0,4,8,8,4,6,7,9],[7,7,1,0,7,3,1,6,3,1,3,0,5,9,2,3,9,8,4,2,0,9,0,0,3,8,0,9,7,8,1,4,7,2,5,4,5,8,3,1,9,8,3,6,0,7,2,9,5,8,1,5,7,3,8,2,2,8,3,1,8,6,5,6,0,2,2,6,1,7,9,2,6,4,1,6,1,3,5,8,1,8,5,0,4,3,4,6,4,7,9,2,4,4,0,4,9,3,2,0,2,3,4,8,7,5,6,0,0,3,8,3,4,4,5,8,9,9,6,3,9,5,8,5,9,0,9,0,5,4,0,8,9,7,8,7,4,5,9,5,0,8,0,4,4,6,4,5,5,0,1,6,5,1,1,7,1,3,7,7,7,0,5,8,9,3,7,3,0,7,0,1,7,1,7,1,9,2,8,6,4,9,3,0,0,4,9,2,7,8,1,6,0,6,5,9,1,2,3,2],[9,5,5,6,6,4,9,7,6,0,4,1,9,9,3,2,3,4,6,3,2,7,9,3,5,6,4,6,9,9,0,0,5,7,9,3,2,0,1,0,0,7,1,2,6,6,6,1,0,2,4,5,9,6,0,6,2,6,2,3,6,3,4,1,0,5,4,2,5,7,5,8,4,8,0,0,5,6,2,7,0,8,4,1,4,6,7,7,3,9,0,9,4,6,0,5,1,6,9,9,4,4,7,0,3,9,3,0,7,5,9,7,5,4,8,0,0,5,7,3,6,9,4,1,8,6,8,9,3,7,0,9,4,9,9,9,8,2,1,5,9,0,2,5,6,2,5,9,7,4,4,6,5,9,7,3,5,5,5,0,4,5,9,0,5,1,9,3,3,2,1,5,3,5,2,1,0,7,0,9,3,5,5,8,6,2,2,1,9,7,2,4,4,3,4,9,4,6,5,0],[8,8,7,3,3,9,5,3,8,7,3,1,2,0,1,8,3,5,2,2,2,6,8,9,1,5,8,6,1,5,8,1,3,5,7,9,4,4,4,2,1,9,5,6,0,6,6,5,2,0,7,6,6,8,7,8,3,6,6,4,3,6,7,7,1,6,8,7,0,2,9,2,2,6,0,4,2,6,9,6,9,8,5,5,6,2,5,9,0,3,5,4,9,5,3,0,1,1,7,2,5,8,6,9,4,6,3,9,4,4,7,5,3,4,1,1,7,8,1,9,4,8,5,3,3,8,6,7,1,5,9,9,4,7,8,0,5,4,1,1,8,1,7,3,5,0,5,2,8,8,4,2,6,9,8,2,0,4,9,1,9,0,0,5,7,1,8,4,5,9,5,5,0,2,9,6,2,6,0,3,4,6,7,2,6,5,6,6,9,7,9,1,7,2,6,6,3,6,0,8],[8,8,5,8,0,4,6,5,2,9,8,8,5,5,3,3,1,9,1,2,9,1,3,8,3,2,5,8,8,7,6,6,5,1,7,8,8,5,3,2,6,1,1,4,8,4,7,9,5,1,4,4,4,9,3,9,1,0,7,2,9,5,0,7,8,9,5,6,5,8,9,3,1,2,7,9,6,7,1,1,8,7,6,4,6,1,3,0,1,0,2,0,5,4,7,3,4,4,2,1,4,3,6,5,5,4,7,3,3,0,4,3,7,2,7,3,3,0,3,4,2,7,7,9,4,6,2,0,1,6,3,5,1,9,3,6,5,2,9,8,2,6,1,1,8,0,6,2,0,0,8,4,9,7,3,3,4,8,3,7,4,8,4,8,8,7,6,3,1,8,4,3,4,7,6,2,8,3,6,0,5,7,5,4,4,8,0,8,8,5,5,3,4,2,3,2,9,9,7,3],[7,3,8,1,1,5,6,9,8,4,1,3,1,6,9,6,7,9,6,5,5,4,0,1,6,3,5,7,5,4,0,2,8,9,6,9,4,4,8,4,8,1,9,2,8,8,8,7,8,6,4,5,0,7,8,6,2,3,6,7,7,6,0,5,7,8,6,3,2,6,7,0,8,6,4,8,5,2,5,5,1,1,2,1,8,0,0,1,5,6,8,4,4,0,2,2,8,8,5,2,5,5,5,5,3,9,3,0,4,8,5,5,1,9,6,0,1,8,1,6,6,1,3,1,4,5,5,4,5,2,7,0,7,2,5,1,1,0,1,7,8,9,2,0,8,1,2,2,9,5,0,6,6,3,9,2,8,4,7,6,6,6,6,6,0,4,9,3,4,0,1,5,1,3,7,2,4,1,4,6,6,4,4,4,8,3,7,8,9,6,4,7,2,3,5,2,9,4,5,5],[7,6,2,8,2,9,2,6,0,6,4,8,3,0,3,3,3,0,3,4,8,8,4,0,3,9,4,4,4,9,9,3,8,4,1,0,3,4,8,6,2,5,4,5,5,7,8,1,9,2,7,7,2,1,9,7,1,3,3,7,5,2,0,5,6,3,5,2,9,5,0,2,2,6,9,8,4,0,1,5,2,8,5,6,0,4,5,3,8,8,2,5,2,4,0,1,7,7,3,7,4,5,1,7,1,0,7,7,2,8,3,6,6,0,2,8,4,9,1,4,7,5,9,0,9,1,3,7,8,6,6,5,3,7,2,4,9,9,4,2,7,7,8,5,9,3,4,3,2,7,0,2,5,1,4,4,3,7,3,3,5,9,8,8,8,2,4,0,1,8,4,0,7,4,8,6,7,4,2,2,1,2,6,6,3,0,3,8,7,6,4,2,8,2,0,8,5,4,0,8],[5,4,1,2,9,9,9,8,3,1,2,6,5,8,3,0,8,8,9,7,6,5,9,4,7,1,3,4,8,5,3,3,0,4,5,1,5,6,1,8,9,6,4,4,4,9,5,5,7,6,2,4,1,4,0,0,5,5,5,3,1,0,8,3,6,6,4,3,2,7,1,2,3,7,6,0,7,3,5,6,9,9,2,2,3,5,3,1,0,0,6,3,2,5,6,0,1,0,3,5,0,4,7,3,3,6,3,2,9,8,9,1,0,3,3,5,8,8,6,1,0,3,4,4,0,3,4,3,5,7,8,5,3,6,9,7,4,2,9,3,3,0,4,5,6,0,0,6,0,7,9,1,2,6,5,2,9,2,7,4,9,5,0,5,3,1,4,7,5,5,3,8,8,9,5,4,9,6,2,2,5,2,3,7,8,0,1,9,2,8,3,4,3,5,1,9,6,5,6,2],[2,9,2,0,9,8,6,0,4,9,2,9,3,7,8,1,8,9,0,2,9,5,6,4,1,9,3,9,4,2,3,7,3,8,9,4,6,6,5,2,7,9,3,0,7,3,3,7,2,3,1,1,0,0,7,3,9,1,3,4,3,8,3,6,6,2,1,4,0,8,6,7,7,1,7,6,4,0,5,8,5,7,1,6,7,9,9,6,0,4,2,5,3,5,3,1,0,6,6,0,6,4,8,6,6,7,2,0,8,0,9,3,7,2,1,6,1,3,4,3,9,7,8,4,4,4,6,6,2,2,7,9,8,7,5,4,4,9,7,4,9,8,0,8,0,1,6,2,6,1,5,6,0,6,0,4,2,6,3,4,0,0,3,9,9,0,5,3,0,2,0,1,0,0,2,3,1,8,5,8,1,2,4,1,0,6,6,2,5,9,7,5,1,2,6,2,3,2,7,5],[4,7,8,7,7,2,0,9,1,7,9,2,9,5,6,2,3,2,4,8,3,3,4,6,6,0,8,1,4,5,6,9,3,6,8,0,9,8,1,2,7,2,6,6,9,2,0,3,6,5,1,1,8,7,7,6,0,7,7,4,3,5,5,8,2,3,0,3,3,4,5,0,8,1,9,8,6,9,3,2,6,4,4,5,4,3,3,4,1,1,8,6,8,4,4,2,7,6,5,3,0,2,3,9,6,2,9,2,4,2,4,0,8,0,7,2,4,1,6,7,4,7,3,2,1,9,5,0,5,2,3,8,5,9,9,1,1,8,3,7,2,9,8,0,2,5,5,6,8,3,5,2,0,8,7,3,7,4,4,4,6,9,2,1,8,1,4,2,9,7,9,3,9,9,4,1,7,1,9,5,4,4,0,5,4,9,8,1,3,4,5,9,4,0,3,4,1,7,6,3],[7,8,6,8,7,2,1,4,3,0,2,0,6,4,7,0,3,7,3,6,2,0,7,8,0,2,2,4,2,1,7,9,9,3,9,8,8,0,5,1,2,7,3,0,3,0,0,6,8,5,4,2,7,3,2,8,6,6,2,8,7,9,9,8,4,8,7,2,0,2,6,2,1,1,4,4,2,6,2,2,1,8,6,8,1,8,8,7,4,0,7,4,1,6,2,8,6,9,2,8,3,0,0,4,2,6,0,6,2,4,8,3,2,6,4,4,4,2,3,0,5,1,4,8,9,9,6,8,0,9,6,4,9,9,0,3,5,3,9,8,7,7,3,0,3,7,6,9,2,9,2,7,2,8,5,4,7,2,2,0,3,0,6,4,1,6,8,7,1,7,7,1,7,0,3,2,0,9,2,2,8,4,1,3,2,6,7,2,0,1,2,5,1,8,0,3,6,0,2,8],[9,9,1,6,1,4,9,1,3,3,5,3,7,6,8,1,5,5,3,7,6,7,3,8,5,5,3,4,5,5,4,4,6,5,3,7,9,2,1,4,5,8,9,4,5,8,5,2,5,1,9,2,0,2,2,8,9,7,2,4,2,6,1,8,1,4,7,2,8,8,8,5,9,9,9,6,7,6,8,3,9,7,7,0,2,1,8,1,8,0,8,2,8,1,0,1,7,9,3,7,0,3,2,9,2,1,5,2,9,3,7,9,2,4,1,6,7,1,0,5,3,8,7,1,1,9,2,8,8,7,5,8,0,9,9,4,2,4,6,1,7,3,2,2,9,3,8,6,6,0,1,9,8,0,0,1,1,4,1,0,1,8,0,3,7,0,8,9,6,6,3,4,0,5,8,9,1,8,6,7,9,9,7,9,0,9,1,3,4,4,3,7,3,6,1,2,8,1,2,4],[7,7,0,7,2,0,9,5,9,5,5,8,6,4,9,8,3,2,2,9,7,7,7,2,5,8,4,3,9,8,0,6,5,0,6,0,1,5,5,2,2,0,2,8,4,1,7,0,4,1,9,3,8,6,5,4,6,9,7,5,8,9,4,3,0,2,3,3,7,1,5,1,3,7,9,0,0,8,0,4,1,1,7,0,8,4,4,6,4,3,4,2,3,0,7,5,2,1,0,1,4,5,2,7,4,3,7,4,4,9,1,5,1,0,5,1,5,1,7,1,7,3,3,2,3,0,7,5,3,9,6,7,4,8,5,0,4,4,6,8,4,9,5,7,2,1,8,7,4,7,8,1,1,3,5,4,5,4,2,9,3,8,8,9,9,5,9,5,0,8,3,6,7,8,5,9,1,5,8,8,2,8,1,5,3,7,2,9,1,4,0,5,4,8,4,3,4,6,8,6],[4,3,4,3,4,9,3,7,4,3,5,8,4,7,6,7,6,8,6,7,4,6,4,8,7,1,4,3,9,4,9,3,8,5,8,4,4,3,1,0,7,9,0,1,8,6,0,4,4,7,3,0,5,0,9,2,1,3,5,0,9,6,5,9,1,5,3,7,9,7,9,8,8,0,9,6,6,1,0,3,8,5,5,6,5,4,8,8,9,6,0,9,2,7,8,6,3,4,5,2,3,5,0,1,7,1,7,5,2,9,0,3,6,6,1,4,0,9,2,2,7,5,3,2,2,3,0,7,7,5,9,2,0,1,3,9,4,2,5,9,3,5,4,0,1,5,4,4,6,8,6,4,3,1,6,6,4,6,3,4,3,5,6,6,8,2,5,3,4,0,2,8,6,8,0,9,3,4,3,9,2,1,3,6,4,1,4,9,9,7,5,3,4,1,1,3,5,6,8,0],[7,2,0,5,0,0,6,3,4,2,4,8,5,0,4,0,1,8,1,3,8,6,8,4,9,9,7,5,7,7,5,6,9,7,1,9,7,0,4,3,2,9,1,7,9,8,9,2,8,0,7,8,8,5,3,8,6,2,3,4,0,0,0,9,7,4,1,6,6,5,9,8,4,2,7,3,0,9,8,9,1,5,7,0,3,0,8,9,5,3,5,5,3,8,6,2,4,7,8,0,3,9,0,9,1,9,5,2,8,3,3,0,8,0,2,3,3,0,3,8,5,0,3,0,8,1,2,4,9,2,4,4,3,6,3,4,6,8,8,6,3,1,6,2,2,0,7,7,2,2,5,7,3,0,7,3,1,1,8,2,3,2,6,6,9,0,3,7,0,1,3,4,5,2,8,9,2,5,6,5,8,3,2,1,3,2,4,6,3,2,9,7,7,5,5,8,7,8,5,8],[2,8,2,9,0,0,8,5,7,6,0,5,9,4,6,2,6,3,8,0,7,9,9,6,7,6,4,4,7,1,2,9,0,6,8,2,8,8,7,6,4,9,1,3,4,0,7,2,5,5,4,2,7,5,9,4,2,5,8,9,7,3,0,9,1,8,3,0,8,3,6,2,2,9,7,8,1,4,3,6,1,7,1,8,3,2,2,5,7,1,6,6,6,6,5,7,4,9,9,4,2,5,6,6,7,3,7,8,9,0,5,0,7,8,9,2,0,1,9,7,4,5,6,0,1,1,0,7,2,9,1,6,7,9,3,4,4,0,4,3,2,1,4,1,9,3,4,9,6,5,9,1,1,5,3,2,8,3,2,1,5,3,7,2,5,2,8,9,4,2,5,6,4,9,8,5,4,4,5,0,9,6,3,0,1,7,5,1,2,9,2,9,4,2,1,9,6,9,1,1],[4,6,9,0,5,7,5,1,3,2,1,3,8,7,5,1,6,2,3,0,1,7,0,6,1,1,7,8,3,8,9,9,4,8,9,1,8,4,4,1,9,6,6,9,5,2,3,1,6,8,1,0,5,1,8,7,5,5,5,0,6,6,9,2,4,8,4,4,4,8,6,5,6,4,5,3,6,0,4,5,8,8,5,5,1,5,4,8,0,1,8,8,7,9,1,4,9,5,8,6,5,6,1,4,1,8,7,9,8,4,6,8,2,3,4,5,8,8,4,9,2,4,7,9,4,8,5,3,5,6,9,3,4,3,7,7,1,6,7,2,0,5,0,4,9,6,0,7,7,4,8,9,8,6,0,4,6,6,8,4,4,9,9,8,2,6,6,4,4,5,6,5,0,8,1,9,5,1,7,2,7,5,3,8,3,3,2,2,1,0,8,5,0,7,6,2,5,4,8,9],[9,6,6,1,5,8,3,2,1,0,6,1,7,9,9,3,2,1,7,6,4,5,3,6,2,9,0,9,3,9,8,4,7,7,6,2,7,1,6,8,1,2,9,0,1,8,3,6,2,0,4,8,7,7,4,9,7,4,8,2,5,9,7,3,6,5,7,5,6,4,3,9,8,5,1,2,5,7,0,7,9,4,7,7,1,1,8,8,8,9,3,3,0,0,8,8,7,6,3,5,2,6,6,2,1,7,4,9,6,4,8,6,0,6,3,2,7,3,2,5,2,5,1,2,7,9,0,6,7,3,1,1,2,9,6,5,7,0,4,3,7,3,1,9,9,4,1,8,0,4,6,2,9,7,7,9,8,9,5,8,5,9,9,7,8,5,4,7,8,9,1,7,4,4,6,5,1,0,5,1,4,1,5,5,0,2,4,9,4,2,7,9,1,8,8,9,6,2,7,6],[3,0,5,9,4,1,4,5,3,0,8,9,3,4,5,4,6,1,3,0,5,2,1,6,0,9,6,8,2,5,4,5,5,9,7,1,3,1,9,6,3,7,6,7,1,3,1,8,6,6,0,2,8,2,8,0,3,6,9,5,1,5,3,8,5,0,0,0,3,9,6,9,6,4,6,8,9,7,8,6,3,0,0,3,4,8,3,8,7,2,5,8,8,0,9,5,2,9,5,8,0,3,7,8,0,3,6,9,2,6,7,7,9,7,0,3,8,5,3,5,0,1,5,0,1,4,5,6,5,2,4,7,7,3,6,7,6,4,9,0,1,6,9,2,4,9,5,4,6,1,9,6,2,6,6,5,3,3,3,8,5,9,6,5,2,2,2,0,6,3,0,9,0,9,1,6,0,9,0,7,0,1,3,4,7,0,1,0,5,5,1,3,4,7,8,7,1,2,9,9],[6,0,9,6,1,2,4,2,3,4,9,3,7,4,9,4,6,3,7,2,8,8,5,4,7,3,3,8,7,3,9,3,5,0,9,6,3,5,8,6,9,9,2,6,6,3,3,2,6,0,6,6,0,3,1,7,8,6,7,6,9,6,9,6,7,1,3,2,6,3,8,8,3,2,6,1,6,9,5,2,1,2,9,1,7,2,0,6,8,7,2,0,4,3,6,3,4,9,5,3,5,5,1,0,8,7,1,6,9,6,8,2,0,9,6,8,1,6,4,2,6,8,4,2,1,0,5,8,2,2,1,7,7,2,7,5,1,0,3,2,6,2,5,9,3,1,7,7,9,3,1,5,1,5,9,4,5,4,2,9,6,5,8,6,7,5,3,1,7,7,3,6,1,0,5,4,3,2,3,5,5,4,2,8,9,2,4,7,8,7,6,7,2,5,3,2,2,6,5,0],[5,0,6,6,1,1,3,6,5,6,1,2,1,4,0,2,6,4,9,4,3,6,1,8,3,6,2,5,3,7,7,0,7,5,7,0,6,0,7,3,8,8,5,1,2,7,4,0,4,5,7,9,3,8,7,6,7,9,4,2,6,1,2,6,7,9,6,5,1,3,1,2,4,6,3,6,6,9,9,0,5,6,9,8,6,9,5,3,0,1,5,9,4,8,5,1,9,3,9,1,7,0,5,1,8,8,9,4,0,8,6,5,4,8,3,3,9,0,6,9,3,4,8,8,2,3,1,1,7,2,4,6,2,9,9,3,0,8,9,0,9,8,7,5,6,2,8,7,5,7,6,8,1,5,6,3,0,8,6,9,2,3,7,5,4,6,0,6,7,9,6,6,9,5,1,7,8,2,4,3,9,1,1,0,8,0,5,0,0,1,2,2,6,9,9,1,6,9,7,5]]
from heapq import heappush, heappop
def dijkstrasOnGridDirectly(grid,r,c,rStop,cStop):
    # Initialize seen, heap,minPathLengths, etc.
    seen = set()
    heap = [(grid[r][c],r,c)] # len,r,c
    numCol = len(grid[0])
    numRow = len(grid)
    while heap:
        currentBestPath,r,c = heappop(heap)
        print(grid)
        if r == rStop and c == cStop:
            print(grid)
            return grid[r][c]
        if (r,c) not in seen:
            seen.add((r,c))
            # Add right edge if possible
            if c < numCol-1:
                re,ce,weight = r,c+1,grid[r][c+1]
                if (re,ce) not in seen:
                    pathLength = currentBestPath+weight
                    if pathLength < grid[re][ce]+grid[r][c]:
                        grid[re][ce] = pathLength
                        heappush(heap,(pathLength,re,ce))
            # Add down edge if possible
            if r < numRow-1:
                re,ce,weight = r+1,c,grid[r+1][c]
                if (re,ce) not in seen:
                    pathLength = currentBestPath+weight
                    if pathLength < grid[re][ce]+grid[r][c]:
                        grid[re][ce] = pathLength
                        heappush(heap,(pathLength,re,ce))
r = 0
c = 0
rStop = len(grid)-1
cStop = len(grid[0])-1
minPathLength = dijkstrasOnGridDirectly(grid,r,c,rStop,cStop)
# print(grid)
minPathLength

# def buildGraph(grid):
#     graph = {}
#     numCol = len(grid[0])-1
#     numRow = len(grid)-1
#     nodeIndex = 0
#     for rowCount,row in enumerate(grid):
#         for colCount,weight in enumerate(row):
#             edges = [] # (nodeIndex,weight)
#             # Add right if possible
#             if colCount < numCol:
#                 edges.append((nodeIndex+1,row[colCount+1]))
#             # Add down if possible
#             if rowCount < numRow:
#                 edges.append((nodeIndex+numCol+1,grid[rowCount+1][colCount]))
#             # Add possible edges
#             graph[nodeIndex] = edges
#             nodeIndex+=1

#     return graph

def buildGraph(grid):
    graph = {}
    numCol = len(grid[0])
    numRow = len(grid)
    nodeIndex = 0
    for rowCount,row in enumerate(grid):
        for colCount,weight in enumerate(row):
            edges = [] # (nodeIndex,weight)
            # Add right if possible
            if colCount < numCol-1:
                edges.append((nodeIndex+1,row[colCount+1]))
            # Add down if possible
            if rowCount < numRow-1:
                edges.append((nodeIndex+numCol,grid[rowCount+1][colCount]))
            # Add possible edges
            graph[nodeIndex] = edges
            nodeIndex+=1

    return graph

def dijkstrasLoop(graph,source):

    # Keep tracks of seen and best path lengths
    # Loop through all nodes, since I dont want to implement a heap I will do the slower O(mn).This implementation is actually tghe same as the heap but with alonger sort time

    seen = [source]
    minPathLengths = {node:float('inf') for node in graph}
    minPathLengths[source] = 0
    while len(seen) < len(graph):
        minOfEdges = float('inf')
        for node in seen:
            edges = graph[node]
            for edge in edges:
                newNode,weight = edge
                if newNode not in seen:
                    pathLength = minPathLengths[node]+weight
                    if pathLength < minPathLengths[newNode]:
                        minPathLengths[newNode] = pathLength
                    if pathLength < minOfEdges:
                        minOfEdges = pathLength
                        minNode = newNode
        seen.append(minNode)
        print(len(seen),len(graph))

    return minPathLengths


from heapq import heappush, heappop
# a min Heap gives constant time look up for min and log(n) for sorting

# Use Dijkstra's because it's a weighted graph, (the key here is all of the weights are !=1 and as are >0)
# O(mn) , with a heap O(mlog(n))
# In terms of original notation, it would be O((nm)**2) or O(mn*log(mn))
def dijkstras(graph,source):

    # Keep tracks of seen and best path lengths
    # Keep track of shortest paths from previous iterations in heap, compare this to all of the new paths available with the addition of the next unseen node
    # Keep a heap or loop through all nodes, since I dont want to implement a heap I will do the slower O(mn)

    seen = set()
    heap = [(0,source)]
    minPathLengths = {node:float('inf') for node in graph}
    minPathLengths[source] = 0
    while heap:
        currentBestPath,node = heappop(heap)
        if node not in seen:
            seen.add(node)
            edges = graph[node]
            for edge in edges:
                newNode,weight = edge
                if newNode not in seen:
                    pathLength = currentBestPath+weight
                    if pathLength < minPathLengths[newNode]:
                        minPathLengths[newNode] = pathLength
                        heappush(heap,(pathLength,newNode))
    return minPathLengths

def dijkstrasSlower(graph,source):

    # Keep tracks of seen and best path lengths
    # Loop through all nodes, since I dont want to implement a heap I will do the slower O(mn).This implementation is actually tghe same as the heap but with alonger sort time

    seen = set()
    stack = [(0,source)]
    minPathLengths = {node:float('inf') for node in graph}
    minPathLengths[source] = 0
    while stack:
        lengthNode = min(stack)
        stack.remove(lengthNode)
        currentBestPath,node = lengthNode
        if node not in seen:
            seen.add(node)
            edges = graph[node]
            for edge in edges:
                newNode,weight = edge
                if newNode not in seen:
                    pathLength = currentBestPath+weight
                    if pathLength < minPathLengths[newNode]:
                        minPathLengths[newNode] = pathLength
                        stack.append((pathLength,newNode))
                        
    return minPathLengths

def dijkstrasFastReturn(graph,source,sink):

    # Keep tracks of seen and best path lengths
    # Keep track of shortest paths from previous iterations in heap, compare this to all of the new paths available with the addition of the next unseen node
    # Keep a heap or loop through all nodes, since I dont want to implement a heap I will do the slower O(mn)

    seen = set()
    heap = [(0,source)]
    minPathLengths = {node:float('inf') for node in graph}
    minPathLengths[source] = 0
    while heap:
        currentBestPath,node = heappop(heap)
        if node == sink:
            return currentBestPath
        if node not in seen:
            seen.add(node)
            edges = graph[node]
            for edge in edges:
                newNode,weight = edge
                if newNode not in seen:
                    pathLength = currentBestPath+weight
                    if pathLength < minPathLengths[newNode]:
                        minPathLengths[newNode] = pathLength
                        heappush(heap,(pathLength,newNode))


graph = buildGraph(grid)
sourceVal = grid[0][0]
sink = len(grid)*len(grid[0])-1
source = 0
minPathLength = dijkstrasFastReturn(graph,source,sink)
minPathLength+sourceVal

def buildGraph(grid):
    graph = {}
    numCol = len(grid[0])
    numRow = len(grid)
    nodeIndex = 0
    for rowCount,row in enumerate(grid):
        for colCount,weight in enumerate(row):
            edges = [] # (nodeIndex,weight)
            # Add right if possible
            if colCount < numCol-1:
                edges.append((nodeIndex+1,row[colCount+1]))
            # Add down if possible
            if rowCount < numRow-1:
                edges.append((nodeIndex+numCol,grid[rowCount+1][colCount]))
            # Add possible edges
            graph[nodeIndex] = edges
            nodeIndex+=1

    return graph


def constructEdges(grid,r,c,numRow,numCol):
    edges = [] # (r,c,weight)
    # Add right if possible
    if c < numCol-1:
        edges.append((r,c+1,grid[r][c+1]))
    # Add down if possible
    if r < numRow-1:
        edges.append((r+1,c,grid[r+1][c]))
    return edges

def dijkstrasOnGrid(grid,r,c):

    # Keep tracks of seen and best path lengths
    # Keep track of shortest paths from previous iterations in heap, compare this to all of the new paths available with the addition of the next unseen node
    # Keep a heap or loop through all nodes, since I dont want to implement a heap I will do the slower O(mn)

    seen = set()
    heap = [(0,r,c)] # len,r,c
    numCol = len(grid[0])
    numRow = len(grid)
    minPathLengths = {(r,c):float('inf') for r in range(numRow) for c in range(numCol)}
    minPathLengths[(r,c)] = grid[r][c]

    while heap:
        currentBestPath,r,c = heappop(heap)
        if (r,c) not in seen:
            seen.add((r,c))
            edges = constructEdges(grid,r,c,numRow,numCol)
            for re,ce,weight in edges:
                if (re,ce) not in seen:
                    pathLength = currentBestPath+weight
                    if pathLength < minPathLengths[(re,ce)]:
                        minPathLengths[(re,ce)] = pathLength
                        heappush(heap,(pathLength,re,ce))
    return minPathLengths

sink = (len(grid)-1,len(grid[0])-1)
minPathLengths = dijkstrasOnGrid(grid,0,0)
minPathLengths[sink]

[[1, 3, 1], [1, 5, 1], [4, 2, 1]]


The file contains an adjacency list representation of an undirected weighted graph with 200 vertices labeled 1 to 200.  Each row consists of the node tuples that are adjacent to that particular vertex along with the length of that edge. For example, the 6th row has 6 as the first entry indicating that this row corresponds to the vertex labeled 6. The next entry of this row "141,8200" indicates that there is an edge between vertex 6 and vertex 141 that has length 8200.  The rest of the pairs of this row indicate the other vertices adjacent to vertex 6 and the lengths of the corresponding edges.

Your task is to run Dijkstra's shortest-path algorithm on this graph, using 1 (the first vertex) as the source vertex, and to compute the shortest-path distances between 1 and every other vertex of the graph. If there is no path between a vertex vv and vertex 1, we'll define the shortest-path distance between 1 and vv to be 1000000. 

You should report the shortest-path distances to the following ten vertices, in order: 7,37,59,82,99,115,133,165,188,197.  You should encode the distances as a comma-separated string of integers. So if you find that all ten of these vertices except 115 are at distance 1000 away from vertex 1 and 115 is 2000 distance away, then your answer should be 1000,1000,1000,1000,1000,2000,1000,1000,1000,1000. Remember the order of reporting DOES MATTER, and the string should be in the same order in which the above ten vertices are given. The string should not contain any spaces.  Please type your answer in the space provided.

IMPLEMENTATION NOTES: This graph is small enough that the straightforward O(mn)O(mn) time implementation of Dijkstra's algorithm should work fine.  OPTIONAL: For those of you seeking an additional challenge, try implementing the heap-based version.  Note this requires a heap that supports deletions, and you'll probably need to maintain some kind of mapping between vertices and their positions in the heap.

In [135]:
G = {}
G.update({'s':{('s','v'):1,('s','w'):4}})
G.update({'v':{('v','s'):1,('v','w'):2,('v','t'):6}})
G.update({'w':{('w','v'):2,('w','s'):4,('w','t'):3}})
G.update({'t':{('t','v'):6,('t','w'):3}})

source = 's'
shortestPaths = dijkstrasAlgorithm(G,source,['t'])
print(shortestPaths)
shortestPaths = dijkstrasAlgorithmHeap(G,source,['t'])
shortestPaths

[6]


[6]

In [138]:
G = buildUndirectedWeightedGraph('dijkstraData.txt')
G

{1: {(1, 80): 982,
  (1, 163): 8164,
  (1, 170): 2620,
  (1, 145): 648,
  (1, 200): 8021,
  (1, 173): 2069,
  (1, 92): 647,
  (1, 26): 4122,
  (1, 140): 546,
  (1, 11): 1913,
  (1, 160): 6461,
  (1, 27): 7905,
  (1, 40): 9047,
  (1, 150): 2183,
  (1, 61): 9146,
  (1, 159): 7420,
  (1, 198): 1724,
  (1, 114): 508,
  (1, 104): 6647,
  (1, 30): 4612,
  (1, 99): 2367,
  (1, 138): 7896,
  (1, 169): 8700,
  (1, 49): 2437,
  (1, 125): 2909,
  (1, 117): 2597,
  (1, 55): 6399},
 2: {(2, 42): 1689,
  (2, 127): 9365,
  (2, 5): 8026,
  (2, 170): 9342,
  (2, 131): 7005,
  (2, 172): 1438,
  (2, 34): 315,
  (2, 30): 2455,
  (2, 26): 2328,
  (2, 6): 8847,
  (2, 11): 1873,
  (2, 17): 5409,
  (2, 157): 8643,
  (2, 159): 1397,
  (2, 142): 7731,
  (2, 182): 7908,
  (2, 93): 8177},
 3: {(3, 57): 1239,
  (3, 101): 3381,
  (3, 43): 7313,
  (3, 41): 7212,
  (3, 91): 2483,
  (3, 31): 3031,
  (3, 167): 3877,
  (3, 106): 6521,
  (3, 76): 7729,
  (3, 122): 9640,
  (3, 144): 285,
  (3, 44): 2165,
  (3, 6): 9006,
 

In [139]:
t1 = time.time()
destinations = [7,37,59,82,99,115,133,165,188,197]
source = 1
shortestPaths = dijkstrasAlgorithm(G,source,destinations)
shortestPaths,time.time()-t1

([2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068],
 0.2510499954223633)

In [140]:
t1 = time.time()
destinations = [7,37,59,82,99,115,133,165,188,197]
source = 1
shortestPaths = dijkstrasAlgorithmHeap(G,source,destinations)
shortestPaths,time.time()-t1

([2599, 2610, 2947, 2052, 2367, 2399, 2029, 2442, 2505, 3068],
 0.0010006427764892578)

In [150]:
import sys
 
    
class myHeap():
    
    def __init__(self,listInput):
        self.length = len(listInput)
        self.Heap = [0]*self.length
        
    def parent(self,pos):
        return pos//2
    
    def leftChild(self,pos):
        return 2*pos
    
    def rightChild(self,pos):
        return 2*pos+1
    
    def isLeaf(self,pos):
        return True if pos >= (self.length//2) and pos <=self.length else False
    
    def swap(self,pos1,pos2):
        self.Heap[pos1],self.Heap[pos2] = self.Heap[pos2],self.Heap[pos1]
    
    #def
    # to be continued 
    
class MinHeap:
 
    def __init__(self, maxsize):
        self.maxsize = maxsize
        self.size = 0
        self.Heap = [0]*(self.maxsize + 1)
        self.Heap[0] = -1 * sys.maxsize
        self.FRONT = 1
 
 
    # Function to heapify the node at pos
    def minHeapify(self, pos):
 
        # If the node is a non-leaf node and greater
        # than any of its child
        if not self.isLeaf(pos):
            if (self.Heap[pos] > self.Heap[self.leftChild(pos)] or
               self.Heap[pos] > self.Heap[self.rightChild(pos)]):
 
                # Swap with the left child and heapify
                # the left child
                if self.Heap[self.leftChild(pos)] < self.Heap[self.rightChild(pos)]:
                    self.swap(pos, self.leftChild(pos))
                    self.minHeapify(self.leftChild(pos))
 
                # Swap with the right child and heapify
                # the right child
                else:
                    self.swap(pos, self.rightChild(pos))
                    self.minHeapify(self.rightChild(pos))
 
    # Function to insert a node into the heap
    def insert(self, element):
        if self.size >= self.maxsize :
            return
        self.size+= 1
        self.Heap[self.size] = element
 
        current = self.size
 
        while self.Heap[current] < self.Heap[self.parent(current)]:
            self.swap(current, self.parent(current))
            current = self.parent(current)
 
    # Function to print the contents of the heap
    def Print(self):
        for i in range(1, (self.size//2)+1):
            print(" PARENT : "+ str(self.Heap[i])+" LEFT CHILD : "+
                                str(self.Heap[2 * i])+" RIGHT CHILD : "+
                                str(self.Heap[2 * i + 1]))
 
    # Function to build the min heap using
    # the minHeapify function
    def minHeap(self):
 
        for pos in range(self.size//2, 0, -1):
            self.minHeapify(pos)
 
    # Function to remove and return the minimum
    # element from the heap
    def remove(self):
 
        popped = self.Heap[self.FRONT]
        self.Heap[self.FRONT] = self.Heap[self.size]
        self.size-= 1
        self.minHeapify(self.FRONT)
        return popped

In [148]:
sys.maxsize

9223372036854775807

In [159]:
import sys
 
class myHeap():
    
    def __init__(self,listInput):
        self.length = len(listInput)
        self.Heap = [0]*self.length
        
    def parent(self,pos):
        return pos//2
    
    def leftChild(self,pos):
        return 2*pos
    
    def rightChild(self,pos):
        return 2*pos+1
    
    def isLeaf(self,pos):
        return True if pos >= (self.length//2) and pos <=self.length else False
    
    def swap(self,pos1,pos2):
        self.Heap[pos1],self.Heap[pos2] = self.Heap[pos2],self.Heap[pos1]
    
    #def
    # to be continued 
    
    
    
class MinHeap:
 
    def __init__(self, maxsize):
        self.maxsize = maxsize
        self.size = 0
        self.Heap = [0]*(self.maxsize + 1)
        self.Heap[0] = -1 * sys.maxsize
        self.FRONT = 1
 
    # Function to return the position of
    # parent for the node currently
    # at pos
    def parent(self, pos):
        return pos//2
 
    # Function to return the position of
    # the left child for the node currently
    # at pos
    def leftChild(self, pos):
        return 2 * pos
 
    # Function to return the position of
    # the right child for the node currently
    # at pos
    def rightChild(self, pos):
        return (2 * pos) + 1
 
    # Function that returns true if the passed
    # node is a leaf node
    def isLeaf(self, pos):
        if pos >= (self.size//2) and pos <= self.size:
            return True
        return False
 
    # Function to swap two nodes of the heap
    def swap(self, fpos, spos):
        self.Heap[fpos], self.Heap[spos] = self.Heap[spos], self.Heap[fpos]
 
    # Function to heapify the node at pos
    def minHeapify(self, pos):
 
        # If the node is a non-leaf node and greater
        # than any of its child
        if not self.isLeaf(pos):
            if (self.Heap[pos] > self.Heap[self.leftChild(pos)] or
               self.Heap[pos] > self.Heap[self.rightChild(pos)]):
 
                # Swap with the left child and heapify
                # the left child
                if self.Heap[self.leftChild(pos)] < self.Heap[self.rightChild(pos)]:
                    self.swap(pos, self.leftChild(pos))
                    self.minHeapify(self.leftChild(pos))
 
                # Swap with the right child and heapify
                # the right child
                else:
                    self.swap(pos, self.rightChild(pos))
                    self.minHeapify(self.rightChild(pos))
 
    # Function to insert a node into the heap
    def insert(self, element):
        if self.size >= self.maxsize :
            return
        self.size+= 1
        self.Heap[self.size] = element
 
        current = self.size
 
        while self.Heap[current] < self.Heap[self.parent(current)]:
            self.swap(current, self.parent(current))
            current = self.parent(current)
 
    # Function to print the contents of the heap
    def Print(self):
        for i in range(1, (self.size//2)+1):
            print(" PARENT : "+ str(self.Heap[i])+" LEFT CHILD : "+
                                str(self.Heap[2 * i])+" RIGHT CHILD : "+
                                str(self.Heap[2 * i + 1]))
 
    # Function to build the min heap using
    # the minHeapify function
    def minHeap(self):
 
        for pos in range(self.size//2, 0, -1):
            self.minHeapify(pos)
 
    # Function to remove and return the minimum
    # element from the heap
    def remove(self):
 
        popped = self.Heap[self.FRONT]
        self.Heap[self.FRONT] = self.Heap[self.size]
        self.size-= 1
        self.minHeapify(self.FRONT)
        return popped
 
# Driver Code
if __name__ == "__main__":
     
    print('The minHeap is ')
    minHeap = MinHeap(15)
    minHeap.insert(5)
    minHeap.insert(3)
    minHeap.insert(17)
    minHeap.insert(10)
    minHeap.insert(84)
    minHeap.insert(19)
    minHeap.insert(6)
    minHeap.insert(22)
    minHeap.insert(9)
    minHeap.minHeap()
 
    minHeap.Print()
    print("The Min val is " + str(minHeap.remove()))

The minHeap is 
 PARENT : 3 LEFT CHILD : 5 RIGHT CHILD : 6
 PARENT : 5 LEFT CHILD : 9 RIGHT CHILD : 84
 PARENT : 6 LEFT CHILD : 19 RIGHT CHILD : 17
 PARENT : 9 LEFT CHILD : 22 RIGHT CHILD : 10
The Min val is 3


In [158]:
# minHeap = MinHeap(15)


In [160]:
minHeap.Heap

[-9223372036854775807, 5, 9, 6, 10, 84, 19, 17, 22, 10, 0, 0, 0, 0, 0, 0]